In [1]:
from bs4 import BeautifulSoup
import mammoth
import json
import logging
import os
import socket
import subprocess
import sys
import time
import requests
from nltk.tokenize import word_tokenize
from nltk.metrics import jaccard_distance
import string
import psutil
from nltk.tokenize import TweetTokenizer
import re, csv
from nltk.stem.wordnet import WordNetLemmatizer
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
import numpy as np
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from emoji.unicode_codes import UNICODE_EMOJI
import emoji
import glob, re, pickle, math, statistics
from collections import Counter
from nltk import sent_tokenize, ngrams
from keras.models import load_model
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import accuracy_score
from random import shuffle
from decimal import *

try:
    from urlparse import urlparse
except ImportError:
    from urllib.parse import urlparse

Using TensorFlow backend.


In [2]:
'''
This class processes interview transcripts and generates question-answer pairs
NOTE: The pre processing requires a timestamp just before the beginning of a conversation. Some of the transcripts have
the timestamps, some don't. In case, a transcript doesn't have the timestamp, it is required to put the timestamp
manually there in order to use the following procedure
'''
class Process:
    def __init__(self, corpus):
        with open(corpus, "rb") as docx_file:
            self.html = mammoth.convert_to_html(docx_file).value

        self.processed_html = ""
        self.ques_ans = []

    def process_html(self):
        soup = BeautifulSoup(self.html, 'html.parser')
        paragraphs = soup.find_all('p')
        initial_time_index = -1

        for i in range(1, len(paragraphs)):
            try:
                if paragraphs[i].get_text().split()[0].count(':') == 2:
                    initial_time_index = i
                    break
            except:
                continue

        for i in range(initial_time_index + 1, len(paragraphs)):
            if paragraphs[i].get_text().split()[0].count(':') == 2:
                continue
            else:
                self.processed_html += str(paragraphs[i])

    def extract_ques_ans(self):
        soup = BeautifulSoup(self.processed_html, 'html.parser')

        for paragraph in soup.find_all('p'):
            if paragraph.get_text().split()[0].count(':') == 2:
                paragraph.extract()

        paragraphs = soup.find_all('p')
        checker = []

        for paragraph in paragraphs:
            if paragraph.find('strong') != None:
                checker.append(1)
            else:
                checker.append(0)

        expected = 1
        temp_list = []
        for i in range(len(paragraphs)):
            if checker[i] == expected:
                temp_list.append(paragraphs[i].get_text())
                expected ^= 1
            else:
                temp_list[len(temp_list) - 1] = temp_list[len(temp_list) - 1] + ' ' + paragraphs[i].get_text()

        for i in range(0,len(temp_list),2):
            ques = temp_list[i].replace(temp_list[i].split()[0], '').strip()
            if i + 1 == len(temp_list):
                ans = ''
            else:
                ans = temp_list[i + 1].replace(temp_list[i + 1].split()[0], '').strip()

            self.ques_ans.append((ques, ans))

        return self.ques_ans

In [3]:

# ********* Initializations ********* #
lmtzr = WordNetLemmatizer()
hedge_words = []
discourse_markers = []
THRESHOLD = 0.8

# ********* Python Wrapper for Stanford CoreNLP ********* #
# ********* Class definition implemented from "https://github.com/Lynten/stanford-corenlp" with slight modifications ********* #

class StanfordCoreNLP:
    def __init__(self, path_or_host, port=None, memory='4g', lang='en', timeout=5000, quiet=True,
                 logging_level=logging.WARNING, max_retries=100):
        self.path_or_host = path_or_host
        self.port = port
        self.memory = memory
        self.lang = lang
        self.timeout = timeout
        self.quiet = quiet
        self.logging_level = logging_level

        logging.basicConfig(level=self.logging_level)

        # Check args
        self._check_args()

        if path_or_host.startswith('http'):
            self.url = path_or_host + ':' + str(port)
            logging.info('Using an existing server {}'.format(self.url))
        else:

            # Check Java
            if not subprocess.call(['java', '-version'], stdout=subprocess.PIPE, stderr=subprocess.STDOUT) == 0:
                raise RuntimeError('Java not found.')

            # Check if the dir exists
            if not os.path.isdir(self.path_or_host):
                raise IOError(str(self.path_or_host) + ' is not a directory.')
            directory = os.path.normpath(self.path_or_host) + os.sep
            self.class_path_dir = directory

            # Check if the language specific model file exists
            switcher = {
                'en': 'stanford-corenlp-[0-9].[0-9].[0-9]-models.jar',
                'zh': 'stanford-chinese-corenlp-[0-9][0-9][0-9][0-9]-[0-9][0-9]-[0-9][0-9]-models.jar',
                'ar': 'stanford-arabic-corenlp-[0-9][0-9][0-9][0-9]-[0-9][0-9]-[0-9][0-9]-models.jar',
                'fr': 'stanford-french-corenlp-[0-9][0-9][0-9][0-9]-[0-9][0-9]-[0-9][0-9]-models.jar',
                'de': 'stanford-german-corenlp-[0-9][0-9][0-9][0-9]-[0-9][0-9]-[0-9][0-9]-models.jar',
                'es': 'stanford-spanish-corenlp-[0-9][0-9][0-9][0-9]-[0-9][0-9]-[0-9][0-9]-models.jar'
            }
            jars = {
                'en': 'stanford-corenlp-x.x.x-models.jar',
                'zh': 'stanford-chinese-corenlp-yyyy-MM-dd-models.jar',
                'ar': 'stanford-arabic-corenlp-yyyy-MM-dd-models.jar',
                'fr': 'stanford-french-corenlp-yyyy-MM-dd-models.jar',
                'de': 'stanford-german-corenlp-yyyy-MM-dd-models.jar',
                'es': 'stanford-spanish-corenlp-yyyy-MM-dd-models.jar'
            }
            if len(glob.glob(directory + switcher.get(self.lang))) <= 0:
                raise IOError(jars.get(
                    self.lang) + ' not exists. You should download and place it in the ' + directory + ' first.')

            self.port = 9999

            # Start native server
            logging.info('Initializing native server...')
            cmd = "java"
            java_args = "-Xmx{}".format(self.memory)
            java_class = "edu.stanford.nlp.pipeline.StanfordCoreNLPServer"
            class_path = '"{}*"'.format(directory)

            args = [cmd, java_args, '-cp', class_path, java_class, '-port', str(self.port)]

            args = ' '.join(args)

            logging.info(args)

            # Silence
            with open(os.devnull, 'w') as null_file:
                out_file = None
                if self.quiet:
                    out_file = null_file

                self.p = subprocess.Popen(args, shell=True, stdout=out_file, stderr=subprocess.STDOUT)
                logging.info('Server shell PID: {}'.format(self.p.pid))

            self.url = 'http://localhost:' + str(self.port)

        # Wait until server starts
        sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        host_name = urlparse(self.url).hostname
        time.sleep(1)  # OSX, not tested
        trial = 1
        while sock.connect_ex((host_name, self.port)):
            if trial > max_retries:
                raise ValueError('Corenlp server is not available')
            logging.info('Waiting until the server is available.')
            trial += 1
            time.sleep(1)
        logging.info('The server is available.')

    def __enter__(self):
        return self

    def __exit__(self, exc_type, exc_val, exc_tb):
        self.close()

    def close(self):
        logging.info('Cleanup...')
        if hasattr(self, 'p'):
            try:
                parent = psutil.Process(self.p.pid)
            except psutil.NoSuchProcess:
                logging.info('No process: {}'.format(self.p.pid))
                return

            if self.class_path_dir not in ' '.join(parent.cmdline()):
                logging.info('Process not in: {}'.format(parent.cmdline()))
                return

            children = parent.children(recursive=True)
            for process in children:
                logging.info('Killing pid: {}, cmdline: {}'.format(process.pid, process.cmdline()))
                # process.send_signal(signal.SIGTERM)
                process.kill()

            logging.info('Killing shell pid: {}, cmdline: {}'.format(parent.pid, parent.cmdline()))
            # parent.send_signal(signal.SIGTERM)
            parent.kill()

    def annotate(self, text, properties=None):
        if sys.version_info.major >= 3:
            text = text.encode('utf-8')

        r = requests.post(self.url, params={'properties': str(properties)}, data=text,
                          headers={'Connection': 'close'})
        return r.text

    def tregex(self, sentence, pattern):
        tregex_url = self.url + '/tregex'
        r_dict = self._request(tregex_url, "tokenize,ssplit,depparse,parse", sentence, pattern=pattern)
        return r_dict

    def tokensregex(self, sentence, pattern):
        tokensregex_url = self.url + '/tokensregex'
        r_dict = self._request(tokensregex_url, "tokenize,ssplit,depparse", sentence, pattern=pattern)
        return r_dict

    def semgrex(self, sentence, pattern):
        semgrex_url = self.url + '/semgrex'
        r_dict = self._request(semgrex_url, "tokenize,ssplit,depparse", sentence, pattern=pattern)
        return r_dict

    def word_tokenize(self, sentence, span=False):
        r_dict = self._request('ssplit,tokenize', sentence)
        tokens = [token['originalText'] for s in r_dict['sentences'] for token in s['tokens']]

        # Whether return token span
        if span:
            spans = [(token['characterOffsetBegin'], token['characterOffsetEnd']) for s in r_dict['sentences'] for token
                     in s['tokens']]
            return tokens, spans
        else:
            return tokens

    def pos_tag(self, sentence):
        r_dict = self._request(self.url, 'pos', sentence)
        words = []
        tags = []
        for s in r_dict['sentences']:
            for token in s['tokens']:
                words.append(token['originalText'])
                tags.append(token['pos'])
        return list(zip(words, tags))

    def ner(self, sentence):
        r_dict = self._request(self.url, 'ner', sentence)
        words = []
        ner_tags = []
        for s in r_dict['sentences']:
            for token in s['tokens']:
                words.append(token['originalText'])
                ner_tags.append(token['ner'])
        return list(zip(words, ner_tags))

    def parse(self, sentence):
        r_dict = self._request(self.url, 'pos,parse', sentence)
        return [s['parse'] for s in r_dict['sentences']][0]

    def dependency_parse(self, text):
        r_dict = self._request(self.url, 'depparse', text)
        ls = []
        for s in r_dict['sentences']:
            tmp = []
            for dep in s['basicDependencies']:
                tmp.append((dep['dep'], dep['governorGloss'], dep['dependentGloss']))
            ls.append(tmp)
        return ls

    def coref(self, text):
        r_dict = self._request('coref', text)

        corefs = []
        for k, mentions in r_dict['corefs'].items():
            simplified_mentions = []
            for m in mentions:
                simplified_mentions.append((m['sentNum'], m['startIndex'], m['endIndex'], m['text']))
            corefs.append(simplified_mentions)
        return corefs

    def switch_language(self, language="en"):
        self._check_language(language)
        self.lang = language

    def _request(self, url, annotators=None, data=None, *args, **kwargs):
        if sys.version_info.major >= 3:
            data = data.encode('utf-8')

        properties = {'annotators': annotators, 'outputFormat': 'json'}
        params = {'properties': str(properties), 'pipelineLanguage': self.lang}
        if 'pattern' in kwargs:
            params = {"pattern": kwargs['pattern'], 'properties': str(properties), 'pipelineLanguage': self.lang}

        logging.info(params)
        r = requests.post(url, params=params, data=data, headers={'Connection': 'close'})
        r_dict = json.loads(r.text)

        return r_dict

    def _check_args(self):
        self._check_language(self.lang)
        if not re.match('\dg', self.memory):
            raise ValueError('memory=' + self.memory + ' not supported. Use 4g, 6g, 8g and etc. ')

    def _check_language(self, lang):
        if lang not in ['en', 'zh', 'ar', 'fr', 'de', 'es']:
            raise ValueError('lang=' + self.lang + ' not supported. Use English(en), Chinese(zh), Arabic(ar), '
                                                   'French(fr), German(de), Spanish(es).')

# ********* Load Lexicons ********* #

def load_lexicons():
    with open("resources/hedge_words.txt", "r") as f:
        for line in f:
            if '#' in line:
                continue
            elif line.strip() != "":
                hedge_words.append(line.strip())

    with open("resources/discourse_markers.txt", "r") as f:
        for line in f:
            if '#' in line:
                continue
            elif line.strip() != "":
                discourse_markers.append(line.strip())

# ********* Initialize CoreNLP ********* #
# Download (if you haven't already) the zip file from this link: https://drive.google.com/open?id=1ROwL9fY1-BJ57O5wkgMk4UAxWKretfTk
# Unzip the file in the resources folder
path = os.path.abspath('resources/stanford-corenlp-full-2018-02-27/')
nlp = StanfordCoreNLP(path)
load_lexicons()

# ********* Disambiguate Hedge Terms ********* #
# ********* Returns true if (hedge) token is true hedge term, otherwise, returns false ********* #

def IsTrueHedgeTerm(hedge, text):
    exclude = set(string.punctuation)

    if hedge == "assume":
        parse_trees = nlp.dependency_parse(text)
        tree = parse_trees[0]
        for pair in tree:
            if pair[0] == "ccomp" and lmtzr.lemmatize(pair[1], 'v') == hedge:
                return True
        return False

    elif hedge == "appear":
        parse_trees = nlp.dependency_parse(text)
        tree = parse_trees[0]
        for pair in tree:
            if (pair[0] in ["xcomp", "ccomp"]) and lmtzr.lemmatize(pair[1], 'v') == hedge:
                return True
        return False

    elif hedge == "suppose":
        parse_trees = nlp.dependency_parse(text)
        tree = parse_trees[0]
        for pair in tree:
            if pair[0] == "xcomp" and lmtzr.lemmatize(pair[1], 'v') == hedge:
                token = pair[2]
                for temp in tree:
                    if temp[0] == "mark" and temp[1] == token and temp[2] == "to":
                        return False
        return True

    elif hedge == "tend":
        parse_trees = nlp.dependency_parse(text)
        tree = parse_trees[0]
        for pair in tree:
            if pair[0] == "xcomp" and lmtzr.lemmatize(pair[1], 'v') == hedge:
                return True
        return False

    elif hedge == "should":
        parse_trees = nlp.dependency_parse(text)
        tree = parse_trees[0]
        for pair in tree:
            if pair[0] == "aux" and pair[2] == hedge:
                token = pair[1]
                for temp in tree:
                    if temp[1] == token and temp[2] == "have":
                        return False
        return True

    elif hedge == "likely":
        parse_trees = nlp.dependency_parse(text)
        tree = parse_trees[0]
        for pair in tree:
            if pair[2] == hedge:
                token = pair[1]
                for temp in tree:
                    if temp[2] == token and temp[1] != "ROOT":
                        tag = nlp.pos_tag(temp[1])
                        if tag[0][1] in ["NN", "NNS", "NNP", "NNPS"]:
                            return False
        return True

    elif hedge == "rather":
        s = ''.join(ch for ch in text if ch not in exclude)
        list_of_words = s.split()
        next_word = list_of_words[list_of_words.index(hedge) + 1]
        if next_word == 'than':
            return False
        else:
            return True

    elif hedge == "think":
        words = word_tokenize(text)
        for i in range(len(words) - 1):
            if words[i] == hedge:
                tag = nlp.pos_tag(words[i + 1])
                if tag[0][1] == "IN":
                    return False
                    break
        return True

    elif hedge in ["feel", "suggest", "believe", "consider", "doubt", "guess", "presume", "hope"]:
        parse_trees = nlp.dependency_parse(text)
        tree = parse_trees[0]
        isRoot = False
        hasNSubj = False
        for pair in tree:
            if lmtzr.lemmatize(pair[2]) in [hedge] and pair[1] == "ROOT":
                isRoot = True
            elif lmtzr.lemmatize(pair[1]) in [hedge] and pair[0] == "nsubj":
                token = lmtzr.lemmatize(pair[1])
                subject = pair[2]
                hasNSubj = True

        if isRoot and hasNSubj:
            tags = nlp.pos_tag(text)
            status1 = False
            status2 = False
            for tag in tags:
                if lmtzr.lemmatize(tag[0]) == token and tag[1] in ["VB", "VBD", "VBG", "VBN", "VBP", "VBZ"]:
                    status1 = True
                if subject.lower() in ["i", "we"]:
                    status2 = True
            if status1 and status2:
                return True
            else:
                return False


# ********* Determines if a sentence is hedged sentence or not ********* #
# ********* Returns true if sentence is hedged sentence, otherwise, returns false ********* #

def IsHedgedSentence(text):
    exclude = set(string.punctuation)
    text = text.lower()

    if "n't" in text:
        text = text.replace("n't", " not")
    elif "n’t" in text:
        text = text.replace("n’t", " not")

    tokenized = word_tokenize(text)
    phrases = []
    status = False

    # Determine the n-grams of the given sentence
    for i in range(1,6):
        phrases += ngrams(tokenized, i)

    # Determine whether hedge terms are present in the sentence and find out if they are true hedge terms
    for hedge in hedge_words:
        if hedge in tokenized and IsTrueHedgeTerm(hedge, text):
            status = True
            break


    # Determine whether disocurse markers are present in the n-grams
    # Use Jaccard distance for measuring similarity
    if not status:
        for A in discourse_markers:
            for B in phrases:
                if (1 - jaccard_distance(set(A.split()), set(list(B)))) >= THRESHOLD:
                    status = True
                    break

            if status:
                break

    return status


In [4]:
feature_dimension = 29

# ********* Load Lexicons ********* #
bingliu_mpqa = {}
nrc_emotion = {}
nrc_affect_intensity = {}
nrc_hashtag_emotion = {}
afinn = {}
ratings = {}
stopwords = []
slangs = {}
negated = {}
emoticons = []

# Vader
analyzer = SentimentIntensityAnalyzer()

def load_lexicons():
    # Ratings by Warriner et al. (2013)
    with open('lexicons/Ratings_Warriner_et_al.csv', 'r') as f:
        reader = csv.reader(f)
        rows = list(reader)
    for i in range(1, len(rows)):
        # Normalize values
        valence = (float(rows[i][2]) - 1.0)/(9.0-1.0)
        arousal = (float(rows[i][5]) - 1.0)/(9.0-1.0)
        dominance = (float(rows[i][8]) - 1.0)/(9.0-1.0)
        ratings[rows[i][1]] = {"Valence": valence, "Arousal": arousal, "Dominance": dominance}


    # NRC Emotion Lexicon (2014)
    with open('lexicons/NRC-emotion-lexicon-wordlevel-v0.92.txt', 'r') as f:
        f.readline()
        for line in f:
            splitted = line.strip().split('\t')
            if splitted[0] not in nrc_emotion:
                nrc_emotion[splitted[0]] = {'anger': float(splitted[1]),
                                                    'disgust': float(splitted[3]),
                                                    'fear': float(splitted[4]),
                                                    'joy': float(splitted[5]),
                                                    'sadness': float(splitted[8]),
                                                    'surprise': float(splitted[9])}

    # NRC Affect Intensity (2018)
    with open('lexicons/nrc_affect_intensity.txt', 'r') as f:
        f.readline()
        for line in f:
            splitted = line.strip().split('\t')
            if splitted[0] not in nrc_affect_intensity:
                nrc_affect_intensity[splitted[0]] = {'anger': float(splitted[1]),
                                                    'disgust': float(splitted[3]),
                                                    'fear': float(splitted[4]),
                                                    'joy': float(splitted[5]),
                                                    'sadness': float(splitted[8]),
                                                    'surprise': float(splitted[9])}

    # NRC Hashtag Emotion Lexicon (2015)
    with open('lexicons/NRC-Hashtag-Emotion-Lexicon-v0.2.txt', 'r') as f:
        f.readline()
        for line in f:
            splitted = line.strip().split('\t')
            splitted[0] = splitted[0].replace('#','')
            if splitted[0] not in nrc_hashtag_emotion:
                nrc_hashtag_emotion[splitted[0]] = {'anger': float(splitted[1]),
                                                    'disgust': float(splitted[3]),
                                                    'fear': float(splitted[4]),
                                                    'joy': float(splitted[5]),
                                                    'sadness': float(splitted[8]),
                                                    'surprise': float(splitted[9])}


    # BingLiu (2004) and MPQA (2005)
    with open('lexicons/BingLiu.txt', 'r') as f:
        for line in f:
            splitted = line.strip().split('\t')
            if splitted[0] not in bingliu_mpqa:
                bingliu_mpqa[splitted[0]] = splitted[1]
    with open('lexicons/mpqa.txt', 'r') as f:
        for line in f:
            splitted = line.strip().split('\t')
            if splitted[0] not in bingliu_mpqa:
                bingliu_mpqa[splitted[0]] = splitted[1]


    with open('lexicons/AFINN-en-165.txt', 'r') as f:
        for line in f:
            splitted = line.strip().split('\t')
            if splitted[0] not in afinn:
                score = float(splitted[1])
                normalized_score = (score - (-5)) / (5-(-5))
                afinn[splitted[0]] = normalized_score


    with open('lexicons/stopwords.txt', 'r') as f:
        for line in f:
            stopwords.append(line.strip())

    with open('lexicons/slangs.txt', 'r') as f:
        for line in f:
            splitted = line.strip().split(',', 1)
            slangs[splitted[0]] = splitted[1]

    with open('lexicons/negated_words.txt', 'r') as f:
        for line in f:
            splitted = line.strip().split(',', 1)
            negated[splitted[0]] = splitted[1]

    with open('lexicons/emoticons.txt', 'r') as f:
        for line in f:
            emoticons.append(line.strip())
load_lexicons()

# ********* Helper Functions ********* #

def clean_texts(texts):
    cleaned_tweets = []
    hash_emos = []

    for text in texts:
        hash_emo = []
        text = re.sub('(!){2,}', ' <!repeat> ', text)
        text = re.sub('(\?){2,}', ' <?repeat> ', text)

        # Tokenize using tweet tokenizer
        tokenizer = TweetTokenizer(strip_handles=False, reduce_len=True)
        tokens = tokenizer.tokenize(text.lower())
        lemmatizer = WordNetLemmatizer()


        # Emojis and emoticons
        if text_has_emoji(text):
            temp = []
            for word in tokens:
                if char_is_emoji(word):
                    hash_emo.append(UNICODE_EMOJI[word])
                elif word in emoticons:
                    hash_emo.append(word)
                else:
                    temp.append(word)
            tokens = temp

        # Hashtags
        temp = []
        for word in tokens:
            if '#' in word:
                word = word.replace('#','')
                hash_emo.append(word)
            else:
                temp.append(word)
        tokens = temp

        # Replace slangs and negated words
        temp = []
        for word in tokens:
            if word in slangs:
                temp += slangs[word].split()
            elif word in negated:
                temp += negated[word].split()
            else:
                temp.append(word)
        tokens = temp

        # Replace user names
        tokens = ['<user>'  if '@' in word else word for word in tokens]

        #Replace numbers
        tokens = ['<number>' if word.isdigit() else word for word in tokens]

        # Remove urls
        tokens = ['' if 'http' in word else word for word in tokens]

        # Lemmatize
        #tokens = [lemmatizer.lemmatize(word) for word in tokens]

        # Remove stop words
        tokens = [word for word in tokens if word not in stopwords]

        # Remove tokens having length 1
        tokens = [word for word in tokens if word != '' and len(word) > 1]

        cleaned_tweets.append(tokens)
        hash_emos.append(hash_emo)

    return cleaned_tweets, hash_emos


# This function returns a n-dimensional feature vector
def feature_generation(texts, hashtags):
    feature_vectors = []

    for i in range(len(texts)):
        feats = [0] * feature_dimension
        for word in texts[i]:
            # Warriner er al.
            if word in ratings:
                feats[0] += ratings[word]['Valence']
                feats[1] += ratings[word]['Arousal']
                feats[2] += ratings[word]['Dominance']

            # Vader Sentiment
            polarity_scores = analyzer.polarity_scores(word)
            feats[3] += polarity_scores['pos']
            feats[4] += polarity_scores['neg']
            feats[5] += polarity_scores['neu']

            # NRC Emotion
            if word in nrc_emotion:
                feats[6] += nrc_emotion[word]['anger']
                feats[7] += nrc_emotion[word]['disgust']
                feats[8] += nrc_emotion[word]['fear']
                feats[9] += nrc_emotion[word]['joy']
                feats[10] += nrc_emotion[word]['sadness']
                feats[11] += nrc_emotion[word]['surprise']

            # NRC Affect Intensity
            if word in nrc_affect_intensity:
                feats[12] += nrc_affect_intensity[word]['anger']
                feats[13] += nrc_affect_intensity[word]['disgust']
                feats[14] += nrc_affect_intensity[word]['fear']
                feats[15] += nrc_affect_intensity[word]['joy']
                feats[16] += nrc_affect_intensity[word]['sadness']
                feats[17] += nrc_affect_intensity[word]['surprise']

            # AFINN
            if word in afinn:
                feats[18] += float(afinn[word])

            # BingLiu and MPQA
            if word in bingliu_mpqa:
                if bingliu_mpqa[word] == 'positive':
                    feats[19] += 1
                else:
                    feats[20] += 1


        count = len(texts[i])
        if count == 0:
            count = 1
        newArray = np.array(feats)/count
        feats = list(newArray)

        # Presence of consecutive exclamation mark or question mark
        for word in texts[i]:
            if word == '<!REPEAT>':
                feats[21] = 1
            elif word == '<?REPEAT>':
                feats[22] = 1

        for word in hashtags[i]:
            #NRC Hashtag Emotion
            if word in nrc_hashtag_emotion:
                feats[23] += nrc_hashtag_emotion[word]['anger']
                feats[24] += nrc_hashtag_emotion[word]['disgust']
                feats[25] += nrc_hashtag_emotion[word]['fear']
                feats[26] += nrc_hashtag_emotion[word]['joy']
                feats[27] += nrc_hashtag_emotion[word]['sadness']
                feats[28] += nrc_hashtag_emotion[word]['surprise']

        feature_vectors.append(feats)
    return np.array(feature_vectors)


def create_tokenizer(lines):
    tokenizer = Tokenizer()
    tokenizer.fit_on_texts(lines)
    return tokenizer


def max_length(lines):
    return max([len(s) for s in lines])


def encode_text(tokenizer, lines, length):
    encoded = tokenizer.texts_to_sequences(lines)
    padded = pad_sequences(encoded, maxlen=length, padding='post')
    return padded

def char_is_emoji(character):
    return character in emoji.UNICODE_EMOJI

def text_has_emoji(text):
    for character in text:
        if character in emoji.UNICODE_EMOJI:
            return True
    return False


In [5]:
# Returns negative if given sentence contains negative emotion, otherwise returns positive
# Input: Sentence or Text
# Output: "Negative" or "Positive"
def get_emotion(sentence):
    cleaned_sentences, hash_emos = clean_texts([sentence])
    features = feature_generation(cleaned_sentences, hash_emos)

    evalX = encode_text(tokenizer_tweets, cleaned_sentences, max_tweet_length)
    encoded_hash_emo = encode_text(tokenizer_hash_emo, hash_emos, max_hash_emo_length)

    predictedY = model.predict([evalX,encoded_hash_emo,features])
    predicted_classes = lb.inverse_transform(predictedY)
    if predicted_classes[0] in NEGATIVE_EMOTIONS:
        return "negative"
    else:
        return "positive"

In [55]:
# Initializations
hedges = []
markers = []
cues = []
boosters = []
NEGATIVE_EMOTIONS = ["anger", 'fear', "sadness"]
QUES_TYPES = ["what", "when", "where", "who", "why", "how", "yesno", "mixed"]

model = load_model('models/model.h5')
with open('models/variables.p', 'rb') as f:
    lb, tokenizer_tweets, max_tweet_length, tokenizer_hash_emo, max_hash_emo_length, embeddings_index = pickle.load(f)


# Load pre-requisites
with open("resources/booster_words.txt", 'r') as f:
    for line in f:
        if '#' not in line.strip():
            boosters.append(line.strip())

with open("resources/cues.txt", 'r') as f:
    for line in f:
        if '#' not in line.strip():
            cues.append(line.strip())
# Returns True if sentence contains boosting, otherwise returns False
# Input: Sentence
# Output: True/False
def IsBoosting(s):
    for word in boosters:
        if word in s:
            list_of_words = s.split()
            if word in list_of_words:
                previous_word = list_of_words[list_of_words.index(word) - 1]
            else:
                previous_word = ""
            if previous_word in ['not', "without"]:
                return False
            else:
                return True

In [7]:
#Convert the interviewee's answer to a paragraph list.
#Input: A String of the file name
#Return: A paragraph list of this file
def processToParaList(file):
    sentenceList = []
    ans = []
    processor = Process(file)
    processor.process_html()
    ques_ans = processor.extract_ques_ans()
    for pair in ques_ans:
        ans.append(sent_tokenize(pair[1].lower()))
    return ans

In [8]:
#Choose a dataset file you want.
#Dataset should be under folder datasets/interview_transcripts/

BertheKayitesi_ans = processToParaList('datasets/interview_transcripts/BertheKayitesi.docx')
len(BertheKayitesi_ans)
#print(BertheKayitesi_ans[:20])
YVETTE_ans = processToParaList('datasets/interview_transcripts/ENTREVUE YVETTE ISHIMWE-LS-English translation.docx')
trainning_dataset = BertheKayitesi_ans + YVETTE_ans # length = 229

In [9]:
#Here, I changed one YVETTE's answer to several paragraphs. Because it has one answer is pretty long and that paragraph includes a tension 
#It will effect the whole result.Thus, I change it to several paragraph.
#The interviewee narrates a long story and the tension happens among the story. So, it indicates the tension has 
#built up during the illustration. 
#Here comes two way to evaluate: 
#        1. use sentence level's hmm. 
#        2.divide the whole paragraph into several paragraphs.
#So, the new trainning_dataset's length will be 236.
trainning_dataset = [['... my name is berthe kayitesi ... i am from gisenyi in rwanda.', 'i was born in gisenyi, but i spent part of my childhood with my maternal grandmother in the former kibuye prefecture ... the former commune of [inaudible].', "... my father's name was [inaudible], my mother's name was mukabilima spéciose, my grandmother's name was [inaudible] valérie.", "i think that’s [inaudible], i think that's all."], ['...i was born on november 11, 1978 ...'], ['in gisenyi.'], ['[laughs] [break in the recording]'], ['... of my mother, i remember her courage in the face of life’s difficulties, she was a fighter.', 'she made sure that everything was going well at home, but she was also very strict, that’s the side of her i didn’t like.', 'but i remember her strong character, her courage, her fighting spirit.', 'that’s my lasting memory of her.', "for example, i remember that every day she would get up at almost five o'clock, five-thirty, and would start working.", 'she advanced ... quite quickly in life.', 'she often regretted not having been able to continue her studies, she always told me: “you know, i was smart, but i was unlucky, so ... you must ... ” [laughs].', 'and also: “i was at the top of my class.', 'how come you allow yourself not to be ... to fail?” so, [laughs] we had to be among the best students in school.', 'and when we weren’t, she was on our case ... [laughs], so to speak.', 'she was also a very beautiful woman.'], ['... my father ... my father had a very gentle character and ... he’d had a difficult past, but he almost never expressed or shared his feelings.', "he was someone very warm, very, very hospitable—how should i say it—it's really hard to describe my father.", 'but ... it’s ... his calm disposition… he also had a certain hardness about him, but he had ... in fact, his kindness ... counterbalanced this hardness, and that’s how we remember him.', 'those who knew him retain that image of my father as someone who was very kind, someone who never complained.', 'i think his father had been unfairly imprisoned for many years, but he never spoke of that.', 'he had left ... well, he kept it all inside and he lived with these feelings, but without letting them affect the family.', 'thinking about it today, i wish he shared with us a little bit more of what he felt inside, but he had decided to keep it all within.'], ['... my father was a tailor, he was one of the first tailors in the region where he was born, in kibuye, gishyita, which he left to settle in gisenyi and continued working as a tailor there, well, that’s what he continued doing until 1994. and as for my mother, she did many things.', 'at one point we had a bar—actually, we had a bar at different times—and she was taking care of it.', 'and shortly before 1994, she ... she was selling things, food, at the market.'], ['... the children also participated, so i think the work was ... shared.', 'so, when my mother got up at five-thirty, sometimes it was because she had to get—how do you say that in french?—because the bar was also serving urwagwa, mutzig and primus ...'], ['yes, banana beer.', 'she had to go get it very early in the morning.'], ["yes, the expression we use is “going kurangura” ... so she would get up very early in the morning, around five-thirty, and when i got up, she would tell me, “before going to school you have to buy ... food for lunch.” so i would wake up around six o'clock and i would go buy ... whatever we were going to eat at noon.", 'and after that i would go to school.', 'upon her return, she would make lunch before going to the bar.'], ['so when i came home in the evening, i would take care of the rest for [inaudible]'], ['... what they were both teaching us, i think it was equal, but for the most part, in terms of household work, it was my mother and me.'], ['... at the time, in 1994, well, when i returned to the house ... after living with my grandmother ... we were ... five.', 'later, my mother, well, my parents, had two more children so we were seven, but one of my younger sisters died just before the genocide.', 'for us, the children, luck was on our side and we ... survived.', 'the six of us survived.', 'but my father had another marriage before ...'], ['yes.', 'so from that marriage, he had three children: two twin sisters who died in 1994 and their mother too, and another sister who survived.', "so with this sister, and with her brother on her mother's side, we found each other afterwards—it’s like being a family of eight."], ['three girls and five boys.', '[laughs]'], ["... i think that rwandan culture at the time ... favoured…—not only through education, but also through social discourse—it's as if the boy child was more important than the girl child.", '[laughs] i remember, i was angered by this.', 'but that’s over now.', 'when i was born—so my paternal grandmother, actually, my paternal grandparents had five boys, so they didn’t have any girls.', 'my father was their eldest son, and until i was born, he only had girls.', '[laughs] so when i was born—i was the first daughter of his second marriage—so my grandmother said, “oh, it’s a girl again!” [laughs] and this made me angry.', 'my mother had not picked up on that, but i did, and it made me angry a little bit.', 'in our family too, as the older sibling and as a girl, i had more on my hands than my younger brother.', 'for example, he could just go get water, but i was expected to do the rest ... and so there was disparity, but there wasn’t a ...—how can i say it—... it went without saying: you are a girl and so you do the housework.', "... that's how i saw things and then ... if my brother went out to play ... that’s okay, he can go play, he can go for a swim, i couldn’t go swimming ... [laughs] i had to be ..."], ['yes.'], ["it’s not only an adult's perspective, but also an outlook on rwandan society from the outside.", '... what i appreciate here, i think, is that we try to [laughs] treat both boys and girls in the same way.', 'i don’t know if that’s something that has changed in rwanda, but they’re making an effort to promote equality.', 'i don’t know how things are in people’s homes, but at least in the social discourse we try to support girls as well, and in my view, this attitude produced through social discourse can also change things within families.', 'i don’t know where we are at on this issue, but i think that if i had children i would try to have a more or less balanced approach, even though a boy can do things that a girl can’t necessarily do.', 'but i think it’s a question of encouraging ... the child’s abilities and helping the child discover what it is that they want to do.'], ['... well, yes, in the city where i lived.', 'maybe that wasn’t the case in the countryside, i don’t know.', "i think that women, girls would not be, for example … making bricks to build houses, so that was work for boys, but i'm talking about ... the way things were when i was 15. i think girls had more to do, but elsewhere maybe it was the boys who…, so i ..."], ["but let's say, for example, beyond my personal experience, when i was interviewing people in rwanda, there was a girl who told me, “my mother said that boys can go study, but not me, because i am a girl.” that's not my experience, but ..."], ['someone else’s, but …'], ['yes.'], ['... one graduated from university and works at the bank of kigali.', 'another works for a private company and the others are in school.', 'and even before, in terms of education, our parents ... this was true for all children; so my mother urged me to study in the same way she urged my brother, so there wasn’t really any imbalance there.', 'it was more ... in terms of housework, but in terms of education, all of us, we were all encouraged to go to school.'], ['... i personally don’t remember my paternal grandparents very much, but i do remember my maternal grandmother.', 'when i went to live with her—i don’t know how old i was, i think two years old or less, i don’t know—i remember that time, she gave me breast milk from her own breast until i was grown.', 'and ... i remember that my grandmother was a very sociable person.', 'she had already lost her husband so i never knew my maternal grandfather, but two of her children—my uncle and my aunt—lived in the area and i grew up with my cousins, my uncle’s and my aunt’s children, and i remember that on weekends we travelled far to visit my grandmother’s ... brother.', 'also, she was a farmer, so we had small fields all around the house and that was what occupied my time and what sustained us, but also her children who were close to her.'], ['... nine years old.'], ['if i had a preference [laughs]…, but at that age… i remember when my mother would come to visit [at my grandmother’s], i would always say, “i want to live with my parents.” and my mom would say, “next time.” so it was always “next time,” but i wanted to live with my parents ... but when i arrived there, i ... i felt like a stranger.'], ['in my own family.', 'and i even think that [laughs] in a sense i remained a stranger [laughs].', 'you know, coming from life in the countryside and suddenly finding yourself in a big city where ...—before, i used to go in the fields or in the forest to collect wood for my grandmother—and then i arrived in a city where ... so i felt out of sync, i think, [?]', 'yeah.'], ['well, when you’re a child, you don’t think if it’s easy or not, you just go about your life.', 'i can look at that time now, from the perspective of today, but back then i didn’t think whether it was difficult or not.', 'besides, i adapted very well during the first two years.', 'it was as though i really wanted ... my feeling of nostalgia for my family was strong.', 'yeah.'], ["at my grandmother's house ..."], ['the neighbourhood where my grandmother lived was ... in africa we say that the child belongs to the whole village, so i would go here and there in the neighbourhood and my grandmother was never worried about me.', 'i could go to other families’ houses and so i was always out and about with the other children, herding the cows or fetching wood together and ... so that’s how life in the countryside was in my grandmother’s village.', "but once i moved to my parents' house, it was more like, you must not go ... to such and such ... people would not necessarily know who we were and i couldn’t go to the neighbours’ house like i did at ... like when i was at my grandmother's.", 'there were restrictions ... luckily, sometimes—there were several houses in a ..., well, in a ...—how can i say this?', '[says the word in kinyarwanda].', "it's like several apartments…"], ['yes, a complex of houses and so we would also ... socialize with the neighbours, but it was really different.', 'i think if i had a preference—in fact, i appreciate that i had both ...—but if i had to choose one, i would prefer the countryside, there was a certain freedom there [laughs] that we didn’t necessarily have in the city.'], ['... i don’t know.', 'i don’t know.', 'i learned later on—because i did stay for a short time with my paternal grandparents—i didn’t know them very well, but later, talking to people who knew them, i was told that my paternal grandmother was a very brave woman.', 'during her lifetime, she was someone who was also ... a great fighter.', 'when her husband was in prison, she kept the family together.', 'she made everything look as if ... as if nothing was happening, while it was, after all, a family drama.', "she had to take care of her brother's children who had just lost their parents.", 'and, apparently, she was also quite strict [laughs].', '... so there are these two traits: the courage of my paternal grandmother and also her severity.', 'i don’t know if we can say it’s this or that.', 'i think i have inherited some traits from my father and i also have in me much from my mother’s personality, but i don’t know if my identity has been shaped by who they were or by the force of events, so it’s hard to say ... we are influenced by many things in life.', 'i’ve probably kept qualities from both of them, but what happened to us is also a very important experience, which brought out aspects of our character that would have never come out had we not gone through these events.', "so, it's hard for me to say … to say that my character was formed by this or that, i think it’s a bit of everything."], ['yes, or maybe that’s only an assumption, so i don’t know [laughs].'], ["... i'm not the oldest, but i'm like the most ... it's me who’s taken over."], ['... i think that my half-sister…—i connected with her later, after the genocide, two or three years—i think, i don’t remember exactly—but two or three years after the genocide.', 'i had already lived with her in my parents’ house as the eldest of the family and i think it stayed that way.'], ['my half-sisters.'], ['... i knew the twins who died.', "but the other sister, whom i found later, after the genocide, i didn’t know her in 1994. so, it's… but i already knew her name because one day i looked at my father's id card and i saw a name written on it.", 'i didn’t know who that was, but i remembered the name.', 'and later someone said to me, “oh, you know what?', 'one of your sisters survived.” so i began looking for her.'], ['except the two [other sisters].', 'they came to visit from time to time.', 'but not her.'], ['... no, my half-sister works in gisenyi, but the others are in kigali.', 'and then there’s marie-ange who is here with me.'], ['i think i didn’t have ... i don’t remember the things that i hated, but i know i liked school.', '... i liked school very much.', 'as for my relationships with the other kids—we played together.', 'and i remember there was competition between us—who would be the best student in the class.', 'we had to study a lot if we really wanted to have the highest rank.', 'i was doing my best to be among those at the top, especially since i also felt pressure from my mother.', 'i think it came naturally to me—i was learning quite quickly.', 'and i think that when you are a good student relationships are formed more or less easily because the teacher likes you and your classmates do too ... those students who are more or less smart enjoy a certain respect and i think it [laughs] it played…, it allowed me to…'], ['yes, to move forward.'], ['yes.', 'yes, they were always ... they were good to me, even if sometimes they told us to stand up as groups of hutus and groups of tutsis—everybody would stand up, but we were always a small minority of tutsis so sometimes, [inaudible] the others were looking at us a little strange, but that was all it was.', 'so we knew there was something, but it wasn’t explicit in the way they looked at us.', 'and it was ... so, we would stand up and sit back down, and we would know in this way who was also tutsi, but that was all, we would then carry on with our day.'], ['yes ... no.', 'yes and no.', '[laughs] when i took the exam to get into high school, my mother said, “in any case you are tutsi, so i would be surprised if you get a place in secondary school in rwanda.” she asked her brother, my uncle who lived in bukavu, to get me a place in a school in bukavu so i could go study there.', 'i went there and my uncle placed me in 6th grade in bukavu.', 'a few months later ... so i think that when i left [nyundo]—i don’t remember exactly—but i think that when i left, the lists had just been released and i was not on the list of those who had passed the national examination.', "later, my mother sent a letter to my uncle's house saying that i had to go back because there was a second list and my name was on it.", 'so i remember i started high school—i came back to nyundo and i think the others had started classes a month and a half or two months earlier, and so i jumped on board with them.'], ['i think you know it, but you don’t understand it and you don’t dwell on it.', 'so, when i try now to understand what those experiences meant to me back then, i think i knew it, ... but i was closing my eyes, so i think that i was living in denial.', '... already in 1990, there was in gisenyi an operation called operation zulu, so tutsi families—mostly those that were more or less well-off—were threatened and grenades were thrown.', 'i remember that i was the only one in my family who had left home and i hid with the nuns and a few other children from tutsi families near my elementary school; later, we left our hiding place.', "so we could see the ... it's hard to interpret this today.", 'i think the discrimination was there, but we ... we didn’t talk about it in my family and it was happening, it was going on, but it was surrounded by silence, and so we didn’t know ... why were grenades thrown at this or that place.', 'we knew where it had happened, but we didn’t talk about it, and life went on.', 'it was as if something had happened and then—silence.', 'and i think we knew, we knew, but without understanding it or without necessarily attributing to it the importance that ... and our parents were trying to live with it, and that was the example we followed.'], ['actually, for my grandfather, according to the information ... i’ve asked the people who knew most about it and they told me, “someone killed somebody else and put the body in your grandfather’s field, and it was the hutus from the village who did that.” so, it was related to ethnicity.', 'that’s true in my mother’s case too, but she didn’t talk about it that much.', 'we can’t know how she felt about that, what really happened, but there was a connection.'], ['yes, i think that maybe they wanted to protect us.', "but it's… today, we can only look back and ask why we didn’t talk about it.", 'and… in fact, why didn’t we know more?', 'why didn’t we discuss it to at least have a clear understanding of what was going on?', 'but today we are stuck asking why.', 'why didn’t we revolt?', "why didn’t we say, we'll all follow my uncle?", 'why did we accept all of that?'], ['in 1992… 1992–1993.'], ['yes, later on.', 'after the genocide, i did a year in a private school in gisenyi.', 'but later i went back to nyundo.'], ['in 1998.'], ['before 1994, i was taking courses in literary studies (option littéraire) in high school.', 'later i took some courses in general education (l’option normale).', 'i returned to the literary option after that, but it wasn’t literature as such any more.'], ['after high school, i went to university to study psychopedagogy.'], ['in psychopedagogy, my undergraduate degree (bac), yes.', 'not in nyundo, but in kigali [inaudible].'], ['yes, almost.', 'i am in education sciences, i’ve had a career as a student.'], ['no, it wasn’t my choice.'], ['we were profiled in advance.', 'so when i went to nyundo, i was told, “you’ve been placed in literary studies,” so it wasn’t me, really, who decided what to do.', "later, i went back to literature, but that's because that was what i knew.", 'i would have preferred much more to study law at the university.', 'when i was in my fourth year in secondary school, i thought i would go into law, but my grades were not good enough to enrol in the university in butare where i could study law more easily.', 'and the ulk (kigali independent university) had just opened, so i didn’t want to go there.', "that's how i enrolled in psychopedagogy."], ['yes, it was my second choice.'], ['... in fact, i think it was already happening in the gisenyi region.', 'there was this period called zulu.', 'i don’t remember which year, but it was between 1980… i think it was 1993. but at the school where i was …, in high school in nyundo ... very close to the cathedral, the bishop of nyundo at that time, the monseigneur, had given refuge to the bagogwe who had been chased from their region where ... well, from their homes.', 'so when we went to mass in nyundo, we saw those refugees who were refugees inside their own country and who were there not because they were bad people, but because they were tutsi.', 'and so, in gisenyi during this operation zulu…, i remember that after that period, many tutsis who had the financial means left gisenyi and moved to kigali.', 'and there were displaced children in my high school, displaced because of the war of the rpf [rwanda patriotic front], and from time to time they were ... showing hatred for their tutsi classmates.', 'my classroom was ... actually ... i didn’t really see much … i didn’t see anything going on in the classroom.', 'but since we were at a boarding school, we could see that small groups were forming.', 'i remember one time, i was with two other girls, we walked by a small [group?]', 'of children, some of whom had been displaced by the war, and i remember we saw them spit on the ground.', 'but it was just ... it was an insult directed at us.', 'at the junior seminary, there were things ...—the seminary was nearby—things of the same nature were happening there too.', 'people, students, were sharing stories in the evenings.', 'and the atmosphere was starting to get tense because of that.', 'and, i think, when i look back at that time now, these are the kinds of events that foreshadowed the worst to come.', 'but at the same time, i didn’t think about it too much.', 'i don’t know if at some point you get used to living with things like this.', 'but also, not all hutu children were like that.', 'i remember in one of my groups there was a girl, actually, several girls with whom we were often together and we never really noticed any animosity against us.', 'so, those things were ... were there, they weren’t widespread yet, but we could really ... we could understand them if we wanted to see the situation as it really was.', 'but, once again, you had to close your eyes and, you know, ... move on.', 'when 1994 happened i was both lucky and unlucky.', 'my luck was that i wasn’t at home.', 'so, prior to 1994, i had run away from home and i had met some of the nuns from the auxiliaries of the apostolate, after which i returned home.', 'so from time to time i would spend the holidays at the convent.', 'and it was easter in 1994, so i was going to spend my second week of vacation with these nuns.', 'i left home on easter monday and the genocide began on the 7th.', 'but i remember that already on the 6th, the night of the 6th, one of the priests came to pay a visit to one of his friends among the auxiliaries.', 'and as he was there, we were told…—since the routine in this convent was to get up in the morning and go to the church to attend mass.', 'so every morning, we went to mass.', "by chance, this priest was there and the priest said—actually, one of the nuns told me: “since the priest is here, he’ll celebrate mass at the convent, so you don’t have to go to the church tomorrow morning, the mass will be held here.” so we got up to go to the mass and the first thing the priest told us was, “the president’s plane has been shot down.” everyone was ... i was surrounded by adults and later that day, sitting around the table, we continued discussing the news and i remember i said, “it’s over.” today, every time i think about why i said “it's over,” i don’t understand why i said it.", 'and it was indeed over.', 'i don’t know if that phrase was the expression of everything that i had seen until then and stockpiled in my mind.', 'and then something else was going on in gisenyi—there were these big buses that ... big buses that were transporting…'], ['it was the militia, i remember, but i think there was an odd euphemism used for them—people were calling them “the magicians.” so they were people who… they always filled the buses and sometimes, they were on top of the buses and they were dancing, you know, they were always making noise, they were singing.', 'i had seen this frequently in my hometown, so…'], ['yes, before 1994. and sometimes we watched them go by and we applauded—they were the magicians.', 'so, i think all of that made me think “okay, it’s over.” a few minutes later, the nuns were receiving phone calls informing them about the priests from this place and the nuns from that place…, what the situation was like outside, so on that day ... it began, the genocide began.'], ['yes.'], ['no, i stayed in the convent.'], ['[inaudible] [laughs].'], ['and i said “it’s over” once more, but at the same time i didn’t really grasp the magnitude of this phrase, “it’s over,” and what they were saying.', 'i remember i felt such fear around me that i ended up not being afraid anymore.', 'all around me there was ... [laughs] there was really a feeling of terror, everyone was moving in all directions.'], ['there was panic ... you would be talking to someone and then they would go away.', 'people were moving around.', 'i’m trying to visualize it, i was saying, “it’s not possible!” so people were going up to the attic and i saw them coming down, they were looking for félicité.', "“hey, mother, what's going on?", 'do you have news of this or that person?” people would abruptly go into the room, they would go in ... and talk in low voices behind locked doors.', 'others would be listening to the radio ... they were trying to capture [?]', 'to find out what was going on.', 'still others spent time in the chapel.', 'and it was in this atmosphere…, but since i knew the house well… and actually, later the female students of saint-fidèle university also sought refuge at the convent and other people from the city who could reach the saint-pierre centre, came as well.', 'and since i was…, since i knew the place better than others, i continued to help.', 'i helped with the cooking and with the housework.', 'i continued participating in the life of the convent as i did before, so ... i kept myself busy and i didn’t really have time to stop.', 'it’s only now that i stop and become frozen ... at that time, i was much more in the action.', 'félicité made sure that i had things to do, but she prohibited me from opening the door when someone rang—i always liked opening the door, but she prevented me from doing that.'], ['yes, well, i went to see many of them, but she was the nun responsible for all others.', 'but i was worried about my parents and especially about my father because a week prior to that he was attacked by militiamen and by soldiers in a bar, and on the way home we went to see him.', 'my mother brought some money and i ... right away, i don’t know why, but i felt afraid for my father.', 'and my mother was a sociable person and she had a lot of friends.', 'i always had the impression that maybe… and she had an mrmd (the national republican movement for democracy and development) membership card, but my father had an rpf card.', "i remember, when i was in congo, my uncle told me: “your mother has an mrmd membership card, she's really brave!” [laughs] so, i thought that maybe that would save her.", 'because, later, i was afraid for everyone…, but right away, i felt that i was losing hope for my father.', '[wipes tears]'], ['no, life continued.', 'well, i can’t know everything that was going on in the communes, but we knew that the tutsis were targeted, that it was about the tutsis.', 'i don’t remember being hustled by any of the hutus who were in the convent.', 'even though félicité and nathalie went to—in fact, on account of these students from saint-fidèle —i remember they went to get another saint-fidèle student who was injured and was in the hospital and they brought her to the convent where she was treated.', 'and félicité had sent two hutu nuns to my parents’ house.', 'upon their arrival, they found out that my parents had been killed, but they didn’t tell me that when they came back.', 'and i don’t know if they had tried to save them or what, but i know that the next day félicité asked me, “do you know anyone in congo?', 'do you, by any chance, have family there?” i said yes—that was around april 19–20.', 'i told her that and she said “okay.” i think it was on the night of april 19 to 20 when she told me, “don’t go to sleep tonight.” actually prior to that date my brother had come—my younger brother, jean-baptiste, had arrived at the convent.', 'and she told me that on the 7th, on april 7th, a classmate of mine from elementary school, whose brother was taking part in the killings [a génocidaire], had said, “you know what?', "all tutsis will die.” and then my brother had told my parents, “i'm leaving.” and my mother had said, “okay, go to this family.” one of my mother’s cousins had a husband whom we didn’t know if he was hutu or tutsi.", 'afterwards, i received written information confirming he was tutsi, but at that time we thought he was hutu.', 'so, my ... then my mother had said [to my brother], “if you’re going there, take marie-ange with you.” and so…'], ['yes, my sister.', "and so jean-baptiste went with marie-ange to jacqueline’s house, but once there, jacqueline's husband told jean-baptiste: “we'll keep marie-ange, but we can’t keep both of you.", 'do you know where berthe is?” jean-baptiste said yes.', "“okay, go to her.” so, jean-baptiste came to where i was staying, but on the way there, he met someone who asked him: “you are alexis’ son, aren’t you?” my brother said, “no, i'm not alexis’ son.” after that he arrived at the convent and so we were both staying there together.", 'and one night félicité told us not to go to bed.', 'later, around midnight, she came back and said: “okay, you can go to sleep now, we were just told that there are militiamen everywhere and you can’t cross [the border].” she wanted us to cross over to congo where she had sent about fifteen people already and she wanted to try that a second time, but it didn’t work.', 'so until the 21st…, when the militiamen came to the convent, so around 3–4 pm… i was in the kitchen, and félicité came running: “tell everyone to hide, the assassins are here.” i ran around the convent saying, “hide!', 'hide!” i hadn’t even finished saying that when the génocidaires arrived and i was then saying that i wasn’t tutsi.', 'they brought us to … one of them took me along with several others and there was ... they were people all over the place, so the militiamen had ordered many of the people to come out from their hiding places.', 'i went to the exit door and i saw many people on their knees and a ... a killer pointing his rifle in their direction.', 'they told us to kneel and we did.', 'they had two minibuses parked outside the convent and we were told to get on the bus.', 'i fought back.', 'one of the killers was bringing people in ... and i said, “don’t hit me.” félicité hadn’t gotten on the bus yet, and they forced her to.', 'so we left with the first bus and they told us that they were going to take us to the red commune.', 'inside the bus—one of the nuns had been teaching us a song—so i started singing it and then we were told to keep quiet because “you killed president habyarimana, so you don’t get to sing.', 'the only hutu who’s on the bus can continue singing.” so, we took ... the driver took the road to the cemetery.'], ['that was the red commune.', 'and on the way there, we saw that, outside, life for some people was ... continuing.', 'while you are on the bus heading to certain death, right next to you—in any case, for the hutus—life hadn’t changed, the market was open and you could see people in the street; at the time when you were afraid to come out of your hiding place and we were going to ... to the cemetery.', 'and when we arrived, many people ... came to assist.', 'on arrival, i also saw militiamen coming out of the [inaudible] with machetes, even with old, traditional machetes… and in the bus i had…, i had learned the name of one of the ... well, the name of the bus driver ... well, of my killer, a militiaman, and someone was saying ... who was saying, “but thomas, don’t you recognize me?', "don’t you know i'm from such and such a place?” i remember that.", "but i also remember that they didn’t really want to kill félicité, because while we were driving, they said: “the colonel's sister must not go on the bus.” but she had insisted, “if you're going to kill these people, i'm going with them.” at the same time, even though the colonel's sister had to get off the bus, they never forced her out—it was just empty words.", 'so when we got to the cemetery, i thought, “maybe they won’t kill her.” so i got off the bus, took her hand, and we all got out.', 'the others got out and immediately lied down on the ground.', 'one man, about thirty years old, tried to escape, but the génocidaires ran after him and caught him.', 'it’s one of the most vicious images i have from that moment.', 'they caught him and he was held on both sides by two génocidaires and two others ... were ... cutting him, and everyone was watching.', 'so, in the meantime, félicité, who, i imagine, didn’t really want to see that whole scene ... let go of my hand and she turned away…—the cemetery was at the foot of a mountain—and so she gazed up towards the mountain as if in a prayer.', 'so, when she let go of my hand ... i took thomas’ hands, the bus driver.', 'and thomas took my hand too.', 'i saw in the crowd a former classmate of mine from elementary school, and i made a sign with my hand, like that [makes a sign with her hands] to ask for help and he did this [makes a sign of helplessness], meaning, he couldn’t do anything.', 'and while thomas was holding my hand, one of the killers snatched me from thomas’ hands and thomas kicked him.', 'in the meantime we realized that omar, who is now in arusha, was shooting those who were lying on the ground.', 'there was a child there, about two years old, and thomas, who was holding my hand, told me to take the child and go back on the bus.', 'a few of the génocidaires had stayed behind in the bus, there was also another girl in there—she was the child’s aunt, but she didn’t want to reveal her identity, anyways, the child’s aunt…'], ['yes, the child’s aunt, the maternal aunt of the child i was holding in my arms.', 'they were saying that the child’s father was hutu, but the militia didn’t want to hear that.', 'at the moment when i took the child to get on the bus, i saw félicité falling to the ground, and at the same time i saw the priest—the same priest who celebrated mass on the first day, who had stayed with us at the saint-pierre centre—i saw him approaching with three policemen to save félicité, but it was too late.', 'so we got back on the bus and ... and it was over.', 'omar returned to the bus with thomas, the driver, and we started on our way back.', 'while we were driving omar asked thomas what was i doing in the bus and thomas replied: “she just lied to me that she is tutsi.” omar said, “you have to stop the bus, i have to kill her too.” but thomas didn’t ... didn’t stop the bus.', 'we dropped omar off at one of the big hotels in gisenyi, palm beach, where i saw people ... people who were drinking, who were eating, so omar joined them and thomas then dropped me off at the saint-pierre centre.'], ["it's the convent, that’s what it’s called."], ['where we were before we left.', 'and when i arrived there, i saw two policemen and i saw nathalie, who was félicité’s assistant.', 'nathalie asked: “so, is félicité dead?” and the driver, thomas, said, “i saved these two children,” as if…'], ['not to answer the question or: even though félicité is dead, he saved two people.', 'surely, since nathalie was angry and ... she was scolding him, he told her, “but i just saved…” later, in the evening, the policemen who were there, apparently came back to protect me ... and they said … there were people still hiding in the convent who had heard the génocidaires saying, “next time, we will burn it all down.” the génocidaires had said they would burn everything.', 'and i began to regret not having died ... not having been shot dead in the cemetery.', 'but that didn’t happen.', 'the next day, actually, that same evening, several soldiers came by to ask how félicité had died and it was as if she was the only person who had died.', 'i tried to tell them what had happened, but i didn’t know that they had captured several of the killers.', "and the next day, félicité's brother and some of his friends went to dig up félicité’s body and give her a dignified burial, among the thousands of dead bodies.", 'and the people, the hutu sisters who were there, later told us that all the corpses were naked.', 'and on that same evening of april 22, we ... sought refuge in congo.', 'we decided ..., we decided to leave with the few people who were still at the convent.', 'in the meantime, my brother had gone out of the convent and when he came back we told him to come to congo with us, but he refused.', "he said, “i’m not going.” we woke him up around midnight and he said, “i'm not going anywhere.” and i knew that this was a serious mistake—but fortunately he survived…—to leave…"], ['yes.', "he said, “i'm not going anywhere.” so i left without him."], ['so it’s… i don’t know, maybe we could have forced him to come with us since we were quite a small group, but he refused and we all left him there.', "i left with a feeling of guilt, but i'm lucky he survived, which eases that feeling a bit."], ['i think it’s five minutes.'], ['it is, but that distance, during a genocide, becomes huge.', 'today, when i think about it, i wonder: “we had to wait for two weeks before we could cross over.” we were there, we could see congo, so it was…'], ['i don’t know.', 'i think you can be there in ten minutes.', 'normally, it’s three minutes to the border, but if you go through the fields, it may be ten minutes—in any case, it was really close.', 'but during a genocide that distance becomes huge.'], ['no, the people who left were several tutsis who had not been found in their hiding places.', 'the nuns and all the others who were not targeted stayed behind.'], ['no, i was with several other people.'], ['yes.', 'we were about to arrive in a few moments and we slept on the ground in ... we were sitting on the grass, waiting for the sun to rise.', "around five o'clock in the morning, we set out towards the houses in birere, in goma and we arrived at a church, i think it was a pentecostal church.", 'we went inside and ... those who were there could see that we were people escaping death.', 'there was a tutsi woman who was living in congo and she decided to take care of me, and it was as if everyone found a place to go.', "there were several female students from saint-fidèle ... and there was one who was studying at the adventist university at the time and she said, “i'm going over there to join the adventists.” and i … when félicité had asked me if i knew anyone in congo, she was thinking of sending me first to goma, to a nun who had left with the first group, among those who … and i said, “can you take me to the bishop or do you know adria?” and that’s how many of us were sent to different places.", 'i was with immaculée, one of the nuns who was tutsi, and with adria—actually, the nun brought us to the episcopal see in goma where adria and the other sisters came to see me.', 'and we went to adria’s family in goma—there were several of us from the saint-pierre centre.', 'on may 3, the orphanage in nyundo was evacuated.', 'after our departure from the convent, nathalie brought my brother and three other kids whose mother had left with the first group [inaudible] to this orphanage.', 'so when the evacuees from the nyundo orphanage arrived in goma, my brother was among them.', 'i then demanded to go to the orphanage in goma and so we left this family and went to the orphanage.', 'later, in may ... my uncle who was in bukavu, found out where i was and sent his wife to come and pick me up.', 'but he thought that i was the only one there from the family.', "when his wife arrived and when she saw that my brother was also there, she said, “okay, i'll take you both to bukavu.” ... so she signed me out from the orphanage to bring us to bukavu.", 'and on the same day we signed all the papers in order to leave the orphanage and after we left the orphanage and were going to spend the night in goma before leaving the next day to bukavu, ... my friends from the orphanage came to tell me, “your three brothers just arrived at the orphanage with the red cross.” and so i went back to the orphanage.', 'we couldn’t get a place in the orphanage again easily because there were many children there.', 'but with luck, some would leave [and we could take their place].', 'i said, “i’m not leaving” ... they were trying to convince me that i had to leave because the director… that we couldn’t ... that we couldn’t have a place there anymore ... that the unhcr [united nations high commissioner for refugees] had to sort this out.', 'and we went back and forth to ... to the unhcr and ... it wasn’t ... it really was a whirl of people trying to…, i [inaudible] who had other things to do and who told me: “go!', 'the children will stay at the orphanage and you ... you will go to your uncle’s.” i said that was not possible, so i [inaudible] [laughs].', 'at one point, there was a priest who intervened, abbé martin ... abbé martin who knew someone at the unhcr, i think.', "the nuns, my uncle's wife and i were in the process of trying to get permission, and then martin intervened and after his intervention, the unhcr ... allowed me to go back to the orphanage, and so we stayed there.", 'later, we returned to rwanda with the other people from the orphanage.'], ['yes.', 'in the case of marie-ange, she had arrived in congo with a wave of hutu refugees.', 'one of the hutu nuns had gone to a [refugee] camp—i don’t know if she had stayed in this camp or not—she came to the orphanage and told me: “you know what?', 'i just saw your little sister in a refugee camp, the [inaudible] refugee camp.” so, i accompanied this nun and she showed me where marie-ange and the family that had taken her in during the genocide were.', 'and when i arrived, jacqueline said, “you have to be very careful.” so, we found marie-ange who was ... not looking well, she had been jostled around.', 'we took marie-ange from ... from the crowd of refugees and we went back together to the orphanage.', "so, that's how we all reunited."], ['yes, the three [brothers] who arrived with the unhcr, were with our parents when the génocidaires came for them.', 'the children hid behind our bar, but they saw the génocidaires leave with our parents.', 'they didn’t see them die, but they saw them leaving with the génocidaires.', 'and then they tried to find shelter at the neighbours’, but the neighbours turned them away.', 'and then they tried to reach the church, the gisenyi church, and along the way they came across several génocidaires who captured them.', 'and then—i don’t know who among the three…, i don’t remember which one…—i think it was gaétan who said, “we just saw some people looting a shop.” because when the génocidaires captured them, they said, “come, show us your home, show us where your parents are.” and the children had the idea to make up a \u200b\u200blie about a place that was being looted and the génocidaires left them alone [laughs].', 'they were going to loot too, they were going to look for that place where the others were looting, and the children managed to reach the church and the bizeramariya convent, which is next to the church.', 'but in the meantime, the génocidaires showed up to kill the nuns in the convent and i don’t know who among the nuns brought the children to the prefecture and it was at the prefecture that the red cross took them and brought them to goma.'], ["yes, that's at least what my brothers knew.", 'later, when i was in high school ..., after the genocide, before i went back to nyundo—i went to the pentecostal high school ... in gisenyi, the école normale primaire de gacuga [?', "]—and that's where i met someone who had seen them die, this girl who told me, “[inaudible] i saw your mother, they clubbed her until she died, your father was cut down with a machete.” it was ... it seems that it happened on her street and that there were also several muslim men there who were playing gisoro and who tried to plead for my mother’s life, but ... it didn’t help or they were just pretending to want to help—i don’t know, that's what she told me."], ['until 1998, well… i never liked the orphanage, i never…'], ['[laughs] i’m an eternal rebel.', 'i said to myself, “i will not stay here.” you know, you’re in the orphanage and you ... after i left [?]', 'when the notre dame lycée opened in nyundo, and since it was very close to the orphanage, i went back so that i can be closer to my brothers and sisters.', "but when you’re in the orphanage and then you go to high school, and everything is within a well-defined area, so you're really ... it felt like two prisons and sometimes i would just leave, i would go to my uncle’s and my uncle [would say]: “but where is it that you are going?", 'you have to stay at the orphanage.” and i would say, “okay.” so, since my third year in secondary school, i’d had the idea to get out of the orphanage, and i was hoping that it would happen miraculously after high school [laughs].', 'so i studied like crazy in my third year, i never worked as hard in school as i worked that year.', 'after the genocide, i thought, “i’ll finish school fast and i’ll leave the orphanage.” i thought that i would get a job right away and that everything would happen naturally.', 'i made contacts with people outside of the orphanage.', "and later, when i learned that i had a half-sister and that she was living with her maternal aunt in kigali, i went to look for her in kigali and when i found her, she said: “oh, i have a very good friend here and you must meet her before you go back to nyundo.” so, [inaudible] she is my sister's friend and she is really a very warm person and she told me, “if you want to come to my house during the school break, the doors are open.” and she was someone i had just met!", '[laughs] i said, “okay.” so, from time to time during the holidays…, i ... we kept in touch.', 'and 1997–1998 was ... the period of the infiltrators, it was still a dangerous time in gisenyi.', 'they were burning buses, so it really was another war.', 'and i thought, “we can’t die now!” i was knocking on people’s doors, i couldn’t see ... i went to my maternal uncle in bukavu and i said, “we can’t stay there, in this chaos of shooting.', 'it might not seem scary to you, but if our fate is to die now, we should do everything we can so that at least some of us are saved.” my uncle tried to find an orphanage, but he wasn’t able to.', 'and my friends, well, the families of my classmates were also trying to find an sos orphanage in kigali, but we made the mistake of saying that we were leaving one orphanage to go to another and the sos told us, “we don’t take children from another orphanage.” so i didn’t know what to do, but i kept in touch with this woman and she told me, “you can bring your three sisters, marie-ange and, well, at least the youngest children.” and this was not something we did at home.', 'it’s a family ..., well, after a genocide, anything is possible, but when i told my uncle about it, he said: “my goodness!” [laughs] and he turned to my two other brothers and asked: “so, what do you think—your sister is doing well, isn’t she?” but i didn’t care ... i didn’t care about what the others might say.', "i said to my younger brother, “you will sign as if…”—it was clear that my relation with the director of the orphanage was strained because of these comings and goings, so i didn’t dare showing up there and [i said to] my younger brother: “you're going to sign as if it were my uncle, you'll say that it's my uncle who…”"], ['yes.', 'or my uncle was forced to sign, i don’t remember exactly.', 'in any case, it was on behalf of my uncle that we left the orphanage ... actually, my two youngest brothers and marie-ange left the orphanage to go to this other family.', 'i stayed in nyundo, and it didn’t matter whether my other brothers and i were going to finish high school or not, because i was comforted to know that at least some of us from the family would survive.', 'so, jean-baptiste, gaétan and i remained in nyundo since they too were in high school—in the art school and i was in the lycée.', 'and marie-ange, jean-yves and yvan were [inaudible] in primary school in muhima and they went to live [inaudible] with this family that we had just met [laughs].', 'during the holidays, i would visit my youngest brothers and my sister and then i would go back to nyundo, and i was doing that until i graduated.'], ['yes ... and after that [inaudible] [laughs].'], ['there was a sort of madness to it too, i don’t think that if i found myself in the same situation now i would do the same thing.'], ['and then we stayed with this family for some time.', 'but, there were too many of us there.', "she had her five children, there were two other children on her husband's side and there were the four or sometimes the five of us from our family.", "jean-baptiste and gaétan stayed at the orphanage or went to my uncle's house, but the rest of us, we stayed with this family.", 'it was getting more and more difficult, especially because it was the mother in the family who took care of everything.', 'then i learned about the farg and i started going to the farg.'], ['assistance fund for genocide survivors [laughs].', 'i went to ... i started going to ibuka first and one of the sisters-in-law in the family who was hosting us, pauline, was working at avega [the association of genocide widows / agahozo] and introduced us to ibuka so that the children can study for free.', 'and it was there that we started addressing our problem that we were ... that we were nearly homeless.', 'so i started the process.', 'at that time, in the beginning of the farg, i had a friend from high school whose cousin was a secretary at the farg—a secretary who later married the secretary general of the farg at that time.', 'so we opened a file on our case and they said, “okay, we can pay you the rent for a house.” so, we agreed with our host family that we would look for a house nearby so we wouldn’t be far and the family would know what was going on with us.', 'we started looking for a house near where we were ... near donatille’s family who had welcomed us.', 'and so, in 1999, we started living in…'], ["[laughs] that's when we really became children, well, children and one head of the household.", 'we settled there, we had the rent, but that was it [laughs].'], ['yes.', 'we also had to jump through many hoops.', "it's really hard to explain, but it worked [laughs].", 'they told us, “rent a house for no more than 25,000 rwandan francs.” we found one for 15,000 frw, 15,000 francs and so we saved 10,000, but that’s cheating, so it wasn’t ... well, afterwards i didn’t care since we did the deal without ... agreement with the owner of the house.', 'i did the deal with donatille who had hosted us in her house and ... with another mom who was with us … in this together.'], ['later, the owner of the house found out about it and she went to the farg.', 'i told the farg, “i don’t care.', 'you knew that you had given us a house.” her complaints did not amount to anything since she had her 15,000 frw anyway.', 'but life in that house—that’s where things got really difficult and i thought, “why did we leave the orphanage?” at that moment, when we started living by ourselves, i was having regrets.'], ['yes.', 'sometimes i would go to my uncle’s ... who was helping us out, but there were many of us, so it wasn’t really ... it was more or less regular help, but it wasn’t enough.', 'at other times friends also helped, i sent letters to strangers, to people i was corresponding with when i was at the orphanage, and this was more or less fruitful.', '[laughs] today i avoid thinking about that time, because i don’t know how we went through that period.', 'but at the same time, well, when we were in that situation, we didn’t perceive it as alarming, which means that my brain was working all the time: where will i be going today?', "i'm going to ibuka again, and one time they sent me to [inaudible].", "tomorrow i'm going to ... so it was always…"], ['and it was an experience that…'], ['no we continued going to school.'], ['it was around the time when i started university, so i also had to cover transportation costs to the university.', 'my brothers were still at the boarding school, well, some of them were still in primary school, but we had a very good neighbourhood, so there were also [inaudible], there was another family of orphans who were heads of their household, but their parents had died of illness and we were really supportive.', 'and we had fields, we planted things, sometimes, we shared.', 'but it wasn’t ... you know, it was harvest season, so there wasn’t a day when we went to bed without having eaten.', 'but when we would be able to eat was—how can i say this—it was both luck and it wasn’t at the same time.'], ['yes.', 'afterwards, i think my brain worked like ... today, i feel like my brain is frozen, but at that time it was—whew!'], ['i think it lasted about a year, since i ... i also knew that my aunts had survived, my maternal aunts had hutu husbands, and so i went looking for them.', 'and at the time when i went to look for them, i was also ... working on a project of [inaudible] that the farg had ... at one point the farg told us, “we can’t just keep giving you charity, you have to do a project.” so we did a project ... to help ourselves, but that was also the time when i went searching for our maternal aunts.', 'one of our aunts had a good situation at the time, and when she came to see us, she said, “okay, but there’s three of us in the family.', "why don’t you split—everything is going to be simpler that way?” so ... and donatille said to me: “that's not good.” but i was all excited to have found more relatives, and i think that was a serious mistake i made with regards to my younger brothers.", 'so i said, “okay, we can disperse within the family.” we split—i think it was in 2001, so from 1999 until 2001 we were ... or until mid-2000 we were alone.', 'so the aunts…, so we dispersed.', 'i went to live close to the university with my university colleagues and my brothers were in the host families, so at my aunts’ houses.', 'but one of the aunts had aids and we didn’t know that, and shortly afterwards she ... she died.', 'and it was she who, at least financially ... she had a difficult character, but she helped us a lot financially, she supported just about everyone.', "and that's when it became difficult because it was this other aunt who then took care of my brothers.", 'that aunt had a husband in prison, so she had her own problems.', 'and she took in my brothers as well as the children of my aunt who had just died.', 'so the children survivors were living with other children as well as with the children whose father was in prison—even if they were all family, it wasn’t easy.', 'and my younger brothers jean-baptiste, jean-yves and yvan suffered a lot.', 'they stayed there until i graduated from university and, once again, i went to the farg.', 'at one point, it was like i was working at the farg—i don’t know how many times i’ve been there.', 'so after that, after graduation, i went back to the farg and i was able to get the house in kimironko where we are still today.'], ['yes.'], ['yes, one of them even ... the youngest, he often ran away from home and he was saying, “i want to live either with you or with my uncle.” and so i had promised…—when he would run away i would take him in sometimes and he would spend time with my university colleagues because i couldn’t [host him] at the time ... i had one room in the university dorm—i couldn’t.', 'the other children were deeply affected, but we have a good laugh about it today.', 'but things were not going well at all at the time.', 'the second youngest, jean-yves, he was quite affected, while the youngest ... the youngest brother, who was running away—today we laugh about it—but he suffered a lot too, so i don’t know if there will be any further consequences later on, but today we’re able to laugh when we talk about that, and we talk about it from time to time.'], ['we were all there, but we evolved differently.', "so, it's ... i think during the years when we were scattered around, there are things that happened in everyone's life and i think that it—how can i say that—we love each other very much, but it's as if ... it's as if everyone has taken their path.", "so, to say that we shared family moments—such moments were missing [she is crying] ... and even today, each of us is still searching for who we are so it's ... it's again time to…"], ['one works in …, in the [inaudible], another works for an oil company, a third is waiting to get into university and the youngest fell very behind in school and is now in fourth year of secondary school.', 'my half-sister is there… so they see each other, they were telling me ..., the boys prefer to go out rather than just sit at home with the youngest brother, but life goes on ... there’s no comparison with what was before, things are happening ... things are looking up.'], ['yes, i tried to ... the sense of responsibility is there, but it doesn’t weigh as heavily as before.'], ["yes, i'm trying to let go, but for the child who is in his third [fourth?]", 'year in secondary school and ... well, for the two youngest brothers, i still try to be there for them as before.'], ['yes.', 'and i also don’t accept the term “parent” because you can’t be a real parent in this way.'], ['actually, i think it adds a sort of [inaudible] to a person’s sense of self.', "i have a hard time thinking of myself as either a sister or a parent as [inaudible] it's ... i'm swaying between the two.", "and it's also paradoxical—now that i don’t have all these responsibilities, it's as if i’ve become childish, at times i feel more ... [laughs]."], ["i think it's both.", 'it’s a situation that makes you vulnerable, so now that i don’t have to spring into action or in my way of doing things sometimes, i realize, “but that’s really childish!” [laughs] at the same time, i also say to myself, “allow yourself to finally be a child, if you can.” still, if i act like that around other people, they might find it strange that at my age i react in a childish manner.', "and so, it's ... i think about that, but these things happen spontaneously, so it's not ... it's not an easy topic.", 'i think i have a divided identity.'], ['yes, i’m swaying between the two.'], ['... i think there were many people like that and ... at different moments.', 'i think, for example, before 1994, when i met félicité and the other nuns, it was initially another nun who introduced me to félicité and she was around 60 years old.', 'at the time i was only 11 and i was dealing with family situations that i didn’t accept and she welcomed me and took the time to talk to me as if she was talking to an equal and she treated me ... treated me with me a very positive attitude, a compassionate and welcoming disposition.', 'and especially ... to also see how she treated everyone, not just me.', 'she was someone who ... who knew ... who took care of some of the vulnerable people in the town.', 'i remember there was an elderly woman who did not leave her home and that nun was taking care of her.', 'she was paying someone to stay with the lady—i think one of the other nuns was staying with her, but she supported the elderly lady in her needs.', 'it was the nun’s simplicity and especially her disposition towards others that were remarkable.', 'and during the genocide nothing in her behaviour changed.', 'and i think that knowing that—despite how broken we were by the events—i think knowing that gave me confidence in life, as if no matter what happens, i don’t have to necessarily expect the worst and i can believe that even one individual among the thousands can spark brightness.', 'later, whether at the orphanage or at school, i was lucky to have teachers who always listened to me, who always appreciated me as a student, well, i could see they loved me and that, i think, carried me forward.', 'also my schoolmates, even when leaving the orphanage, even when ... many people were saying “this child won’t get anywhere.', 'she’s taking her younger brothers out of the orphanage.', 'what will happen to her?', 'she will become mayibobo, she will become ikirara, a street child.” despite that, my classmates who knew me, and even their families whom i got to know through my classmates, were supportive.', 'and then there was donatille and her husband fénariel who welcomed us without knowing us.', 'donatille was someone exceptional, she was ... a very kind, very generous woman who was really there for us during a difficult time.', "we ... it's as if we were sisters, we didn’t know each other, but she regarded me as her younger sister and she loved my little brothers as she loved her own children.", 'so, there were many positive things.', 'today, we’ve moved past this situation and when we look back on it we realize that it has weakened us, but that there were a lot of things that ... that, despite our vulnerability at the time, helped us get where we are today.', "so, i think that's it.", 'life here or there, well ... i was living in that situation, but i can’t say that ... that it was dramatic.', 'perhaps it seemed dramatic on the outside, to the people observing the situation, but i never experienced that.', 'our situation was what it was, i couldn’t stop and analyze it, so i lived with it and i carried on.', "but it's really thanks to ... to all these people that i’ve met.", 'and then, when i arrived here, you were here, emmanuel and callixte were also here.'], ['... actually, after my studies in rwanda, i got a job in [inaudible] in kibuye ... but i liked to study, so i entertained the idea that if the opportunity presents itself, i will continue studying.', 'so, i learned that there were scholarships for students from the francophonie, i applied and i was lucky to get one.', 'the destination was canada.', "so that's how we ... [laughs]"], ['yes, i came as a student.', 'i arrived in trois-rivières and i think that was the moment when i started to look back and to see that it wasn’t ... a normal or an ordinary situation.', 'and that’s ... how i could get in touch with emmanuel and with callixte with whom i spoke, i think, in the first months after my arrival in canada.', 'and that’s how it happened.', 'emmanuel and callixte asked me to share this experience in public and so here i am with you today!', '[laughs]'], ['... i think a big part of me is still there ..., so i think i live here physically while i’m often there mentally.', "and even if it's not there as a physical place, it's there as an experience.", 'so ... all the years i lived in rwanda, that’s what ... made me who i am.', 'i came here as a student, in conditions ... in good conditions, compared to everything i went through there.', 'being here has allowed me to go back…, well, it gave me the time to think about what i had experienced in rwanda.', 'in recent years…, actually, all the years that i’ve spent in canada have been years of reflection.', "i don’t know if i’m really integrating in the culture here [laughs] or if i'm inside my shell.", 'at the same time, i’m starting to have a social network in this country—i have friends in trois-rivières, people from my cohort in graduate school, people from my summer work, people from the place where i teach french in the summer, so i meet a lot of people, and i embrace ... i more or less embrace the culture here, western culture.', 'i also think that having shared my experience on different occasions ... i think that has also allowed me to ... to adapt, in a way, to adapt by accepting … who i am.', 'although i should not remain in this role of a witness, because it’s a vulnerable position, but i ... being here as a student … has also been a kind of a stepping stone for me.'], ['both at the same time.', 'well, there was ... when i arrived, emmanuel wasn’t really here, he put me in touch with callixte.', 'but it was mainly with a couple of friends from france, who were also foreigners in this country, that i built a very strong friendship, right from the first months of our studies in canada, and these friends later came to rwanda and i went to visit them in france.', "so, with them it was ... and with other québécois friends who joined our group towards the end of the master's degree.", "it was ... it was much more the community ... so i think it's both, it's the canadian community, well, the western community, but also a large part of the rwandan community.", 'actually, there wasn’t a rwandan community in trois-rivières, but i was in contact with page-rwanda, although not regularly.', 'and then i met josée, josée gicali, who had known my paternal grandparents whom i didn’t know, and it…, it helped create a very strong bond between us.', 'so she also helped me integrate in the first months, and later i expanded my horizons elsewhere.'], ["i think…, well, i’ve never compared them, it's a bit of a difficult question, but i think it's different.", 'most of the rwandans i meet here are survivors so ... obviously [?]', '... [laughs] we speak ... our language.', 'i remember the first time i was in quebec city before meeting other rwandans, it was ... it was wonderful to see ... i was at this place where they were performing dances from different cultures—it amazed me to see my culture in quebec.', 'most often, when rwandans get together ... we talk as if we’re still in rwanda ... we also talk about our experiences during the genocide, we talk about everything, actually.', '... in the company of canadians, we talk about a bit of everything as well and we share our experiences too, but our experiences are very different.', "so, they’ll talk to me about vacations and i'll talk to them about rwanda.", 'maybe i will slip in a few words about my experience ... during my first months in canada, that’s all i was talking about and i felt overwhelmed, so today i don’t talk about it much.', 'but at the time, i spoke about it often even to canadians and to my french friends so it wasn’t ... but they are removed from my experience, so the conversation takes on another shape and there is a very big gap between what they tell me and what i tell them.', 'still, they were listening, they were very ... they didn’t rush or pressure me, well, i have been pressured, but not ... not on the part of these friends.'], ['i don’t know [laughs].', "... it's a question of mutual acceptance.", "sometimes we can be suspicious, thinking that canadians haven’t lived through such major events, but ... they also have things to share ... i think it's a question of ... opening up to the experience of others.", 'but it shouldn’t go in one direction only: just because we’ve had a painful experience doesn’t mean that we should not listen in return.', 'i think that the history of canada also contains some traumatic events and i think that sometimes we must accept to open ourselves to positive experiences.', "and especially when we’re talking about a genocide, i think, it's an issue, a phenomenon that is of concern to everyone.", "so during those conversations, it's not only about listening or accepting my experience, but it's much more about using this experience in relation to others, since, i think, genocides continue to happen, but they don’t happen twice in the same place.", 'they always happen to other people, other places.', 'so, i think that not only canadians, but everyone has to become aware of our experience.', 'that’s the greater usefulness of witness accounts— if possible, to protect others in other places.'], ['[laughs] ... i don’t know.', "it's a difficult question ... i don’t know since ... i think that all rwandans, well, every rwandan has his or her own experience and i’m not sure that i should be asking them to think about me in a certain way.", 'perhaps i should be asking how we should all consider each other.', 'i think everyone lives their experience in their own way, and we should maybe try to listen to each other and abstain from judging each other.', 'what upsets me the most, i think, is to be judged: “oh, you’ll become mayibobo.” the perception that others have of you can really weaken you and you might think, “oh, maybe ... i really am like that.” we live in a society and, unfortunately, one downside that i ... that i’ve noticed often about our community is that we’re always looking on a surface level and ... not everyone is like that, but in general it seems that we’re focusing on what is wrong with the other person.', 'so, in this case [inaudible] we need to always make the effort to bring out those qualities in others that can help them advance, not the ones that can knock them down.', "i've been knocked down to the ground enough times that i don’t see the point of, for example, sitting around and, you know, criticizing others: “you know what?", 'oh, now [inaudible] is constantly drinking.', 'but why do they drink?', "it's ... [inaudible] of orphaned children, it's a catastrophe!", 'all they do is make babies and do drugs.', 'why do they do drugs?” why don’t we try to find out what’s wrong, instead of…, i mean, to try to focus on the good things.'], ["i don’t mean to say that it’s always the case or that everyone is like that, but it's something that i have noticed occasionally."], ["it's also normal that we are more curious about the things that are not going well, but it's just a matter of going further, of ... not stopping at that ... you can ... in the orphan village where my brothers and sisters live there are, say, two or three young mothers.", 'when we see that the girl is pregnant, we might think that, for example, only orphans get pregnant.', 'but that’s far from the truth!', "and it's not just the orphans who drink a lot.", 'people whose parents are still alive drink too.', 'for me, that’s …'], ['there you go.', 'so that’s something that often makes me mad and i don’t have an outlet to express this anger, but it makes me mad.'], ['i feel very nostalgic for certain places and people.', 'i’m always afraid when i’m in rwanda because i know that there are killers walking in the streets.', 'well, people who have killed back then and i don’t know if they won’t kill again.', "so i'm afraid.", 'but it’s also with great joy that i return to the places i frequented for the most part of my life.'], ["... not at my grandmother's, i haven’t been able to go back there, but every time i visit rwanda, i go to gisenyi, i go to nyundo, i visit the orphanage and i stay in kigali."], ['no, because when they were killed in 1994, their bodies were immediately thrown into a mass grave dug in the cemetery, so we don’t know where they are and since it was in a cemetery, we’re not able to disinter the bodies.'], ["no, but what i did find out, although i don’t know if it's true, is that one of the killers died in congo.", 'a gentleman came to see me after the genocide to ask me to testify against someone who had been imprisoned and who had taken part in the killing.', 'but when he came to see me…, well, i wasn’t there when my parents died, so it was difficult for me to testify against whomever ... i didn’t know that person, i had never seen him.', 'but for the moment, we are looking for the bodies of my twin sisters who died in bisesero.', 'we haven’t found the place yet, but we’re looking.'], ['no, and for the time being, i don’t think i want to, anyway, at this point in my life, i don’t think i want to meet them.', 'i know that my brothers sometimes go back there.', 'and when i go back to gisenyi i ... can’t bring myself to ... i went once where we…—actually twice, once to see if there were any photos left and a second time with my friends from france to show them the place.', 'but we didn’t ... i didn’t stay long.', 'i saw the neighbours who are still there, but i’m not really interested in ... staying in contact with them.'], ['yeah, i don’t want to meet them, so i don’t know if i would have the strength to go to the gacaca, but one of my brothers has gone.', 'the people from the neighbourhood where we used to live were denying that we had lived there and were saying that they didn’t know our parents, so my brother had to go there.', 'and while he was there words were exchanged that didn’t really make sense.', 'the neighbours were saying, “oh yes, that’s right, you are the son of [inaudible].', 'oh, i wasn’t here during the genocide, i was ...”—i don’t know where they said they were.', 'and so they started admitting that our parents really lived there, that we lived there.', 'but they were also saying that everyone had moved away by the time my parents died.', "it's a difficult subject.", 'i think that the survivors would like to know more about certain aspects of what happened—for example, where the bodies are.', 'but i don’t think that it really…, i don’t know if it has a healing effect.', 'someone who’s actually been at the gacaca … will be able to say more about it, i think.', 'since i’ve never taken part in the gacaca, i can’t say ...'], ["it concerns me personally and collectively speaking, so ... if i try to understand the politics behind it... i don’t really understand it ... i can’t find the words, but ... personally, i'm opposed to forgiveness, i’m opposed to reconciliation.", 'that doesn’t mean that my heart is filled with hatred and that i want the killers to be killed, no.', 'but i think i prefer ... to live with whomever i want to live, whether they are hutu or tutsi, but that it is me who decides.', "so if we choose to live with each other, we know we'll have to deal with those questions and that’s a personal choice.", 'but i can’t see myself meeting someone who has killed my father, who has cut him with a machete, and who says to me, “i cut him with the machete, forgive me.” so, i think ... i think, i won’t be going any further with ... the gacaca.', 'i see it as a policy that perhaps restricts choice.', 'if i had to give a choice—the people who held the machetes and who were killing for months and months should be relocated and live elsewhere.', 'i think that the survivors of the rwandan genocide are a sacrificed generation, they see their killers day after day.', "i think it's like ... inadvertently, this might be reinforcing their denial.", 'with a crime like that ... justice has to be done first and i don’t know if the gacaca bring real justice.', "and for this too it's not really up to me to say, because i don’t have direct experience with it."], ['i don’t know if i would address the young survivors specifically, since everyone is dealing with this experience in their own way, so i don’t want to assume the role of a counselor.', 'but from my personal experience, some of which is shared by others, it is more important to convey a message ... to the young survivors’ circle of friends and family.', 'and of course the survivors ... i think that every survivor’s story must be heard in the singularity of experience that it recounts, so i can’t come forward as an example or a model.', 'or if i am to be such an example, it is up to the young person to request it—it shouldn’t come from me.', "i think that's the way to do it.", 'but, more important to me is the message to [inaudible] the survivor’s circle of people.', "it's ... to really listen intently, to try to understand.", "the experiences we've been through ... can change our outlook on life, can change our ways of ... our behaviour.", "so it’s up to the people around us to try to understand what is going on ... because what's happening is not, sometimes ... necessarily under our control, you know, and so i can encourage the survivors to say nanana but ... [break in the recording]"], ["it's really a question of solidarity and ... understanding each person’s experience in its singularity, even though it’s an experience shared by many survivors ... i think."], ['i really don’t know if i would live there.', 'until i graduate, i’ll continue going back and forth.', "i don’t know where i'm going to settle in this world [laughs]."], ['... i think i need to continue with the process of recovery.', 'i love rwanda very much and i go back as much as i can, but for the moment i’m not planning to live there.', 'that doesn’t mean that i won’t think about it tomorrow or in the future.'], ['not really.', "but, it's a question of how i feel when i'm there and how i feel when i'm here—that's what will determine my return there or my choice of another place to live."], ['i didn’t think about the questions [laughs].', 'i didn’t really think about the questions.'], ['yes, i know, but, no, i think ... i think we covered everything ... thank you.', '[laughs]'], ["... at times, it feels good to talk about it, but the moments right before, the preparatory stage, if you will, when we know we're going into the conversation, is not always easy, especially when we talk about it often and when we become ... overly solicited [for interviews].", "it's like ... it's as if we [inaudible] since we continuously go back to it, even though there are variations in the story, we often recount the same things.", "at the same time it's up to me to decide not to talk about it.", 'but sometimes, because we’ve become used to it, we can’t bring ourselves to say no.', 'but ... at a certain point, it’s mentally exhausting.', '[end of the session]'], ['(y): okay.', "my name is  ishimwe, i was born in kigali in a family of four children and i'm the eldest, i have two younger brothers and one younger sister and… that's it."], ['yes, that’s right.'], ['i remember really well my maternal grandparents.', 'i didn’t know my paternal grandfather, because, as far as i know, he died when i was a little child, or even before i was born.', 'but i knew my maternal grandmother who also died when i was—if i remember correctly—10 years old.', 'so i remember her vaguely, yeah.'], ['whether i think about my maternal grandparents or my paternal grandmother, i remember the summer vacations, because we went there every summer.', 'i remember how impatiently they waited for us and i remember especially the traditional dishes that they would make for us, even if we didn’t like eating that food very much.', "but they would prepare them with so much love… that's the memory i have engraved in my mind—how they wanted to make us happy, even though i— [laughs] i didn’t like the food very much, but it’s something that i remember, yeah."], ["i know that mom was her dad's favourite.", '[laughs] other than that—well, it was a pretty normal relationship.'], ['as i mentioned, we visited them every year and every time someone i knew was going to their house, i always took the time to write each one of them a letter, telling them how i was doing, how my life was going, asking them for news about themselves.', 'i know i took the time to do this at least three times a year.'], ['my father was a doctor and my mother always worked for an ngo, a non-governmental organization that helped orphaned children or children who came from families—who couldn’t take care of themselves.', 'she was project coordinator there.'], ['yes.'], ['my mother worked during the day, while we were at school, so when we came home after school she was always there.', 'sometimes, my dad worked at night because he was a doctor, but that wasn’t his regular schedule, it didn’t happen often… so almost every night we were at home with our parents.'], ['it depends on the activity.', 'i remember that my mother taught me how to do “girls’ stuff”—how to sew, how to make the bed, how to fold clothes, how to sweep, how to clean, how to… these are things that my father never taught me, it was my mom who showed me that.', 'while my dad—he was telling me about history, he was showing me some technical stuff—that was more his domain.', 'in terms of school work, my father was helping me with math and related subjects and my mother, with language and other related subjects.', 'so i knew that for math homework, i could ask my father for help and when it was something in french or kinyarwanda, i could ask my mother.'], ['as parents?'], ['i would say that as parents they were very strict.', '[laughs] i don’t know how you say that in french—the kind of parents who would look at you from the corner of their eye and, without them having to say anything, you would know what mistake you’ve made and you would correct the mistake at the same time.', 'it was—in any case, i think—not that i regret it, but i think we had a very strict upbringing.'], ['yes, we were spoiled as children, but that didn’t mean that we weren’t punished when we made mistakes.', 'but when we weren’t causing any trouble, we were pampered.', 'we ate out at the restaurant at least once a week, on saturday nights.', 'we rented movies—not during the week because those were school nights.', 'and we went to the park on saturday evenings.', 'we went on airplane tours.', 'we were more than spoiled, i would say.'], ['i started kindergarten when i was three years old and when i was five, i think, i started elementary school.', 'i finished it at 12. i started high school at the age of 13. so, the first four years of school was the common core education, without specialization.', 'later, when i started fifth grade—since i was in a high school in benin—we had to choose a specialization in fifth, sixth and seventh grade, and i continued in accounting.'], ['when the time came to make a choice, i didn’t know what [laughs] to choose, but i was—because i was quite good at math, i went into accounting.', 'at first, i wanted to study medicine, but i didn’t like—for that i had to take physics and i hated physics.', 'so i decided to study something that was easy for me and i didn’t have to deal with physics, so i went into accounting, and that’s how it happened.', '[laughs]'], ['in the neighbourhood, i would say that what “well-to-do” meant was relative, but we were—some families were richer, others poorer, so i would say ours was maybe in the middle.', 'at school we were—i went to school with the children of ministers and so on, so i would say that my classmates’ families were much better-off than us.'], ['no.', '[laughs] no.', 'so the way we were taught was that our parents would come to pick us up after school and we went straight home.', 'we weren’t allowed to go anywhere unless our parents came with us or unless we were going to visit cousins or family friends with our parents.', 'the only friends we had in the neighbourhood were our neighbours with whom we could talk over the fence.'], ['i’ll express my personal opinion about this.', 'i’d say that it wasn’t only a question of education in the sense that parents didn’t want their children to socialize with—i’d say, with just anyone… i think they were also concerned about the children’s safety.', 'these are the two reasons that come to mind.'], ['actually, after the genocide, the ngo where my mother worked opened a branch in benin.', 'i don’t know how it happened and my mom had to—in any case, she was transferred from rwanda to benin and we had to leave with her.'], ['the moment that always comes to my mind is when my mother told us we were leaving.', 'she told us—because we were going to school at the time, i think it had been a few weeks since the beginning of the school year—she told us, “you’ll have to study french harder, because in your new school you’ll be studying in french only.” the idea of leaving was exciting to me, but the fact that i was going to be in a school, in an environment where we only spoke french and that maybe i wouldn’t be able to express myself correctly was scaring me.', 'apart from that, the other [?]', 'was leaving the people i’ve grown up with, my cousins, and knowing that i wouldn’t be able to see them anytime i wanted, that was also bothering me a little.', 'but i was delighted by the idea of \u200b\u200bchange and exploring something new.'], ['for sure, the beginning was difficult for me and my brothers and sister precisely on account of the language because we were in a place where we didn’t speak the language, so it wasn’t easy.', 'we arrived in january, so right in the middle of the school year, and we had to adapt, stay on track, learn french, adapt and at the same time not fall behind, which was much more difficult.', 'it was also a cultural shock, because these are two completely different cultures, and settling down was difficult too.', 'but i don’t know if it was because we were young or because i myself was young, but i’d say that it didn’t take too long before i felt almost at home there.'], ['yes, i remember that when i arrived in the school i was introduced to the students in my class, and everyone was trying hard to help me, to explain things, to give me their notes to copy, and this felt good because when you go someplace new, you feel quite—you think to yourself, “will everything be okay?” and all that.', 'but when people would come up to me at recess and say, “don’t worry, you can copy my notes”—the teachers would also come up to me and say, “come to see me if you need explanations”—that was reassuring.', 'but the fact that we were far from our family, far away, and that we didn’t know almost anyone there, that part wasn’t easy.'], ['thirteen years old.', 'i celebrated my 13th birthday—it is in january—so i celebrated my 13th birthday there.'], ['yes, i started high school there.'], ['[laughs] i still wonder to this day.', 'because not only she was working and taking care of the children, i think it also fell on her to do the greatest share of the work of finding out how things functioned in the new place.', 'to this day, i don’t understand how she did it, with four children.', 'if i were in her place, i don’t think i would have succeeded.', 'i’m still asking myself this.', 'i think she is the right person to ask this question to.'], ['yes, many.'], ['not just from my mom [?].', 'but from her i think i have that go-getter attitude, i must have inherited that from her.', 'the resourcefulness, always wanting to go forward despite the obstacles on the way—this comes, i’m sure, from my mother.', 'another trait is the love for rwandan culture—that, i think, one hundred percent is from her, one hundred percent.', 'that little bit of calm disposition that i have, i think it’s from my mom too… [laughs] reading—i don’t remember if my dad liked to read much, but i know my mom loves it, so i have that from her as well.'], ['from dad?', 'first, him being a doctor, he was more interested in mathematics, chemistry, things like that and so i must have gotten that from my dad.', 'apparently, my dad also had a strong character, so that too must come from him… what else?', 'the side of me that wants success not only for myself but for the people around me as well, the side that likes to help others, the generous side—that, i think, i also carry from my dad.'], ['that’s correct.'], ['i’d say that when experiencing such atrocities i don’t think you make the effort to think about what will happen tomorrow, you live day-by-day.', 'actually, not even day-by-day, you live in the present moment, really, and you wait to see what happens next.'], ['i think that even me, even though i was young, i saw some signs, but i could have never, ever imagined that such a thing would happen.'], ['for two or three years before 1994 the national security in the country wasn’t good, there were always demonstrations.', 'the fact that at school the hutus were asked to stand up and people from the other ethnicity were also asked to stand up and both were counted—that was one of the signs.', 'and the fact that we had already been attacked once at home before 1994 also had to be a sign that something was wrong, yeah.'], ['yes, they would ask the students from one ethnic group to get up, they would count them and tell them to sit back down and then ask the students from the other ethnic group to get up, and so on.', 'and there was a form we had to fill in and we had to indicate our ethnicity, our names, our parents’ names, where we were born and so on and the question about ethnicity was also on there…'], ['i don’t know how i came to know it, but i did [laughs], i knew it.', "it's a form that we’ve had to fill in since we were young, when we were in kindergarten, i think, we were given that form to fill in at home and bring back.", 'and when we were in elementary school we would fill it in ourselves, and if we didn’t know how to fill in certain parts, we had to ask our parents and the next day we would bring the missing information to our teacher.'], ['personally, i was too young, i didn’t notice that.', 'it’s possible that was the case, but personally i didn’t realize it.'], ['how do you mean?'], ['so, how it happened… i remember, it started in the evening, we were having dinner, it was around 8 o’clock in the evening, my paternal uncle was there with us, on his vacation.', 'we were at the table, having dinner, and then we heard the sound of the plane.', 'i didn’t realize it was the sound of the plane because we were always hearing that kind of noise, grenades, here and there, and i thought that it was just the usual.', 'then my father asked us to go to bed—actually, not to bed but to go under the beds, because every time this happened, we were told to hide under the bed until my parents would come and tell us to… and then, a few minutes later, we were told: “come out from under the bed and go to sleep.” then they woke us up only the next day—because i was getting ready—i remember, i was preparing to write the final exam at the end of high school [translator’s note: or elementary school?]', 'and i was taking classes—it was easter vacation, but i was taking extra classes to prepare for my exam.', 'so when they came to wake us up, i got up quickly to go take a shower, because i thought i was late for school.', "and my dad followed me and said “no, you’re not going to school today.” then i saw that they were laying down mattresses in the corridor, and my father told us, “sit down, we'll explain what's going on.” so they laid the mattresses down, they arranged all that and said, “in fact, the president's plane has been shot down and things are not going well at all for the moment, we're going to stay here until the situation improves.” i was so young that i didn’t understand how great the danger was.", "i said, “so i'm not going to school?” my father said, “no, you’re not going to school until things get better.” i said, “okay, no problem.” and so we stayed in the house.", 'the plane was shot down on the 6th at night and we stayed there until the 11th.', 'it was on the 11th that we were attacked in the house.', 'i remember that when it happened my father went out, we stayed behind—no—yes, we stayed behind—my mom and my brothers, my sister and uncle.', 'just before he went out, my father said, “let’s sit together and pray.” we took a few minutes to pray and then my father went out of the house.', "my mother told me to get my brothers and sister and go hide under my parents’ bed, then she closed the door and said, “i'll come to get you soon.” we stayed there, but i could hear noises, there were noises."], ['i remember i came out from under the bed and i went to the window.', 'i saw my father being struck, and i went back under the bed.', 'but i didn’t say anything to my brothers, i was speechless.', 'a few minutes later, someone entered the room.', 'my sister said, “daddy is back.” but it wasn’t our dad, it was someone else.', 'he started searching, rummaging around the room… and when he looked under the bed—there we were.', 'he told us to come out, and so we did, he told us to go outside, and we followed.', 'so we were there and i didn’t know what had happened to my mother, i didn’t know how my father was doing, and all that.', 'i didn’t know what had happened to my uncle or to the servants either.', "we were standing in front and then the man asked me, “are you the eldest child in the family?” i said, “yes, i'm the eldest.” he said, “where do your parents keep the money, the guns, and all that?” i said i didn’t know, i said that i had never seen any guns in the house.", 'i said that i really didn’t know.', 'i told him, “you can ask mom or dad, i don’t know.” then he began to tease me: “oh, those rich kids, it’s always the same thing.” he told many stories and then he said: “anyway, it doesn’t matter if you know or not.', "we’ll kill all of you and then we’ll search the house and we’ll find it.” i said, “i can’t tell you anything because i don’t know.” then he told me, “before we kill you, you’ll take this gun and you will kill your brothers.” i told him, “i don’t know how to use a gun,” that's what i said.", 'and he said, “oh yes, you do know.', 'you, the inkotanyi children, you all know how to use guns.” so he handed the gun over to me, i remember that it was very heavy.', "he said, “so, there, you’re going to kill them quickly, then we’ll kill you, then we'll get on with what we need to do, and then we’ll leave.” i remember that he gave me the gun and i gave it back and said, “look, do what you want, i can’t kill my brothers.” the man said, “oh no?", 'in any case, if you don’t kill them, we’ll kill you all.” and i said, “do what you want, i’m not going to kill my brothers.” so he stayed there and a few minutes later, my mom came out of the house accompanied by somebody else, i think he was the leader of the whole gang.', 'they came out and he said to the guy who was with us, “leave them alone, i made an arrangement with the lady, let them go.', 'i don’t think they’ll go far, anyway…” so my mother said to me, “we’re going to the neighbours’.” we passed through the fence and we went to the neighbours’ house.', 'of course, when they had heard that we were being attacked, they had fled, so when we got there, the house was empty.', 'meanwhile, i think they had told my mother that my father was dead and my mother could not stop crying, and we didn’t understand why… we kept asking her, “mom, what is it?', "what's wrong?” and she kept saying, “it's going to be all right, it's going to be all right.” [tears] [temporary break in the recording] afterwards, when my mother—my mom calmed down later and she told us, “your father is dead.” in the meantime, while my mother was out, my uncle had also left."], ['they had tried to cut off his arm and he was bleeding; we tried to stop the blood with a cloth.', 'we stayed in the house, we waited for the neighbours to come back, we just stayed there, and then, much later, they returned.', 'so the man—in the house where we were—he said, “you can’t stay here because they know you’re here, and if they know it, we’re next.” and then even i said, “where do you think i can go with the kids?” he begged us to leave and then he said, “okay, no problem, you can sleep here tonight, but tomorrow morning you’ll have to start making arrangements and see where you can go.” so, we went to bed and i think that during the night, my mother and my uncle went to the neighbours’ to see if there was a place for us to stay, but—and i found that out the next day—that plan didn’t work.'], ['and i know that in the house where we were, the whole family was about to leave and go to butare.', 'so mom asked them, “can you take us with you?” but the man said, “listen, we can’t,” and so on.', 'my mother then asked if they could at least drop us off at the house of my mother’s older brother who lived in kiyovu.', 'then suddenly it started to rain.', 'it was raining heavily.', 'now that it was raining, the man said, it was certain that the guards at the roadblocks would leave their posts and so we could go, but he would only drive us to my uncle’s.', 'so we got to my uncle’s house, we knocked on the door, we knocked, we knocked and there was no one… fortunately, there was a small opening at the bottom of the fence.', 'my mom said to me: “go and see if there’s anyone in the house.” i passed through and i went upstairs.', 'my uncle and his family weren’t there, only the guard whom i knew and who said: “listen, unfortunately, you can’t stay here because there is someone watching the neighbourhood and he knows that there are two of us here in the house, if they find you here, they’ll kill us all.” so i went back and told that to my mom and the others.', 'in the meantime, my mother had an idea—my uncle’s family had a friend who worked at the belgian red cross, which was not too far away.', 'so mom said, “you’re going to drop us off there and the red cross probably—they will shelter us.” so, they dropped us off there and they left.', "once we arrived, we asked for the friend of my uncle’s family, he came out, saw us—he knew us—and he said: “unfortunately, they don’t shelter refugees here.” he said, “there’s another branch of the belgian red cross further down, we will send you there.” my mom said: “with all the roadblocks, we won’t be able to pass through.” and he said, “okay.” he took one of the cars of the red cross, we got in, and he said, “listen, we're going to pass through as if we”—we were hidden at the back of the car—“we're going to pretend that it’s a red cross vehicle passing… if they stop us, unfortunately there is nothing we can do, but if they don’t stop us, great.” but it was still raining so every time we came up to a roadblock, it was empty, so we passed through and they left us there."], ['they dropped us off at the red cross and there were other refugees living there, about sixty or so people, families, and so on.', 'so they left us there and that’s where we stayed.', 'in the beginning, everything went well, we had food to eat, but sometimes there wasn’t any food left and they told us that we would divide the work by family.', 'each family—they said that we were going to have a schedule of people who would do the cleaning, people who would do the dishes, people who would cook—who would go get food, people who would go get water.', "because i was the eldest in the family, i was told, no problem, i can help with the cooking, but for those going out—my mom couldn’t go out, and so they said, “look, you're the eldest, you have no choice.” i said, “okay.” anyway, i was young—getting water?", 'no problem!', 'but we had to get water for bathing, for washing the dishes, for cooking.', "so i said, “listen, i don’t mind going, but how will i manage?” they said, “no problem, we’ll think of something.” then they said, “we'll do two rounds because not everyone can go out at the same time.” and since they designated only one family member per activity, i said: “i’ll go out in the first round and will come back in the second, it’s not a problem.” someone was surveilling the neighbourhood.", "apparently, they were going to kill us at the very end, after they had finished killing everyone else, we were to be the dessert, that's why—because no one was protecting us at the red cross, and there was this man who told us that they were going to kill us in the end."], ['there was a roadblock right at the entrance of the red cross; so every time we went out to get water, this man accompanied us, we would get water and come back.', 'so i would leave with the first group, give the water to my mom and come back with the second group, and so on.', 'one day we went out and they started shooting at us.', 'when shots are fired, everyone starts running here and there.', 'i remember i ran, i went into a house, i didn’t know where i was, i climbed a tree and sat there and waited.', 'they were shooting, shooting, shooting... at one point, i don’t know what happened, but the shooting stopped.', 'quietly, i climbed back down from where i was, in the tree, and i was looking for the man because i knew that he was going to protect me—and i had been tracking his movements with my eyes.', 'so when everything calmed down, i climbed down and followed the man.', 'everybody came out of their hiding places except one guy who was shot and we took him back to the red cross.', 'after that, we just continued on our way to get water and then we returned to the house.', 'i think my mother was scared and she said: “i don’t think you’ll be going alone next time.” so from then on, my brother went with me.', 'and we continued going as usual.', 'one morning, apparently someone from the red cross had fired bullets at the roadblock at the entrance.', 'so the interahamwe came and said: “we won’t wait until the end, after all.” we were all taken out of the house, we were taken outside and they said: “say your prayers now, we will kill you.” we were all sitting in a row, i remember i was with my mom and i don’t know where my sister and my brothers were, but in any event, we were the last ones in the row.', 'i don’t remember where my brothers were.', 'we started praying, and then they killed the first person in the line, i think it was a mother with her baby, then the second, and then the man who was surveilling the neighbourhood showed up.', 'he said, “but why are you killing these people?” and they said, “wasn’t it them who shot at us?” the man said, “no, no, no, i know this house, i’ve searched it, there are no guns here,” and stuff like that.', 'anyway, he said, “i told you, we’ll kill these people towards the end.” and so they let us go, we went back into the house.', 'this happened, i think, twice.', 'the first time, they killed people, but the second time one person was wounded, but no one died.', 'so we stayed there and i continued going out as if that was my normal.', "i said to mom, “don’t worry, i'm not going to die.” i don’t know why, but i was sure i wasn’t going to die."], ['all three [translator’s note: or five?]', 'of us stayed there.', 'i went out every day to fetch water, to get some food with the others.', 'even when my brothers didn’t come with me, i would still go, because i was sure that nothing was going to happen to me.', 'i went out every day.', 'i saw the killings in some places, but i had gotten to a point where it didn’t shock me anymore, they could shoot someone and i could pass nearby without a problem.', 'as long as they didn’t touch me—that’s what was scaring me.', 'my mother would ask me, “have you seen the dead people?” and i would say, “they are everywhere.” it has become just a normal thing for me that one person would kill another; i saw a woman being raped and it seemed normal to me.', 'all i was doing was to go out, get water and bring it back, because i didn’t want my family to starve.', 'then finally, one morning we woke up and the roadblock was gone.', 'someone looked out the door and the people who were guarding the roadblock were no longer there.', 'then i think someone turned on the radio—the radio wasn’t working.', 'we wondered what was going on, and, i think as a reflex from the war, we said, “let’s get ready, something is happening, we should leave.” we got dressed, we started packing a few small things, because we thought we were going to change locations, and while we were getting ready a soldier arrived.', 'i remember my mother cried out, “it’s the inkotanyi.” [laughs] and my mom said, “don’t take too many things with you because they [the soldiers] have arrived.” i remember i was packing and i was asking, “mom, should we take this?” she was saying, “no, no, no, don’t worry, we’re safe.” [laughs and tears of joy] and then she said to me and my brothers, “don’t worry, everything is fine.” i remember we were the second ones to go out, there was one guy in front of us and then me and my mom, and she said, “go where this gentleman tells you to go, because it’s fine, we’re safe.” [laughs] then they took us to a place called [inaudible].', 'everyone from kiyovu was there, well, from the neighbourhood anyways.', 'they took us there, they gathered us to tell us that the war was over and that we were going to stay there until they found a safe place for us to go because there were grenades everywhere and it wasn’t safe.', 'i remember that they brought us food, they gave us to eat, and told us that the next morning we were going to kicukiro.', 'we were to walk from kiyovu to kicukiro.', 'i said, “mom, how are we going to walk?” [laughs] mom said, “now that we’ve gotten this far, we’ll do whatever it takes, if we have to walk, you’ll walk.” mom said, “in any case, don’t take too many things with you because we’re safe and these people will feed us at the place where we’re going.” the next day they told us, “it’s time to go.” we gathered our things, yes, and we set off for kicukiro.'], ['and everyone—each family was assigned to a house.', 'we were told, “this is a safe place, you’ll stay here a few days until we secure the rest of the city.” i think we stayed there for at least—i think a week, the equivalent of a week.', 'after that they said, “you can now go back to your homes.” when we returned to the house—i wasn’t—my mother was accompanied by someone, i don’t know who—apparently, we couldn’t live in our house because it wasn’t a pretty sight—in any case, i didn’t know what it looked like.', 'we were told to go sleep at my aunt’s house.', 'when they had brought us all together in that place, we reunited with one of my aunts.', 'she had lost her children.', 'she was my godmother.', 'she [yvette’s mother] told us, “you, your sister and your brothers are going to stay with your godmother and i will stay with your other uncle” with whom we had also reunited.', '“we’ll clean the house and once it’s ready you’ll come too.” i think i stayed with my aunt for two days, the time it took to put the house in order a little bit, and then we returned home.'], ['speaking for myself, i think that it was like the two sides of a mirror: i was a spoiled child who had suddenly become an adult.', 'i realized that life would never be the same.', 'i don’t know how i understood that, but it was like a mirror, yes.', 'we stayed there, we tried to clean the house—the places where there was blood, all the books that were on the floor because they had ransacked everything.', 'we cleaned the house and, fortunately, not long after, the ngo where my mom worked restarted—they opened for business again and mom had to go back to work.', 'since i was the eldest in the family, i had to take over the responsibilities of the house, i had to make sure that we had something to eat, i cooked, i cleaned the house… i had to get my brothers organized—i knew that this brother had to get up and clean this room or that brother had to get up and do something else.', 'i knew when we had to go get water and i learned to clean and cook by myself.', 'i was trying to remember what our servants did when i was little and when i was watching them work.', 'i was improvising.', 'in any case, i know that when mom would come home from work, everything was in order.', 'how did i do it?', 'i don’t know.', 'but i knew that it had to be done, because mom was working, she couldn’t be in two places at the same time, so i learned all of this by myself.', 'of course, when mom was home, i would sit next to her, ask her questions: “how do i do that?', 'how is this done?', 'what do you think i should do tomorrow?” she would tell me, but i knew that i had to take on responsibility for everything to be in order when she came home from work.'], ['i think we went back to school—if i remember correctly, it was two months later, i think in october?', 'yes, i think it was october when we went back to school.', 'two months later.'], ['yes, i went back to the same school.', 'of course, i didn’t know 90% of the faces there.', 'i remember there was a girl i went to elementary school with.', 'when i saw her after the war—she was the first one i saw—i think we hugged and we stayed in each other’s arms for a long time because she was the only one i knew from school from before.', 'and to see her alive, it felt good.', "and i remember there was a teacher who came up to me and said, “no, no, no, you can’t be alive.” i said, “yes, i'm here.” there was also a boy who was back—i remember we were in fifth grade together before the war.", 'i didn’t recognize him because they had cut him all over his body.', 'i didn’t recognize him, and the girl asked me, “did you recognize ange?” i said, “that’s ange?', 'no, i didn’t recognize him.” i felt so bad for not having recognized him, someone who had been part of my life from before, who felt much closer to me after the war than the others, especially those returning from outside of the country—he was like [part of?]', 'my family.', 'not recognizing him really affected me deeply.', 'but, for sure, when school started, those of us who were in rwanda after the war, we stuck together.', 'the people who came back from burundi, they kept their distance.', 'and so did the people who came back from uganda.', 'that’s how it always was.', 'and i remember our conversations: “how did you survive?', 'what happened?', 'how are you doing?', 'are your parents still alive?', 'are your sisters still alive?” that was it, at least for the first two weeks we only talked about that.', 'about nothing else.'], ['sixth.'], ['yes.'], ['i think, from the teachers who had taught me before, there was only one at the school after the war.', 'only one was there.', 'there were five sixth grade classes and we had about five teachers, and i knew only one of them from before.'], ['i imagine—i think that it wasn’t easy not only for the students but also for the teachers, because not only—i’m looking for the word in french—guhahamuka—'], ['the trauma, that’s it.', 'it was a recurrent thing.', 'you would be studying and all of a sudden you would hear someone scream—things like that.', 'it affected us, but i think it affected the teachers more.', 'they not only had to ensure the well-being of the students, to teach them in class, the class material and all that, the knowledge they had to transmit, but also take care of these children who were not doing very well.', 'so it was a combination of these three things, and i imagine they themselves were not feeling—anyway, i think that, psychologically, they weren’t able to combine these three roles.', 'i guess they had to do it, they had no choice.'], ['trust—my thinking was, “i have to go to school” and “life goes on, you have no choice.” whether you trusted others or not, you had to fight, you had to go forward.', 'whether you liked it or not, you thought to yourself, “yes, what happened, happened.” now i understand: whether we like it or not, life continues, we had to fight to survive and continue to live.'], ['in elementary school, this teacher—the fact that after the war, he came up to me and said, “you are alive?', "i'm glad you're here.” that touched me.", 'it felt like there was at school, besides my friends, an adult that i knew or at least someone i could talk to, someone who knew me from before.', 'and what he said was reassuring: “if there is anything i can do, i’m here for you, i can help you.” it affected me, but it didn’t—it touched me, but not so deeply.', 'for me, at the time, that wasn’t essential.', 'i told myself that i had to live and the rest was not that important.'], ['i could see how my mom was fighting and i was thinking, “if she’s fighting like that, me, as the eldest in the family, i have—,” i thought i had the duty not only to continue going to school in order to succeed, but also in order to help my mom.', 'i was thinking that she couldn’t continue going it alone.', 'that’s what motivated me.', 'and i was also thinking, “i am lucky to still have my mom, there are people who have lost their mothers.” so if i still have her and she is taking care of us, i thought “i have to do the impossible to move forward, to succeed and to lighten her load.” that’s what motivated me, because i remember that before the war, i went to school for the sake of going to school.', 'i remember that if i placed 19th or lower in my class, that was a perfect score for me.', 'as long as i wasn’t 20th, because i thought that being in 20th place was low, but up to 19 was perfect.', 'i had never wanted to be first, second, all that.', 'i wasn’t concerned with this because what mattered for me at school was to see my friends.', 'and since i knew i was going to be scolded at the end of the semester [if i had bad grades], i would do everything i could so as not to be 20th in my class.', 'but after the war, right after the war, i was first in my class because i understood better than before the use of school.', "that's why i said that before and after was like the two sides of a mirror."], ['no.', 'no, not yet.', '[laughs] i hope to, someday, but no, i haven’t reached it yet.'], ['no, we never found his body, we were not able to bury him.', '[people say?]', 'that he’s in either nyanza or kicukiro[?', '], but we really don’t have any confirmation about that.', 'we tried—we know that the person who killed my father is still in kigali.', 'apparently, during the gacaca—i wasn’t there—he was presumed innocent.', 'the trial is still going on, it’s not over yet.', 'but to date, unless he accepts his wrongdoing and tells us what happened—where he [the body of yvette’s father] is now, so far we don’t know.'], ['speaking for myself?', 'i don’t think i would have the strength to pursue this, i don’t think i have the energy.', 'i appreciate the people in kigali who follow up on this, it’s good if they do it, but personally, i don’t think i have that energy.', 'the only motivation that would make me do it is to find out where the body of my father is, but the rest—attending the gacaca and all that—it’s good, but i don’t think i have enough energy to do that.', 'even if i were there, in rwanda, i don’t think i would have the energy to do it.'], ['ideally, the killer would admit it and he would tell us exactly what happened after we left the house, so that we know.', 'that would be the only way.', 'or, if all of a sudden people who were there at the time and who know what happened—if they tell us… that would be the only way.', 'but if the only way, really, is to find out through that person, i doubt that i would be able to do it.', 'not me.'], ['i did, i went back in 1997.'], ['no.'], ["it's a good question.", 'i don’t know.', 'we think we’d meet them, but meeting someone who says “no, i didn’t do that,” yet we know they have, it’s also—in any case, i am reluctant.', 'i am not so sure.', 'but sometimes i think to myself: “i would like to meet this person and look at him.” to be face-to-face with him, to at least ask him a few questions.', 'but i think it would anger me more, it would make me even more—if he says, “no, i didn’t do that,” i don’t know if i could handle it.', 'knowing that he’s done it and that he says no, that’s what i doubt very much.', "i find it’s too—i think it's the highest level of viciousness.", 'we’re not asking for much, we want to know at least where the body is so we can bury it.', "but to meet with someone who might know where the body is and would not say, i'm not sure that i could handle it."], ['i lived there for a few months before going to benin, but i think i was more—i didn’t really think about what had happened.', 'i was trying hard to look on the bright side, to see my family that i hadn’t seen, my uncles, my aunts, my cousins… i don’t think that—i was trying not to mention it.', 'and to this day, i’m still not sure if i would go back, i don’t know what it would be like.'], ['yes.', 'but when, i don’t know.'], ['whether i like it or not, rwanda will always remain my country.', "that's one thing.", 'two, i still have relatives there.', 'but the other thing that i really want to do one day—and to have the courage to do—is to go, but not only to be there, but to be there in april, that would be ideal.', "i would like to relive that month as i lived through it in 1994. maybe it would be a kind of therapy for me, i don’t know, but that's what i'm going to do, try to live through—to remember what i was doing every day—or at least live for a few days there during that month."], ['visit the places, go to the memorials, return to the neighbourhood, yes.', 'all of this is what i would like to do.', 'when?', 'i don’t know.'], ['i came to montreal in 2001. it might sound crazy, but i have always loved canada.', 'why?', 'i don’t know.', "i was hearing montreal french being spoken, and i had a cousin who lived here and he told me, “listen, it's nice here, it's good, you'll enjoy it, come.” when i arrived, i came for—i said, “i'll try it for maybe a year.", "if i like it i'll stay, if i don’t like it, too bad.” and now, it’s been almost nine years.", '[laughs]'], ['yes, i would say yes, i loved it.'], ["it's like i say, it was the first time that—pardon me—that i was leaving my family, and i thought it was a crazy thing to do.", 'i wasn’t even eighteen years old yet, it was an adventure.', 'it was absolutely an adventure.', 'i didn’t know what i was getting myself into, i didn’t have any expectations.', "i thought, “i’ll go and then we’ll see.” i had no expectations, i didn’t know what i was getting into and i thought, “i'll go, if it works—great, if it doesn’t work—hey, we'll try something else.”"], ['i think it was hard for her to let me go because it was the first time she was separating from one of her children.', 'i don’t think she realized it until the last day or maybe she was hoping i would come back.', 'but i was thinking, “not being next to her doesn’t mean that i’m not helping her.” and i was also thinking: “i want to go there, i want to leave for—.” as the saying goes, you have to take a step backwards in order to make a better jump forward.', 'in my mind, that’s why i did it.', 'it was crazy, because i was young and i didn’t have a precise plan and all that, but it was really with the intention to move forward and become someone, to be able to reach my goals, that was the reason why.', "it was really a crazy thing to do, leave your parents while you are still under 18, go to canada, a country, i don’t know how many thousands—from your parents, it's crazy.", 'i don’t think that my children would do something like that.', '[laughs]'], ['because i was with my cousin, i had someone who was helping me do this and that.', 'that was helpful.', 'also i was still a minor and i went back to high school.', 'that was easy.', "that's what made my integration easier because when you start school, you make contacts.", 'i started school, i was at the end of high school, so i finished high school.', "and i remember, i started in grade 10 and when the school year finished everyone was looking for work and so i said “let's go!", 'let’s find a job.” i remember i didn’t know where to start.', 'i remember i was buying something at zellers and there was a line-up, a long line.', "and i thought, “these people are probably looking for cashiers.” [laughs] i went home, printed my cv, went back to the store, gave it to them and said, “i think you're looking to hire people.” i told this to the person there because there was a long queue.", 'and it was true.', 'i handed in my cv, say today, and two days later they called me, they interviewed me and i started work on monday.', 'so i worked there during the summer and in september i went back to school.', 'they kept me working part-time because i was going to school.', 'i went to school in the mornings and i was working a few evenings and on the weekend.', 'there was a project with the bank of montreal [bmo] through my school, the bank wanted to integrate young students into the financial sector.', 'so they would go into montreal schools and select a maximum of three students per school, and they chose me.', 'after they selected me, i did the internship once a week.', 'i worked at the bank as an intern and i learned in this way what the job of the people at the bank entailed.', 'and when i graduated from high school, they called me to ask if i was looking for a job.', 'i said yes.', "the woman who was coordinating the internship told me, “i’ll keep your cv, if you’re ever looking for a job.” and when they had a job opening she said to me: “listen, there are jobs that have become available, i'm going to send your cv, so expect us to call you if you are selected.” finally, they called me and i got the job at bmo, and i worked there.", 'so that was it, my integration.'], ["i'm not complaining.", 'it wasn’t that difficult.'], ['i was at école secondaire la voie and it wasn’t so difficult because i understood the language, people were speaking and i could understand very well what they were saying.', 'it wasn’t that difficult.', 'except for my shyness, which always gets in the way... but it was—i was doing so well that in the second year i was president of the student council, which probably means that if they had chosen me, i must have done pretty well, i’d think.', 'no, it wasn’t that hard.', 'i would say it was easier than the first time, but i’d also say that the language facilitated things, because i understood the language, so i went [inaudible].'], ['yes, precisely.'], ['it was easier, yes.', "it was really multicultural, it's true.", 'but i remember that the assistant director there also helped me a lot, because she would often stop by to ask me if everything was going okay: “are you having any problems?” i think she came to see me every day, and i found that also reassuring… when there’s someone you can turn to if something goes wrong, i think that also reassured me, yes.'], ['i don’t think that she did the same thing for everyone, but in any case not a day went by without me seeing her—i don’t remember.', 'of course there were days when she wasn’t there, when we had activities, but especially in the beginning, she would see me every day.', 'she would stand in front of her office, you know, and when the students came out of class and when i passed by i would say hello, she would call me and ask, “are you okay, are there any problems?”, things like that… she was my resource person, if i had any questions.'], ['yes, to the assistant director—i didn’t go into too many details, but i told her that i was rwandan.', 'she asked me if i had lived through the genocide, i said yes, that was all.', 'but i remember one time, it was during history class i think, we were talking about wars and all that, and they asked me to tell them what had happened in rwanda.', 'and again, i didn’t go into details, but i told them what had happened, and i said a few words about my own experience, so yeah, i’ve told the story at least once.'], ['it certainly changes something when you know what someone has lived through, especially if it has been something difficult.', 'as long as we don’t gloss over it and we don’t feel sorry the person… but for sure, it changes something.'], ['i did, i participated in one organization.', 'i participated once.', 'if i remember correctly, it was during the first summer, i was already working, and there was a youth centre across the street from where i worked, and i said to myself, “i’ll go and see what they’re doing there,” and i went in.', 'they said that, yes, they do activities to help young people integrate and out of curiosity i said, “i’ll join in.” i was—we were a group of young people, yes, we did some activities, we did small jobs.', 'i didn’t stay long, i think i stayed only for the two months in the summer.', 'i was just curious to see what they were doing.'], ['no, not at all.', 'not really.'], ["yes, i'm part of insangano."], ["it's a dance group where we do traditional rwandan dances and other activities.", 'we have a community newspaper, we organize christmas parties for the children, and so on.', 'and i’ve been part of insangano, i think, since 2003—no, 2004—i’ve been part of insangano.', 'i’m also part of page-rwanda, since 2005–2006, so those are the two community groups to which—i was also a member of the founding committee of the rwandan cultural centre, and i think that’s all.'], ['i know there’s another dance group, ihozo.', 'but i don’t know much more about them.', 'but i know they exist.'], ['absolutely.', 'yes.'], ['does one have to be rwandan to notice it?', 'because i am rwandan, i don’t know if people who see—but as a rwandan, yes, i am aware of it.', 'does one have to absolutely be [rwandan] to understand it?', 'i don’t know.', 'maybe yes, i don’t know.'], ['i think it’s just group dynamics.', 'bringing together the two ethnicities is not something that can happen in two years or five years.', 'if it is to happen at all, it will take time and there will be a learning curve, and it will be long-lasting.', "i don’t think it's something that should be forced or pushed either, we really have to let time do its work."], ['no, i would say they have changed, especially when i look at the young people.', 'the young people have the desire to make sure that our country advances.', 'we want things to be better for the country, we really want it to move forward, we want the rwanda of tomorrow to be a different rwanda from the one we have known.'], ['we did this as part of insangano, we had what we call tuganire, we sat down and we talked about what had happened.', 'and not only about those events—we could address all kinds of issues.', 'but we often talked about what had happened in rwanda, about education today compared to the education of our parents or our grandparents, yes, we often referred to that.'], ['yes, there is the urumuri mothers association, i imagine that they also do this kind of gatherings, yes.', 'but that’s all i know.'], ['i left bmo not long after 2003, and since then i have been with bell, since 2003, and i continued my studies in accounting, management and accounting.', 'i think i will be working more in the financial sector.'], ["i don’t think it’s been a failure, but i think—i hope—that i'm on the right track.", 'there is still a long journey ahead of me.'], ['what bothers me and affects me the most is the negationism, the fact that despite everything that has happened, that has been recorded on camera, the people who have lived through it, the victims who continue to testify, there are still people who dare say that there was no genocide—i don’t support that.'], ['when i watch movies about genocides—of course, a film can’t tell the full story of what happened during those three months and all over the country—does it represent it really or are we trying to represent reality… i think more work remains to be done on this.'], ['to be an actress?'], ['my god, no!'], ['[laughs] no, the cameras—i don’t think—i don’t like cameras and they must not like me either.'], ['personally, no.', 'i can give ideas on a project that someone else would like to carry out, but i don’t think i have the talent for that.', 'one never knows, but for the moment, i don’t think so.'], ['about me?'], ['the future is based on the past.', 'this means that, whether i like it or not, what happened is going to influence what i am doing now and what i will be doing in the future.', 'this means that what i have seen and what i have lived through will be reflected in my every decision or every action.', 'and i think to myself that, yes, what i’ve lived through was an atrocity, but at the same time it awoke a part of me that keeps telling me that life did not stop, life goes on and that we must not just survive, we must live.', 'whatever the obstacles you encounter in life, whoever you meet on your journey, don’t hesitate to move forward.', 'and i think—as they say in english, last but not least—i think that if i am who i am, if i am where i am, it is all because of god.', "and i say to myself, “everything i've lived through, everything i do, everything i know—if i didn’t have god, if i didn’t have prayer, i wouldn’t be here.” there’s no doubt about that."], ['my pleasure.'], ["thank you, that's very kind.", '[smiles] (end of interview session 1 of 1)']]

In [11]:
tension_paragraph = [['... the children also participated, so i think the work was ... shared.', 'so, when my mother got up at five-thirty, sometimes it was because she had to get—how do you say that in french?—because the bar was also serving urwagwa, mutzig and primus ...'],
                     ['three girls and five boys.', '[laughs]'],
                     ['yes, or maybe that’s only an assumption, so i don’t know [laughs].'],
                     ['yes.', 'yes, they were always ... they were good to me, even if sometimes they told us to stand up as groups of hutus and groups of tutsis—everybody would stand up, but we were always a small minority of tutsis so sometimes, [inaudible] the others were looking at us a little strange, but that was all it was.', 'so we knew there was something, but it wasn’t explicit in the way they looked at us.', 'and it was ... so, we would stand up and sit back down, and we would know in this way who was also tutsi, but that was all, we would then carry on with our day.'],
                     ['yes.', 'and i also don’t accept the term “parent” because you can’t be a real parent in this way.'],
                     ['actually, i think it adds a sort of [inaudible] to a person’s sense of self.', "i have a hard time thinking of myself as either a sister or a parent as [inaudible] it's ... i'm swaying between the two.", "and it's also paradoxical—now that i don’t have all these responsibilities, it's as if i’ve become childish, at times i feel more ... [laughs]."],
                     ['i don’t know if i would address the young survivors specifically, since everyone is dealing with this experience in their own way, so i don’t want to assume the role of a counselor.', 'but from my personal experience, some of which is shared by others, it is more important to convey a message ... to the young survivors’ circle of friends and family.', 'and of course the survivors ... i think that every survivor’s story must be heard in the singularity of experience that it recounts, so i can’t come forward as an example or a model.', 'or if i am to be such an example, it is up to the young person to request it—it shouldn’t come from me.', "i think that's the way to do it.", 'but, more important to me is the message to [inaudible] the survivor’s circle of people.', "it's ... to really listen intently, to try to understand.", "the experiences we've been through ... can change our outlook on life, can change our ways of ... our behaviour.", "so it’s up to the people around us to try to understand what is going on ... because what's happening is not, sometimes ... necessarily under our control, you know, and so i can encourage the survivors to say nanana but ... [break in the recording]"],
                     ['how do you mean?'],
                     ['i remember i came out from under the bed and i went to the window.', 'i saw my father being struck, and i went back under the bed.', 'but i didn’t say anything to my brothers, i was speechless.', 'a few minutes later, someone entered the room.', 'my sister said, “daddy is back.” but it wasn’t our dad, it was someone else.', 'he started searching, rummaging around the room… and when he looked under the bed—there we were.', 'he told us to come out, and so we did, he told us to go outside, and we followed.', 'so we were there and i didn’t know what had happened to my mother, i didn’t know how my father was doing, and all that.', 'i didn’t know what had happened to my uncle or to the servants either.', "we were standing in front and then the man asked me, “are you the eldest child in the family?” i said, “yes, i'm the eldest.” he said, “where do your parents keep the money, the guns, and all that?” i said i didn’t know, i said that i had never seen any guns in the house.", 'i said that i really didn’t know.', 'i told him, “you can ask mom or dad, i don’t know.” then he began to tease me: “oh, those rich kids, it’s always the same thing.” he told many stories and then he said: “anyway, it doesn’t matter if you know or not.', "we’ll kill all of you and then we’ll search the house and we’ll find it.” i said, “i can’t tell you anything because i don’t know.” then he told me, “before we kill you, you’ll take this gun and you will kill your brothers.” i told him, “i don’t know how to use a gun,” that's what i said.", 'and he said, “oh yes, you do know.', 'you, the inkotanyi children, you all know how to use guns.” so he handed the gun over to me, i remember that it was very heavy.', "he said, “so, there, you’re going to kill them quickly, then we’ll kill you, then we'll get on with what we need to do, and then we’ll leave.” i remember that he gave me the gun and i gave it back and said, “look, do what you want, i can’t kill my brothers.” the man said, “oh no?", 'in any case, if you don’t kill them, we’ll kill you all.” and i said, “do what you want, i’m not going to kill my brothers.” so he stayed there and a few minutes later, my mom came out of the house accompanied by somebody else, i think he was the leader of the whole gang.', 'they came out and he said to the guy who was with us, “leave them alone, i made an arrangement with the lady, let them go.', 'i don’t think they’ll go far, anyway…” so my mother said to me, “we’re going to the neighbours’.” we passed through the fence and we went to the neighbours’ house.', 'of course, when they had heard that we were being attacked, they had fled, so when we got there, the house was empty.', 'meanwhile, i think they had told my mother that my father was dead and my mother could not stop crying, and we didn’t understand why… we kept asking her, “mom, what is it?', "what's wrong?” and she kept saying, “it's going to be all right, it's going to be all right.” [tears] [temporary break in the recording] afterwards, when my mother—my mom calmed down later and she told us, “your father is dead.” in the meantime, while my mother was out, my uncle had also left."], 
                     ['i think it’s just group dynamics.', 'bringing together the two ethnicities is not something that can happen in two years or five years.', 'if it is to happen at all, it will take time and there will be a learning curve, and it will be long-lasting.', "i don’t think it's something that should be forced or pushed either, we really have to let time do its work."],
                    ]

In [ ]:
# param - tension_paragraph
# return - ratio of hedge between each pair - this is from tension to tension
def ration_between_tension(tension_paragraph):
    res = []
    for response in tension_paragraph:
        

In [72]:
#To put the tesion paragraph has hedge in tension_is_hedge['hedge'], 
#put the tension paragraph has no hedge in tension_is_hedge['no_hedge']
tension_is_hedge = 0
tension_is_nohedge = 0
tension_is_booster = 0
tension_is_nobooster = 0
tension_is_Neg_Emotion = 0
tension_is_Pos_Emotion = 0
tension_is_Question = 0
tension_is_noQuestion = 0
tension_cuePresent = 0
tension_nocuePresent = 0

In [73]:
flag_hedge = 0
flag_boost = 0
flag_emotion = 0
flag_cue = 0
flag_ques = 0
for p in tension_paragraph:
    for s in p:
        if IsHedgedSentence(s):
            flag_hedge = 1
            break
    if flag_hedge == 1:
        tension_is_hedge += 1
    else:
        tension_is_nohedge +=1
    for s in p:
        if IsBoosting(s):
            flag_boost = 1
            break
    if flag_boost == 1:
        tension_is_booster += 1
    else:
        tension_is_nobooster += 1
    for s in p:
        if get_emotion(s) == "negative":
            flag_emotion = 1
            break
    if flag_emotion == 1:
        tension_is_Neg_Emotion += 1
    else:
        tension_is_Pos_Emotion += 1
    #cueNum = 0
    for s in p:
        for cue in cues:
            if cue in s: 
                flag_cue = 1
                break; 
        if flag_cue == 1:
            break
    if flag_cue == 1:
        tension_cuePresent += 1
    else:
        tension_nocuePresent += 1
    for s in p:
        for qt in ["what", "when", "where", "who", "why", "how"]:
            if len(s) > 0 and qt in s or "?" in s:
                flag_ques = 1
                break
    if flag_ques == 1:
        tension_is_Question += 1
    else:
        tension_is_noQuestion += 1
    flag_hedge = 0
    flag_boost = 0
    flag_emotion = 0
    flag_cue = 0 
    flag_ques = 0

In [78]:
tension_cuePresent + tension_nocuePresent

10

In [79]:
#Z = {hedge, negative emotion, booster words, marker, is a question sentence, cue}
#Set P(z|Xt = tension)
prob_hedge_tension = tension_is_hedge/(tension_is_hedge + tension_is_nohedge) #P(hedge|Tension)
prob_nohedge_tension = tension_is_nohedge/(tension_is_hedge + tension_is_nohedge)
prob_boost_tension = tension_is_booster/(tension_is_booster + tension_is_nobooster)
prob_noboost_tension = tension_is_nobooster/(tension_is_booster + tension_is_nobooster)
prob_negEmo_tension = tension_is_Neg_Emotion/(tension_is_Neg_Emotion + tension_is_Pos_Emotion)
prob_notneg_tension = tension_is_Pos_Emotion/(tension_is_Neg_Emotion + tension_is_Pos_Emotion)
prob_cue_tension = tension_cuePresent/(tension_cuePresent + tension_nocuePresent)
prob_no_cue_tension = tension_nocuePresent/(tension_cuePresent + tension_nocuePresent)
prob_question_tension = tension_is_Question/(tension_is_Question + tension_is_noQuestion)
prob_no_ques_tension = tension_is_noQuestion/(tension_is_Question + tension_is_noQuestion)

In [80]:
print(prob_hedge_tension)
print(prob_boost_tension)
print(prob_negEmo_tension)
print(prob_cue_tension)
print(prob_question_tension)

0.7
0.5
0.6
0.6
0.5


In [28]:
no_tension_paragraph = [['... my name is berthe kayitesi ... i am from gisenyi in rwanda.', 'i was born in gisenyi, but i spent part of my childhood with my maternal grandmother in the former kibuye prefecture ... the former commune of [inaudible].', "... my father's name was [inaudible], my mother's name was mukabilima spéciose, my grandmother's name was [inaudible] valérie.", "i think that’s [inaudible], i think that's all."], ['...i was born on november 11, 1978 ...'], ['in gisenyi.'], ['[laughs] [break in the recording]'], ['... of my mother, i remember her courage in the face of life’s difficulties, she was a fighter.', 'she made sure that everything was going well at home, but she was also very strict, that’s the side of her i didn’t like.', 'but i remember her strong character, her courage, her fighting spirit.', 'that’s my lasting memory of her.', "for example, i remember that every day she would get up at almost five o'clock, five-thirty, and would start working.", 'she advanced ... quite quickly in life.', 'she often regretted not having been able to continue her studies, she always told me: “you know, i was smart, but i was unlucky, so ... you must ... ” [laughs].', 'and also: “i was at the top of my class.', 'how come you allow yourself not to be ... to fail?” so, [laughs] we had to be among the best students in school.', 'and when we weren’t, she was on our case ... [laughs], so to speak.', 'she was also a very beautiful woman.'], ['... my father ... my father had a very gentle character and ... he’d had a difficult past, but he almost never expressed or shared his feelings.', "he was someone very warm, very, very hospitable—how should i say it—it's really hard to describe my father.", 'but ... it’s ... his calm disposition… he also had a certain hardness about him, but he had ... in fact, his kindness ... counterbalanced this hardness, and that’s how we remember him.', 'those who knew him retain that image of my father as someone who was very kind, someone who never complained.', 'i think his father had been unfairly imprisoned for many years, but he never spoke of that.', 'he had left ... well, he kept it all inside and he lived with these feelings, but without letting them affect the family.', 'thinking about it today, i wish he shared with us a little bit more of what he felt inside, but he had decided to keep it all within.'], ['... my father was a tailor, he was one of the first tailors in the region where he was born, in kibuye, gishyita, which he left to settle in gisenyi and continued working as a tailor there, well, that’s what he continued doing until 1994. and as for my mother, she did many things.', 'at one point we had a bar—actually, we had a bar at different times—and she was taking care of it.', 'and shortly before 1994, she ... she was selling things, food, at the market.'], ['yes, banana beer.', 'she had to go get it very early in the morning.'], ["yes, the expression we use is “going kurangura” ... so she would get up very early in the morning, around five-thirty, and when i got up, she would tell me, “before going to school you have to buy ... food for lunch.” so i would wake up around six o'clock and i would go buy ... whatever we were going to eat at noon.", 'and after that i would go to school.', 'upon her return, she would make lunch before going to the bar.'], ['so when i came home in the evening, i would take care of the rest for [inaudible]'], ['... what they were both teaching us, i think it was equal, but for the most part, in terms of household work, it was my mother and me.'], ['... at the time, in 1994, well, when i returned to the house ... after living with my grandmother ... we were ... five.', 'later, my mother, well, my parents, had two more children so we were seven, but one of my younger sisters died just before the genocide.', 'for us, the children, luck was on our side and we ... survived.', 'the six of us survived.', 'but my father had another marriage before ...'], ['yes.', 'so from that marriage, he had three children: two twin sisters who died in 1994 and their mother too, and another sister who survived.', "so with this sister, and with her brother on her mother's side, we found each other afterwards—it’s like being a family of eight."], ["... i think that rwandan culture at the time ... favoured…—not only through education, but also through social discourse—it's as if the boy child was more important than the girl child.", '[laughs] i remember, i was angered by this.', 'but that’s over now.', 'when i was born—so my paternal grandmother, actually, my paternal grandparents had five boys, so they didn’t have any girls.', 'my father was their eldest son, and until i was born, he only had girls.', '[laughs] so when i was born—i was the first daughter of his second marriage—so my grandmother said, “oh, it’s a girl again!” [laughs] and this made me angry.', 'my mother had not picked up on that, but i did, and it made me angry a little bit.', 'in our family too, as the older sibling and as a girl, i had more on my hands than my younger brother.', 'for example, he could just go get water, but i was expected to do the rest ... and so there was disparity, but there wasn’t a ...—how can i say it—... it went without saying: you are a girl and so you do the housework.', "... that's how i saw things and then ... if my brother went out to play ... that’s okay, he can go play, he can go for a swim, i couldn’t go swimming ... [laughs] i had to be ..."], ['yes.'], ["it’s not only an adult's perspective, but also an outlook on rwandan society from the outside.", '... what i appreciate here, i think, is that we try to [laughs] treat both boys and girls in the same way.', 'i don’t know if that’s something that has changed in rwanda, but they’re making an effort to promote equality.', 'i don’t know how things are in people’s homes, but at least in the social discourse we try to support girls as well, and in my view, this attitude produced through social discourse can also change things within families.', 'i don’t know where we are at on this issue, but i think that if i had children i would try to have a more or less balanced approach, even though a boy can do things that a girl can’t necessarily do.', 'but i think it’s a question of encouraging ... the child’s abilities and helping the child discover what it is that they want to do.'], ['... well, yes, in the city where i lived.', 'maybe that wasn’t the case in the countryside, i don’t know.', "i think that women, girls would not be, for example … making bricks to build houses, so that was work for boys, but i'm talking about ... the way things were when i was 15. i think girls had more to do, but elsewhere maybe it was the boys who…, so i ..."], ["but let's say, for example, beyond my personal experience, when i was interviewing people in rwanda, there was a girl who told me, “my mother said that boys can go study, but not me, because i am a girl.” that's not my experience, but ..."], ['someone else’s, but …'], ['yes.'], ['... one graduated from university and works at the bank of kigali.', 'another works for a private company and the others are in school.', 'and even before, in terms of education, our parents ... this was true for all children; so my mother urged me to study in the same way she urged my brother, so there wasn’t really any imbalance there.', 'it was more ... in terms of housework, but in terms of education, all of us, we were all encouraged to go to school.'], ['... i personally don’t remember my paternal grandparents very much, but i do remember my maternal grandmother.', 'when i went to live with her—i don’t know how old i was, i think two years old or less, i don’t know—i remember that time, she gave me breast milk from her own breast until i was grown.', 'and ... i remember that my grandmother was a very sociable person.', 'she had already lost her husband so i never knew my maternal grandfather, but two of her children—my uncle and my aunt—lived in the area and i grew up with my cousins, my uncle’s and my aunt’s children, and i remember that on weekends we travelled far to visit my grandmother’s ... brother.', 'also, she was a farmer, so we had small fields all around the house and that was what occupied my time and what sustained us, but also her children who were close to her.'], ['... nine years old.'], ['if i had a preference [laughs]…, but at that age… i remember when my mother would come to visit [at my grandmother’s], i would always say, “i want to live with my parents.” and my mom would say, “next time.” so it was always “next time,” but i wanted to live with my parents ... but when i arrived there, i ... i felt like a stranger.'], ['in my own family.', 'and i even think that [laughs] in a sense i remained a stranger [laughs].', 'you know, coming from life in the countryside and suddenly finding yourself in a big city where ...—before, i used to go in the fields or in the forest to collect wood for my grandmother—and then i arrived in a city where ... so i felt out of sync, i think, [?]', 'yeah.'], ['well, when you’re a child, you don’t think if it’s easy or not, you just go about your life.', 'i can look at that time now, from the perspective of today, but back then i didn’t think whether it was difficult or not.', 'besides, i adapted very well during the first two years.', 'it was as though i really wanted ... my feeling of nostalgia for my family was strong.', 'yeah.'], ["at my grandmother's house ..."], ['the neighbourhood where my grandmother lived was ... in africa we say that the child belongs to the whole village, so i would go here and there in the neighbourhood and my grandmother was never worried about me.', 'i could go to other families’ houses and so i was always out and about with the other children, herding the cows or fetching wood together and ... so that’s how life in the countryside was in my grandmother’s village.', "but once i moved to my parents' house, it was more like, you must not go ... to such and such ... people would not necessarily know who we were and i couldn’t go to the neighbours’ house like i did at ... like when i was at my grandmother's.", 'there were restrictions ... luckily, sometimes—there were several houses in a ..., well, in a ...—how can i say this?', '[says the word in kinyarwanda].', "it's like several apartments…"], ['yes, a complex of houses and so we would also ... socialize with the neighbours, but it was really different.', 'i think if i had a preference—in fact, i appreciate that i had both ...—but if i had to choose one, i would prefer the countryside, there was a certain freedom there [laughs] that we didn’t necessarily have in the city.'], ['... i don’t know.', 'i don’t know.', 'i learned later on—because i did stay for a short time with my paternal grandparents—i didn’t know them very well, but later, talking to people who knew them, i was told that my paternal grandmother was a very brave woman.', 'during her lifetime, she was someone who was also ... a great fighter.', 'when her husband was in prison, she kept the family together.', 'she made everything look as if ... as if nothing was happening, while it was, after all, a family drama.', "she had to take care of her brother's children who had just lost their parents.", 'and, apparently, she was also quite strict [laughs].', '... so there are these two traits: the courage of my paternal grandmother and also her severity.', 'i don’t know if we can say it’s this or that.', 'i think i have inherited some traits from my father and i also have in me much from my mother’s personality, but i don’t know if my identity has been shaped by who they were or by the force of events, so it’s hard to say ... we are influenced by many things in life.', 'i’ve probably kept qualities from both of them, but what happened to us is also a very important experience, which brought out aspects of our character that would have never come out had we not gone through these events.', "so, it's hard for me to say … to say that my character was formed by this or that, i think it’s a bit of everything."], ["... i'm not the oldest, but i'm like the most ... it's me who’s taken over."], ['... i think that my half-sister…—i connected with her later, after the genocide, two or three years—i think, i don’t remember exactly—but two or three years after the genocide.', 'i had already lived with her in my parents’ house as the eldest of the family and i think it stayed that way.'], ['my half-sisters.'], ['... i knew the twins who died.', "but the other sister, whom i found later, after the genocide, i didn’t know her in 1994. so, it's… but i already knew her name because one day i looked at my father's id card and i saw a name written on it.", 'i didn’t know who that was, but i remembered the name.', 'and later someone said to me, “oh, you know what?', 'one of your sisters survived.” so i began looking for her.'], ['except the two [other sisters].', 'they came to visit from time to time.', 'but not her.'], ['... no, my half-sister works in gisenyi, but the others are in kigali.', 'and then there’s marie-ange who is here with me.'], ['i think i didn’t have ... i don’t remember the things that i hated, but i know i liked school.', '... i liked school very much.', 'as for my relationships with the other kids—we played together.', 'and i remember there was competition between us—who would be the best student in the class.', 'we had to study a lot if we really wanted to have the highest rank.', 'i was doing my best to be among those at the top, especially since i also felt pressure from my mother.', 'i think it came naturally to me—i was learning quite quickly.', 'and i think that when you are a good student relationships are formed more or less easily because the teacher likes you and your classmates do too ... those students who are more or less smart enjoy a certain respect and i think it [laughs] it played…, it allowed me to…'], ['yes, to move forward.'], ['yes ... no.', 'yes and no.', '[laughs] when i took the exam to get into high school, my mother said, “in any case you are tutsi, so i would be surprised if you get a place in secondary school in rwanda.” she asked her brother, my uncle who lived in bukavu, to get me a place in a school in bukavu so i could go study there.', 'i went there and my uncle placed me in 6th grade in bukavu.', 'a few months later ... so i think that when i left [nyundo]—i don’t remember exactly—but i think that when i left, the lists had just been released and i was not on the list of those who had passed the national examination.', "later, my mother sent a letter to my uncle's house saying that i had to go back because there was a second list and my name was on it.", 'so i remember i started high school—i came back to nyundo and i think the others had started classes a month and a half or two months earlier, and so i jumped on board with them.'], ['i think you know it, but you don’t understand it and you don’t dwell on it.', 'so, when i try now to understand what those experiences meant to me back then, i think i knew it, ... but i was closing my eyes, so i think that i was living in denial.', '... already in 1990, there was in gisenyi an operation called operation zulu, so tutsi families—mostly those that were more or less well-off—were threatened and grenades were thrown.', 'i remember that i was the only one in my family who had left home and i hid with the nuns and a few other children from tutsi families near my elementary school; later, we left our hiding place.', "so we could see the ... it's hard to interpret this today.", 'i think the discrimination was there, but we ... we didn’t talk about it in my family and it was happening, it was going on, but it was surrounded by silence, and so we didn’t know ... why were grenades thrown at this or that place.', 'we knew where it had happened, but we didn’t talk about it, and life went on.', 'it was as if something had happened and then—silence.', 'and i think we knew, we knew, but without understanding it or without necessarily attributing to it the importance that ... and our parents were trying to live with it, and that was the example we followed.'], ['actually, for my grandfather, according to the information ... i’ve asked the people who knew most about it and they told me, “someone killed somebody else and put the body in your grandfather’s field, and it was the hutus from the village who did that.” so, it was related to ethnicity.', 'that’s true in my mother’s case too, but she didn’t talk about it that much.', 'we can’t know how she felt about that, what really happened, but there was a connection.'], ['yes, i think that maybe they wanted to protect us.', "but it's… today, we can only look back and ask why we didn’t talk about it.", 'and… in fact, why didn’t we know more?', 'why didn’t we discuss it to at least have a clear understanding of what was going on?', 'but today we are stuck asking why.', 'why didn’t we revolt?', "why didn’t we say, we'll all follow my uncle?", 'why did we accept all of that?'], ['in 1992… 1992–1993.'], ['yes, later on.', 'after the genocide, i did a year in a private school in gisenyi.', 'but later i went back to nyundo.'], ['in 1998.'], ['before 1994, i was taking courses in literary studies (option littéraire) in high school.', 'later i took some courses in general education (l’option normale).', 'i returned to the literary option after that, but it wasn’t literature as such any more.'], ['after high school, i went to university to study psychopedagogy.'], ['in psychopedagogy, my undergraduate degree (bac), yes.', 'not in nyundo, but in kigali [inaudible].'], ['yes, almost.', 'i am in education sciences, i’ve had a career as a student.'], ['no, it wasn’t my choice.'], ['we were profiled in advance.', 'so when i went to nyundo, i was told, “you’ve been placed in literary studies,” so it wasn’t me, really, who decided what to do.', "later, i went back to literature, but that's because that was what i knew.", 'i would have preferred much more to study law at the university.', 'when i was in my fourth year in secondary school, i thought i would go into law, but my grades were not good enough to enrol in the university in butare where i could study law more easily.', 'and the ulk (kigali independent university) had just opened, so i didn’t want to go there.', "that's how i enrolled in psychopedagogy."], ['yes, it was my second choice.'], ['... in fact, i think it was already happening in the gisenyi region.', 'there was this period called zulu.', 'i don’t remember which year, but it was between 1980… i think it was 1993. but at the school where i was …, in high school in nyundo ... very close to the cathedral, the bishop of nyundo at that time, the monseigneur, had given refuge to the bagogwe who had been chased from their region where ... well, from their homes.', 'so when we went to mass in nyundo, we saw those refugees who were refugees inside their own country and who were there not because they were bad people, but because they were tutsi.', 'and so, in gisenyi during this operation zulu…, i remember that after that period, many tutsis who had the financial means left gisenyi and moved to kigali.', 'and there were displaced children in my high school, displaced because of the war of the rpf [rwanda patriotic front], and from time to time they were ... showing hatred for their tutsi classmates.', 'my classroom was ... actually ... i didn’t really see much … i didn’t see anything going on in the classroom.', 'but since we were at a boarding school, we could see that small groups were forming.', 'i remember one time, i was with two other girls, we walked by a small [group?]', 'of children, some of whom had been displaced by the war, and i remember we saw them spit on the ground.', 'but it was just ... it was an insult directed at us.', 'at the junior seminary, there were things ...—the seminary was nearby—things of the same nature were happening there too.', 'people, students, were sharing stories in the evenings.', 'and the atmosphere was starting to get tense because of that.', 'and, i think, when i look back at that time now, these are the kinds of events that foreshadowed the worst to come.', 'but at the same time, i didn’t think about it too much.', 'i don’t know if at some point you get used to living with things like this.', 'but also, not all hutu children were like that.', 'i remember in one of my groups there was a girl, actually, several girls with whom we were often together and we never really noticed any animosity against us.', 'so, those things were ... were there, they weren’t widespread yet, but we could really ... we could understand them if we wanted to see the situation as it really was.', 'but, once again, you had to close your eyes and, you know, ... move on.', 'when 1994 happened i was both lucky and unlucky.', 'my luck was that i wasn’t at home.', 'so, prior to 1994, i had run away from home and i had met some of the nuns from the auxiliaries of the apostolate, after which i returned home.', 'so from time to time i would spend the holidays at the convent.', 'and it was easter in 1994, so i was going to spend my second week of vacation with these nuns.', 'i left home on easter monday and the genocide began on the 7th.', 'but i remember that already on the 6th, the night of the 6th, one of the priests came to pay a visit to one of his friends among the auxiliaries.', 'and as he was there, we were told…—since the routine in this convent was to get up in the morning and go to the church to attend mass.', 'so every morning, we went to mass.', "by chance, this priest was there and the priest said—actually, one of the nuns told me: “since the priest is here, he’ll celebrate mass at the convent, so you don’t have to go to the church tomorrow morning, the mass will be held here.” so we got up to go to the mass and the first thing the priest told us was, “the president’s plane has been shot down.” everyone was ... i was surrounded by adults and later that day, sitting around the table, we continued discussing the news and i remember i said, “it’s over.” today, every time i think about why i said “it's over,” i don’t understand why i said it.", 'and it was indeed over.', 'i don’t know if that phrase was the expression of everything that i had seen until then and stockpiled in my mind.', 'and then something else was going on in gisenyi—there were these big buses that ... big buses that were transporting…'], ['it was the militia, i remember, but i think there was an odd euphemism used for them—people were calling them “the magicians.” so they were people who… they always filled the buses and sometimes, they were on top of the buses and they were dancing, you know, they were always making noise, they were singing.', 'i had seen this frequently in my hometown, so…'], ['yes, before 1994. and sometimes we watched them go by and we applauded—they were the magicians.', 'so, i think all of that made me think “okay, it’s over.” a few minutes later, the nuns were receiving phone calls informing them about the priests from this place and the nuns from that place…, what the situation was like outside, so on that day ... it began, the genocide began.'], ['yes.'], ['no, i stayed in the convent.'], ['[inaudible] [laughs].'], ['and i said “it’s over” once more, but at the same time i didn’t really grasp the magnitude of this phrase, “it’s over,” and what they were saying.', 'i remember i felt such fear around me that i ended up not being afraid anymore.', 'all around me there was ... [laughs] there was really a feeling of terror, everyone was moving in all directions.'], ['there was panic ... you would be talking to someone and then they would go away.', 'people were moving around.', 'i’m trying to visualize it, i was saying, “it’s not possible!” so people were going up to the attic and i saw them coming down, they were looking for félicité.', "“hey, mother, what's going on?", 'do you have news of this or that person?” people would abruptly go into the room, they would go in ... and talk in low voices behind locked doors.', 'others would be listening to the radio ... they were trying to capture [?]', 'to find out what was going on.', 'still others spent time in the chapel.', 'and it was in this atmosphere…, but since i knew the house well… and actually, later the female students of saint-fidèle university also sought refuge at the convent and other people from the city who could reach the saint-pierre centre, came as well.', 'and since i was…, since i knew the place better than others, i continued to help.', 'i helped with the cooking and with the housework.', 'i continued participating in the life of the convent as i did before, so ... i kept myself busy and i didn’t really have time to stop.', 'it’s only now that i stop and become frozen ... at that time, i was much more in the action.', 'félicité made sure that i had things to do, but she prohibited me from opening the door when someone rang—i always liked opening the door, but she prevented me from doing that.'], ['yes, well, i went to see many of them, but she was the nun responsible for all others.', 'but i was worried about my parents and especially about my father because a week prior to that he was attacked by militiamen and by soldiers in a bar, and on the way home we went to see him.', 'my mother brought some money and i ... right away, i don’t know why, but i felt afraid for my father.', 'and my mother was a sociable person and she had a lot of friends.', 'i always had the impression that maybe… and she had an mrmd (the national republican movement for democracy and development) membership card, but my father had an rpf card.', "i remember, when i was in congo, my uncle told me: “your mother has an mrmd membership card, she's really brave!” [laughs] so, i thought that maybe that would save her.", 'because, later, i was afraid for everyone…, but right away, i felt that i was losing hope for my father.', '[wipes tears]'], ['no, life continued.', 'well, i can’t know everything that was going on in the communes, but we knew that the tutsis were targeted, that it was about the tutsis.', 'i don’t remember being hustled by any of the hutus who were in the convent.', 'even though félicité and nathalie went to—in fact, on account of these students from saint-fidèle —i remember they went to get another saint-fidèle student who was injured and was in the hospital and they brought her to the convent where she was treated.', 'and félicité had sent two hutu nuns to my parents’ house.', 'upon their arrival, they found out that my parents had been killed, but they didn’t tell me that when they came back.', 'and i don’t know if they had tried to save them or what, but i know that the next day félicité asked me, “do you know anyone in congo?', 'do you, by any chance, have family there?” i said yes—that was around april 19–20.', 'i told her that and she said “okay.” i think it was on the night of april 19 to 20 when she told me, “don’t go to sleep tonight.” actually prior to that date my brother had come—my younger brother, jean-baptiste, had arrived at the convent.', 'and she told me that on the 7th, on april 7th, a classmate of mine from elementary school, whose brother was taking part in the killings [a génocidaire], had said, “you know what?', "all tutsis will die.” and then my brother had told my parents, “i'm leaving.” and my mother had said, “okay, go to this family.” one of my mother’s cousins had a husband whom we didn’t know if he was hutu or tutsi.", 'afterwards, i received written information confirming he was tutsi, but at that time we thought he was hutu.', 'so, my ... then my mother had said [to my brother], “if you’re going there, take marie-ange with you.” and so…'], ['yes, my sister.', "and so jean-baptiste went with marie-ange to jacqueline’s house, but once there, jacqueline's husband told jean-baptiste: “we'll keep marie-ange, but we can’t keep both of you.", 'do you know where berthe is?” jean-baptiste said yes.', "“okay, go to her.” so, jean-baptiste came to where i was staying, but on the way there, he met someone who asked him: “you are alexis’ son, aren’t you?” my brother said, “no, i'm not alexis’ son.” after that he arrived at the convent and so we were both staying there together.", 'and one night félicité told us not to go to bed.', 'later, around midnight, she came back and said: “okay, you can go to sleep now, we were just told that there are militiamen everywhere and you can’t cross [the border].” she wanted us to cross over to congo where she had sent about fifteen people already and she wanted to try that a second time, but it didn’t work.', 'so until the 21st…, when the militiamen came to the convent, so around 3–4 pm… i was in the kitchen, and félicité came running: “tell everyone to hide, the assassins are here.” i ran around the convent saying, “hide!', 'hide!” i hadn’t even finished saying that when the génocidaires arrived and i was then saying that i wasn’t tutsi.', 'they brought us to … one of them took me along with several others and there was ... they were people all over the place, so the militiamen had ordered many of the people to come out from their hiding places.', 'i went to the exit door and i saw many people on their knees and a ... a killer pointing his rifle in their direction.', 'they told us to kneel and we did.', 'they had two minibuses parked outside the convent and we were told to get on the bus.', 'i fought back.', 'one of the killers was bringing people in ... and i said, “don’t hit me.” félicité hadn’t gotten on the bus yet, and they forced her to.', 'so we left with the first bus and they told us that they were going to take us to the red commune.', 'inside the bus—one of the nuns had been teaching us a song—so i started singing it and then we were told to keep quiet because “you killed president habyarimana, so you don’t get to sing.', 'the only hutu who’s on the bus can continue singing.” so, we took ... the driver took the road to the cemetery.'], ['that was the red commune.', 'and on the way there, we saw that, outside, life for some people was ... continuing.', 'while you are on the bus heading to certain death, right next to you—in any case, for the hutus—life hadn’t changed, the market was open and you could see people in the street; at the time when you were afraid to come out of your hiding place and we were going to ... to the cemetery.', 'and when we arrived, many people ... came to assist.', 'on arrival, i also saw militiamen coming out of the [inaudible] with machetes, even with old, traditional machetes… and in the bus i had…, i had learned the name of one of the ... well, the name of the bus driver ... well, of my killer, a militiaman, and someone was saying ... who was saying, “but thomas, don’t you recognize me?', "don’t you know i'm from such and such a place?” i remember that.", "but i also remember that they didn’t really want to kill félicité, because while we were driving, they said: “the colonel's sister must not go on the bus.” but she had insisted, “if you're going to kill these people, i'm going with them.” at the same time, even though the colonel's sister had to get off the bus, they never forced her out—it was just empty words.", 'so when we got to the cemetery, i thought, “maybe they won’t kill her.” so i got off the bus, took her hand, and we all got out.', 'the others got out and immediately lied down on the ground.', 'one man, about thirty years old, tried to escape, but the génocidaires ran after him and caught him.', 'it’s one of the most vicious images i have from that moment.', 'they caught him and he was held on both sides by two génocidaires and two others ... were ... cutting him, and everyone was watching.', 'so, in the meantime, félicité, who, i imagine, didn’t really want to see that whole scene ... let go of my hand and she turned away…—the cemetery was at the foot of a mountain—and so she gazed up towards the mountain as if in a prayer.', 'so, when she let go of my hand ... i took thomas’ hands, the bus driver.', 'and thomas took my hand too.', 'i saw in the crowd a former classmate of mine from elementary school, and i made a sign with my hand, like that [makes a sign with her hands] to ask for help and he did this [makes a sign of helplessness], meaning, he couldn’t do anything.', 'and while thomas was holding my hand, one of the killers snatched me from thomas’ hands and thomas kicked him.', 'in the meantime we realized that omar, who is now in arusha, was shooting those who were lying on the ground.', 'there was a child there, about two years old, and thomas, who was holding my hand, told me to take the child and go back on the bus.', 'a few of the génocidaires had stayed behind in the bus, there was also another girl in there—she was the child’s aunt, but she didn’t want to reveal her identity, anyways, the child’s aunt…'], ['yes, the child’s aunt, the maternal aunt of the child i was holding in my arms.', 'they were saying that the child’s father was hutu, but the militia didn’t want to hear that.', 'at the moment when i took the child to get on the bus, i saw félicité falling to the ground, and at the same time i saw the priest—the same priest who celebrated mass on the first day, who had stayed with us at the saint-pierre centre—i saw him approaching with three policemen to save félicité, but it was too late.', 'so we got back on the bus and ... and it was over.', 'omar returned to the bus with thomas, the driver, and we started on our way back.', 'while we were driving omar asked thomas what was i doing in the bus and thomas replied: “she just lied to me that she is tutsi.” omar said, “you have to stop the bus, i have to kill her too.” but thomas didn’t ... didn’t stop the bus.', 'we dropped omar off at one of the big hotels in gisenyi, palm beach, where i saw people ... people who were drinking, who were eating, so omar joined them and thomas then dropped me off at the saint-pierre centre.'], ["it's the convent, that’s what it’s called."], ['where we were before we left.', 'and when i arrived there, i saw two policemen and i saw nathalie, who was félicité’s assistant.', 'nathalie asked: “so, is félicité dead?” and the driver, thomas, said, “i saved these two children,” as if…'], ['not to answer the question or: even though félicité is dead, he saved two people.', 'surely, since nathalie was angry and ... she was scolding him, he told her, “but i just saved…” later, in the evening, the policemen who were there, apparently came back to protect me ... and they said … there were people still hiding in the convent who had heard the génocidaires saying, “next time, we will burn it all down.” the génocidaires had said they would burn everything.', 'and i began to regret not having died ... not having been shot dead in the cemetery.', 'but that didn’t happen.', 'the next day, actually, that same evening, several soldiers came by to ask how félicité had died and it was as if she was the only person who had died.', 'i tried to tell them what had happened, but i didn’t know that they had captured several of the killers.', "and the next day, félicité's brother and some of his friends went to dig up félicité’s body and give her a dignified burial, among the thousands of dead bodies.", 'and the people, the hutu sisters who were there, later told us that all the corpses were naked.', 'and on that same evening of april 22, we ... sought refuge in congo.', 'we decided ..., we decided to leave with the few people who were still at the convent.', 'in the meantime, my brother had gone out of the convent and when he came back we told him to come to congo with us, but he refused.', "he said, “i’m not going.” we woke him up around midnight and he said, “i'm not going anywhere.” and i knew that this was a serious mistake—but fortunately he survived…—to leave…"], ['yes.', "he said, “i'm not going anywhere.” so i left without him."], ['so it’s… i don’t know, maybe we could have forced him to come with us since we were quite a small group, but he refused and we all left him there.', "i left with a feeling of guilt, but i'm lucky he survived, which eases that feeling a bit."], ['i think it’s five minutes.'], ['it is, but that distance, during a genocide, becomes huge.', 'today, when i think about it, i wonder: “we had to wait for two weeks before we could cross over.” we were there, we could see congo, so it was…'], ['i don’t know.', 'i think you can be there in ten minutes.', 'normally, it’s three minutes to the border, but if you go through the fields, it may be ten minutes—in any case, it was really close.', 'but during a genocide that distance becomes huge.'], ['no, the people who left were several tutsis who had not been found in their hiding places.', 'the nuns and all the others who were not targeted stayed behind.'], ['no, i was with several other people.'], ['yes.', 'we were about to arrive in a few moments and we slept on the ground in ... we were sitting on the grass, waiting for the sun to rise.', "around five o'clock in the morning, we set out towards the houses in birere, in goma and we arrived at a church, i think it was a pentecostal church.", 'we went inside and ... those who were there could see that we were people escaping death.', 'there was a tutsi woman who was living in congo and she decided to take care of me, and it was as if everyone found a place to go.', "there were several female students from saint-fidèle ... and there was one who was studying at the adventist university at the time and she said, “i'm going over there to join the adventists.” and i … when félicité had asked me if i knew anyone in congo, she was thinking of sending me first to goma, to a nun who had left with the first group, among those who … and i said, “can you take me to the bishop or do you know adria?” and that’s how many of us were sent to different places.", 'i was with immaculée, one of the nuns who was tutsi, and with adria—actually, the nun brought us to the episcopal see in goma where adria and the other sisters came to see me.', 'and we went to adria’s family in goma—there were several of us from the saint-pierre centre.', 'on may 3, the orphanage in nyundo was evacuated.', 'after our departure from the convent, nathalie brought my brother and three other kids whose mother had left with the first group [inaudible] to this orphanage.', 'so when the evacuees from the nyundo orphanage arrived in goma, my brother was among them.', 'i then demanded to go to the orphanage in goma and so we left this family and went to the orphanage.', 'later, in may ... my uncle who was in bukavu, found out where i was and sent his wife to come and pick me up.', 'but he thought that i was the only one there from the family.', "when his wife arrived and when she saw that my brother was also there, she said, “okay, i'll take you both to bukavu.” ... so she signed me out from the orphanage to bring us to bukavu.", 'and on the same day we signed all the papers in order to leave the orphanage and after we left the orphanage and were going to spend the night in goma before leaving the next day to bukavu, ... my friends from the orphanage came to tell me, “your three brothers just arrived at the orphanage with the red cross.” and so i went back to the orphanage.', 'we couldn’t get a place in the orphanage again easily because there were many children there.', 'but with luck, some would leave [and we could take their place].', 'i said, “i’m not leaving” ... they were trying to convince me that i had to leave because the director… that we couldn’t ... that we couldn’t have a place there anymore ... that the unhcr [united nations high commissioner for refugees] had to sort this out.', 'and we went back and forth to ... to the unhcr and ... it wasn’t ... it really was a whirl of people trying to…, i [inaudible] who had other things to do and who told me: “go!', 'the children will stay at the orphanage and you ... you will go to your uncle’s.” i said that was not possible, so i [inaudible] [laughs].', 'at one point, there was a priest who intervened, abbé martin ... abbé martin who knew someone at the unhcr, i think.', "the nuns, my uncle's wife and i were in the process of trying to get permission, and then martin intervened and after his intervention, the unhcr ... allowed me to go back to the orphanage, and so we stayed there.", 'later, we returned to rwanda with the other people from the orphanage.'], ['yes.', 'in the case of marie-ange, she had arrived in congo with a wave of hutu refugees.', 'one of the hutu nuns had gone to a [refugee] camp—i don’t know if she had stayed in this camp or not—she came to the orphanage and told me: “you know what?', 'i just saw your little sister in a refugee camp, the [inaudible] refugee camp.” so, i accompanied this nun and she showed me where marie-ange and the family that had taken her in during the genocide were.', 'and when i arrived, jacqueline said, “you have to be very careful.” so, we found marie-ange who was ... not looking well, she had been jostled around.', 'we took marie-ange from ... from the crowd of refugees and we went back together to the orphanage.', "so, that's how we all reunited."], ['yes, the three [brothers] who arrived with the unhcr, were with our parents when the génocidaires came for them.', 'the children hid behind our bar, but they saw the génocidaires leave with our parents.', 'they didn’t see them die, but they saw them leaving with the génocidaires.', 'and then they tried to find shelter at the neighbours’, but the neighbours turned them away.', 'and then they tried to reach the church, the gisenyi church, and along the way they came across several génocidaires who captured them.', 'and then—i don’t know who among the three…, i don’t remember which one…—i think it was gaétan who said, “we just saw some people looting a shop.” because when the génocidaires captured them, they said, “come, show us your home, show us where your parents are.” and the children had the idea to make up a \u200b\u200blie about a place that was being looted and the génocidaires left them alone [laughs].', 'they were going to loot too, they were going to look for that place where the others were looting, and the children managed to reach the church and the bizeramariya convent, which is next to the church.', 'but in the meantime, the génocidaires showed up to kill the nuns in the convent and i don’t know who among the nuns brought the children to the prefecture and it was at the prefecture that the red cross took them and brought them to goma.'], ["yes, that's at least what my brothers knew.", 'later, when i was in high school ..., after the genocide, before i went back to nyundo—i went to the pentecostal high school ... in gisenyi, the école normale primaire de gacuga [?', "]—and that's where i met someone who had seen them die, this girl who told me, “[inaudible] i saw your mother, they clubbed her until she died, your father was cut down with a machete.” it was ... it seems that it happened on her street and that there were also several muslim men there who were playing gisoro and who tried to plead for my mother’s life, but ... it didn’t help or they were just pretending to want to help—i don’t know, that's what she told me."], ['until 1998, well… i never liked the orphanage, i never…'], ['[laughs] i’m an eternal rebel.', 'i said to myself, “i will not stay here.” you know, you’re in the orphanage and you ... after i left [?]', 'when the notre dame lycée opened in nyundo, and since it was very close to the orphanage, i went back so that i can be closer to my brothers and sisters.', "but when you’re in the orphanage and then you go to high school, and everything is within a well-defined area, so you're really ... it felt like two prisons and sometimes i would just leave, i would go to my uncle’s and my uncle [would say]: “but where is it that you are going?", 'you have to stay at the orphanage.” and i would say, “okay.” so, since my third year in secondary school, i’d had the idea to get out of the orphanage, and i was hoping that it would happen miraculously after high school [laughs].', 'so i studied like crazy in my third year, i never worked as hard in school as i worked that year.', 'after the genocide, i thought, “i’ll finish school fast and i’ll leave the orphanage.” i thought that i would get a job right away and that everything would happen naturally.', 'i made contacts with people outside of the orphanage.', "and later, when i learned that i had a half-sister and that she was living with her maternal aunt in kigali, i went to look for her in kigali and when i found her, she said: “oh, i have a very good friend here and you must meet her before you go back to nyundo.” so, [inaudible] she is my sister's friend and she is really a very warm person and she told me, “if you want to come to my house during the school break, the doors are open.” and she was someone i had just met!", '[laughs] i said, “okay.” so, from time to time during the holidays…, i ... we kept in touch.', 'and 1997–1998 was ... the period of the infiltrators, it was still a dangerous time in gisenyi.', 'they were burning buses, so it really was another war.', 'and i thought, “we can’t die now!” i was knocking on people’s doors, i couldn’t see ... i went to my maternal uncle in bukavu and i said, “we can’t stay there, in this chaos of shooting.', 'it might not seem scary to you, but if our fate is to die now, we should do everything we can so that at least some of us are saved.” my uncle tried to find an orphanage, but he wasn’t able to.', 'and my friends, well, the families of my classmates were also trying to find an sos orphanage in kigali, but we made the mistake of saying that we were leaving one orphanage to go to another and the sos told us, “we don’t take children from another orphanage.” so i didn’t know what to do, but i kept in touch with this woman and she told me, “you can bring your three sisters, marie-ange and, well, at least the youngest children.” and this was not something we did at home.', 'it’s a family ..., well, after a genocide, anything is possible, but when i told my uncle about it, he said: “my goodness!” [laughs] and he turned to my two other brothers and asked: “so, what do you think—your sister is doing well, isn’t she?” but i didn’t care ... i didn’t care about what the others might say.', "i said to my younger brother, “you will sign as if…”—it was clear that my relation with the director of the orphanage was strained because of these comings and goings, so i didn’t dare showing up there and [i said to] my younger brother: “you're going to sign as if it were my uncle, you'll say that it's my uncle who…”"], ['yes.', 'or my uncle was forced to sign, i don’t remember exactly.', 'in any case, it was on behalf of my uncle that we left the orphanage ... actually, my two youngest brothers and marie-ange left the orphanage to go to this other family.', 'i stayed in nyundo, and it didn’t matter whether my other brothers and i were going to finish high school or not, because i was comforted to know that at least some of us from the family would survive.', 'so, jean-baptiste, gaétan and i remained in nyundo since they too were in high school—in the art school and i was in the lycée.', 'and marie-ange, jean-yves and yvan were [inaudible] in primary school in muhima and they went to live [inaudible] with this family that we had just met [laughs].', 'during the holidays, i would visit my youngest brothers and my sister and then i would go back to nyundo, and i was doing that until i graduated.'], ['yes ... and after that [inaudible] [laughs].'], ['there was a sort of madness to it too, i don’t think that if i found myself in the same situation now i would do the same thing.'], ['and then we stayed with this family for some time.', 'but, there were too many of us there.', "she had her five children, there were two other children on her husband's side and there were the four or sometimes the five of us from our family.", "jean-baptiste and gaétan stayed at the orphanage or went to my uncle's house, but the rest of us, we stayed with this family.", 'it was getting more and more difficult, especially because it was the mother in the family who took care of everything.', 'then i learned about the farg and i started going to the farg.'], ['assistance fund for genocide survivors [laughs].', 'i went to ... i started going to ibuka first and one of the sisters-in-law in the family who was hosting us, pauline, was working at avega [the association of genocide widows / agahozo] and introduced us to ibuka so that the children can study for free.', 'and it was there that we started addressing our problem that we were ... that we were nearly homeless.', 'so i started the process.', 'at that time, in the beginning of the farg, i had a friend from high school whose cousin was a secretary at the farg—a secretary who later married the secretary general of the farg at that time.', 'so we opened a file on our case and they said, “okay, we can pay you the rent for a house.” so, we agreed with our host family that we would look for a house nearby so we wouldn’t be far and the family would know what was going on with us.', 'we started looking for a house near where we were ... near donatille’s family who had welcomed us.', 'and so, in 1999, we started living in…'], ["[laughs] that's when we really became children, well, children and one head of the household.", 'we settled there, we had the rent, but that was it [laughs].'], ['yes.', 'we also had to jump through many hoops.', "it's really hard to explain, but it worked [laughs].", 'they told us, “rent a house for no more than 25,000 rwandan francs.” we found one for 15,000 frw, 15,000 francs and so we saved 10,000, but that’s cheating, so it wasn’t ... well, afterwards i didn’t care since we did the deal without ... agreement with the owner of the house.', 'i did the deal with donatille who had hosted us in her house and ... with another mom who was with us … in this together.'], ['later, the owner of the house found out about it and she went to the farg.', 'i told the farg, “i don’t care.', 'you knew that you had given us a house.” her complaints did not amount to anything since she had her 15,000 frw anyway.', 'but life in that house—that’s where things got really difficult and i thought, “why did we leave the orphanage?” at that moment, when we started living by ourselves, i was having regrets.'], ['yes.', 'sometimes i would go to my uncle’s ... who was helping us out, but there were many of us, so it wasn’t really ... it was more or less regular help, but it wasn’t enough.', 'at other times friends also helped, i sent letters to strangers, to people i was corresponding with when i was at the orphanage, and this was more or less fruitful.', '[laughs] today i avoid thinking about that time, because i don’t know how we went through that period.', 'but at the same time, well, when we were in that situation, we didn’t perceive it as alarming, which means that my brain was working all the time: where will i be going today?', "i'm going to ibuka again, and one time they sent me to [inaudible].", "tomorrow i'm going to ... so it was always…"], ['and it was an experience that…'], ['no we continued going to school.'], ['it was around the time when i started university, so i also had to cover transportation costs to the university.', 'my brothers were still at the boarding school, well, some of them were still in primary school, but we had a very good neighbourhood, so there were also [inaudible], there was another family of orphans who were heads of their household, but their parents had died of illness and we were really supportive.', 'and we had fields, we planted things, sometimes, we shared.', 'but it wasn’t ... you know, it was harvest season, so there wasn’t a day when we went to bed without having eaten.', 'but when we would be able to eat was—how can i say this—it was both luck and it wasn’t at the same time.'], ['yes.', 'afterwards, i think my brain worked like ... today, i feel like my brain is frozen, but at that time it was—whew!'], ['i think it lasted about a year, since i ... i also knew that my aunts had survived, my maternal aunts had hutu husbands, and so i went looking for them.', 'and at the time when i went to look for them, i was also ... working on a project of [inaudible] that the farg had ... at one point the farg told us, “we can’t just keep giving you charity, you have to do a project.” so we did a project ... to help ourselves, but that was also the time when i went searching for our maternal aunts.', 'one of our aunts had a good situation at the time, and when she came to see us, she said, “okay, but there’s three of us in the family.', "why don’t you split—everything is going to be simpler that way?” so ... and donatille said to me: “that's not good.” but i was all excited to have found more relatives, and i think that was a serious mistake i made with regards to my younger brothers.", 'so i said, “okay, we can disperse within the family.” we split—i think it was in 2001, so from 1999 until 2001 we were ... or until mid-2000 we were alone.', 'so the aunts…, so we dispersed.', 'i went to live close to the university with my university colleagues and my brothers were in the host families, so at my aunts’ houses.', 'but one of the aunts had aids and we didn’t know that, and shortly afterwards she ... she died.', 'and it was she who, at least financially ... she had a difficult character, but she helped us a lot financially, she supported just about everyone.', "and that's when it became difficult because it was this other aunt who then took care of my brothers.", 'that aunt had a husband in prison, so she had her own problems.', 'and she took in my brothers as well as the children of my aunt who had just died.', 'so the children survivors were living with other children as well as with the children whose father was in prison—even if they were all family, it wasn’t easy.', 'and my younger brothers jean-baptiste, jean-yves and yvan suffered a lot.', 'they stayed there until i graduated from university and, once again, i went to the farg.', 'at one point, it was like i was working at the farg—i don’t know how many times i’ve been there.', 'so after that, after graduation, i went back to the farg and i was able to get the house in kimironko where we are still today.'], ['yes.'], ['yes, one of them even ... the youngest, he often ran away from home and he was saying, “i want to live either with you or with my uncle.” and so i had promised…—when he would run away i would take him in sometimes and he would spend time with my university colleagues because i couldn’t [host him] at the time ... i had one room in the university dorm—i couldn’t.', 'the other children were deeply affected, but we have a good laugh about it today.', 'but things were not going well at all at the time.', 'the second youngest, jean-yves, he was quite affected, while the youngest ... the youngest brother, who was running away—today we laugh about it—but he suffered a lot too, so i don’t know if there will be any further consequences later on, but today we’re able to laugh when we talk about that, and we talk about it from time to time.'], ['we were all there, but we evolved differently.', "so, it's ... i think during the years when we were scattered around, there are things that happened in everyone's life and i think that it—how can i say that—we love each other very much, but it's as if ... it's as if everyone has taken their path.", "so, to say that we shared family moments—such moments were missing [she is crying] ... and even today, each of us is still searching for who we are so it's ... it's again time to…"], ['one works in …, in the [inaudible], another works for an oil company, a third is waiting to get into university and the youngest fell very behind in school and is now in fourth year of secondary school.', 'my half-sister is there… so they see each other, they were telling me ..., the boys prefer to go out rather than just sit at home with the youngest brother, but life goes on ... there’s no comparison with what was before, things are happening ... things are looking up.'], ['yes, i tried to ... the sense of responsibility is there, but it doesn’t weigh as heavily as before.'], ["yes, i'm trying to let go, but for the child who is in his third [fourth?]", 'year in secondary school and ... well, for the two youngest brothers, i still try to be there for them as before.'], ["i think it's both.", 'it’s a situation that makes you vulnerable, so now that i don’t have to spring into action or in my way of doing things sometimes, i realize, “but that’s really childish!” [laughs] at the same time, i also say to myself, “allow yourself to finally be a child, if you can.” still, if i act like that around other people, they might find it strange that at my age i react in a childish manner.', "and so, it's ... i think about that, but these things happen spontaneously, so it's not ... it's not an easy topic.", 'i think i have a divided identity.'], ['yes, i’m swaying between the two.'], ['... i think there were many people like that and ... at different moments.', 'i think, for example, before 1994, when i met félicité and the other nuns, it was initially another nun who introduced me to félicité and she was around 60 years old.', 'at the time i was only 11 and i was dealing with family situations that i didn’t accept and she welcomed me and took the time to talk to me as if she was talking to an equal and she treated me ... treated me with me a very positive attitude, a compassionate and welcoming disposition.', 'and especially ... to also see how she treated everyone, not just me.', 'she was someone who ... who knew ... who took care of some of the vulnerable people in the town.', 'i remember there was an elderly woman who did not leave her home and that nun was taking care of her.', 'she was paying someone to stay with the lady—i think one of the other nuns was staying with her, but she supported the elderly lady in her needs.', 'it was the nun’s simplicity and especially her disposition towards others that were remarkable.', 'and during the genocide nothing in her behaviour changed.', 'and i think that knowing that—despite how broken we were by the events—i think knowing that gave me confidence in life, as if no matter what happens, i don’t have to necessarily expect the worst and i can believe that even one individual among the thousands can spark brightness.', 'later, whether at the orphanage or at school, i was lucky to have teachers who always listened to me, who always appreciated me as a student, well, i could see they loved me and that, i think, carried me forward.', 'also my schoolmates, even when leaving the orphanage, even when ... many people were saying “this child won’t get anywhere.', 'she’s taking her younger brothers out of the orphanage.', 'what will happen to her?', 'she will become mayibobo, she will become ikirara, a street child.” despite that, my classmates who knew me, and even their families whom i got to know through my classmates, were supportive.', 'and then there was donatille and her husband fénariel who welcomed us without knowing us.', 'donatille was someone exceptional, she was ... a very kind, very generous woman who was really there for us during a difficult time.', "we ... it's as if we were sisters, we didn’t know each other, but she regarded me as her younger sister and she loved my little brothers as she loved her own children.", 'so, there were many positive things.', 'today, we’ve moved past this situation and when we look back on it we realize that it has weakened us, but that there were a lot of things that ... that, despite our vulnerability at the time, helped us get where we are today.', "so, i think that's it.", 'life here or there, well ... i was living in that situation, but i can’t say that ... that it was dramatic.', 'perhaps it seemed dramatic on the outside, to the people observing the situation, but i never experienced that.', 'our situation was what it was, i couldn’t stop and analyze it, so i lived with it and i carried on.', "but it's really thanks to ... to all these people that i’ve met.", 'and then, when i arrived here, you were here, emmanuel and callixte were also here.'], ['... actually, after my studies in rwanda, i got a job in [inaudible] in kibuye ... but i liked to study, so i entertained the idea that if the opportunity presents itself, i will continue studying.', 'so, i learned that there were scholarships for students from the francophonie, i applied and i was lucky to get one.', 'the destination was canada.', "so that's how we ... [laughs]"], ['yes, i came as a student.', 'i arrived in trois-rivières and i think that was the moment when i started to look back and to see that it wasn’t ... a normal or an ordinary situation.', 'and that’s ... how i could get in touch with emmanuel and with callixte with whom i spoke, i think, in the first months after my arrival in canada.', 'and that’s how it happened.', 'emmanuel and callixte asked me to share this experience in public and so here i am with you today!', '[laughs]'], ['... i think a big part of me is still there ..., so i think i live here physically while i’m often there mentally.', "and even if it's not there as a physical place, it's there as an experience.", 'so ... all the years i lived in rwanda, that’s what ... made me who i am.', 'i came here as a student, in conditions ... in good conditions, compared to everything i went through there.', 'being here has allowed me to go back…, well, it gave me the time to think about what i had experienced in rwanda.', 'in recent years…, actually, all the years that i’ve spent in canada have been years of reflection.', "i don’t know if i’m really integrating in the culture here [laughs] or if i'm inside my shell.", 'at the same time, i’m starting to have a social network in this country—i have friends in trois-rivières, people from my cohort in graduate school, people from my summer work, people from the place where i teach french in the summer, so i meet a lot of people, and i embrace ... i more or less embrace the culture here, western culture.', 'i also think that having shared my experience on different occasions ... i think that has also allowed me to ... to adapt, in a way, to adapt by accepting … who i am.', 'although i should not remain in this role of a witness, because it’s a vulnerable position, but i ... being here as a student … has also been a kind of a stepping stone for me.'], ['both at the same time.', 'well, there was ... when i arrived, emmanuel wasn’t really here, he put me in touch with callixte.', 'but it was mainly with a couple of friends from france, who were also foreigners in this country, that i built a very strong friendship, right from the first months of our studies in canada, and these friends later came to rwanda and i went to visit them in france.', "so, with them it was ... and with other québécois friends who joined our group towards the end of the master's degree.", "it was ... it was much more the community ... so i think it's both, it's the canadian community, well, the western community, but also a large part of the rwandan community.", 'actually, there wasn’t a rwandan community in trois-rivières, but i was in contact with page-rwanda, although not regularly.', 'and then i met josée, josée gicali, who had known my paternal grandparents whom i didn’t know, and it…, it helped create a very strong bond between us.', 'so she also helped me integrate in the first months, and later i expanded my horizons elsewhere.'], ["i think…, well, i’ve never compared them, it's a bit of a difficult question, but i think it's different.", 'most of the rwandans i meet here are survivors so ... obviously [?]', '... [laughs] we speak ... our language.', 'i remember the first time i was in quebec city before meeting other rwandans, it was ... it was wonderful to see ... i was at this place where they were performing dances from different cultures—it amazed me to see my culture in quebec.', 'most often, when rwandans get together ... we talk as if we’re still in rwanda ... we also talk about our experiences during the genocide, we talk about everything, actually.', '... in the company of canadians, we talk about a bit of everything as well and we share our experiences too, but our experiences are very different.', "so, they’ll talk to me about vacations and i'll talk to them about rwanda.", 'maybe i will slip in a few words about my experience ... during my first months in canada, that’s all i was talking about and i felt overwhelmed, so today i don’t talk about it much.', 'but at the time, i spoke about it often even to canadians and to my french friends so it wasn’t ... but they are removed from my experience, so the conversation takes on another shape and there is a very big gap between what they tell me and what i tell them.', 'still, they were listening, they were very ... they didn’t rush or pressure me, well, i have been pressured, but not ... not on the part of these friends.'], ['i don’t know [laughs].', "... it's a question of mutual acceptance.", "sometimes we can be suspicious, thinking that canadians haven’t lived through such major events, but ... they also have things to share ... i think it's a question of ... opening up to the experience of others.", 'but it shouldn’t go in one direction only: just because we’ve had a painful experience doesn’t mean that we should not listen in return.', 'i think that the history of canada also contains some traumatic events and i think that sometimes we must accept to open ourselves to positive experiences.', "and especially when we’re talking about a genocide, i think, it's an issue, a phenomenon that is of concern to everyone.", "so during those conversations, it's not only about listening or accepting my experience, but it's much more about using this experience in relation to others, since, i think, genocides continue to happen, but they don’t happen twice in the same place.", 'they always happen to other people, other places.', 'so, i think that not only canadians, but everyone has to become aware of our experience.', 'that’s the greater usefulness of witness accounts— if possible, to protect others in other places.'], ['[laughs] ... i don’t know.', "it's a difficult question ... i don’t know since ... i think that all rwandans, well, every rwandan has his or her own experience and i’m not sure that i should be asking them to think about me in a certain way.", 'perhaps i should be asking how we should all consider each other.', 'i think everyone lives their experience in their own way, and we should maybe try to listen to each other and abstain from judging each other.', 'what upsets me the most, i think, is to be judged: “oh, you’ll become mayibobo.” the perception that others have of you can really weaken you and you might think, “oh, maybe ... i really am like that.” we live in a society and, unfortunately, one downside that i ... that i’ve noticed often about our community is that we’re always looking on a surface level and ... not everyone is like that, but in general it seems that we’re focusing on what is wrong with the other person.', 'so, in this case [inaudible] we need to always make the effort to bring out those qualities in others that can help them advance, not the ones that can knock them down.', "i've been knocked down to the ground enough times that i don’t see the point of, for example, sitting around and, you know, criticizing others: “you know what?", 'oh, now [inaudible] is constantly drinking.', 'but why do they drink?', "it's ... [inaudible] of orphaned children, it's a catastrophe!", 'all they do is make babies and do drugs.', 'why do they do drugs?” why don’t we try to find out what’s wrong, instead of…, i mean, to try to focus on the good things.'], ["i don’t mean to say that it’s always the case or that everyone is like that, but it's something that i have noticed occasionally."], ["it's also normal that we are more curious about the things that are not going well, but it's just a matter of going further, of ... not stopping at that ... you can ... in the orphan village where my brothers and sisters live there are, say, two or three young mothers.", 'when we see that the girl is pregnant, we might think that, for example, only orphans get pregnant.', 'but that’s far from the truth!', "and it's not just the orphans who drink a lot.", 'people whose parents are still alive drink too.', 'for me, that’s …'], ['there you go.', 'so that’s something that often makes me mad and i don’t have an outlet to express this anger, but it makes me mad.'], ['i feel very nostalgic for certain places and people.', 'i’m always afraid when i’m in rwanda because i know that there are killers walking in the streets.', 'well, people who have killed back then and i don’t know if they won’t kill again.', "so i'm afraid.", 'but it’s also with great joy that i return to the places i frequented for the most part of my life.'], ["... not at my grandmother's, i haven’t been able to go back there, but every time i visit rwanda, i go to gisenyi, i go to nyundo, i visit the orphanage and i stay in kigali."], ['no, because when they were killed in 1994, their bodies were immediately thrown into a mass grave dug in the cemetery, so we don’t know where they are and since it was in a cemetery, we’re not able to disinter the bodies.'], ["no, but what i did find out, although i don’t know if it's true, is that one of the killers died in congo.", 'a gentleman came to see me after the genocide to ask me to testify against someone who had been imprisoned and who had taken part in the killing.', 'but when he came to see me…, well, i wasn’t there when my parents died, so it was difficult for me to testify against whomever ... i didn’t know that person, i had never seen him.', 'but for the moment, we are looking for the bodies of my twin sisters who died in bisesero.', 'we haven’t found the place yet, but we’re looking.'], ['no, and for the time being, i don’t think i want to, anyway, at this point in my life, i don’t think i want to meet them.', 'i know that my brothers sometimes go back there.', 'and when i go back to gisenyi i ... can’t bring myself to ... i went once where we…—actually twice, once to see if there were any photos left and a second time with my friends from france to show them the place.', 'but we didn’t ... i didn’t stay long.', 'i saw the neighbours who are still there, but i’m not really interested in ... staying in contact with them.'], ['yeah, i don’t want to meet them, so i don’t know if i would have the strength to go to the gacaca, but one of my brothers has gone.', 'the people from the neighbourhood where we used to live were denying that we had lived there and were saying that they didn’t know our parents, so my brother had to go there.', 'and while he was there words were exchanged that didn’t really make sense.', 'the neighbours were saying, “oh yes, that’s right, you are the son of [inaudible].', 'oh, i wasn’t here during the genocide, i was ...”—i don’t know where they said they were.', 'and so they started admitting that our parents really lived there, that we lived there.', 'but they were also saying that everyone had moved away by the time my parents died.', "it's a difficult subject.", 'i think that the survivors would like to know more about certain aspects of what happened—for example, where the bodies are.', 'but i don’t think that it really…, i don’t know if it has a healing effect.', 'someone who’s actually been at the gacaca … will be able to say more about it, i think.', 'since i’ve never taken part in the gacaca, i can’t say ...'], ["it concerns me personally and collectively speaking, so ... if i try to understand the politics behind it... i don’t really understand it ... i can’t find the words, but ... personally, i'm opposed to forgiveness, i’m opposed to reconciliation.", 'that doesn’t mean that my heart is filled with hatred and that i want the killers to be killed, no.', 'but i think i prefer ... to live with whomever i want to live, whether they are hutu or tutsi, but that it is me who decides.', "so if we choose to live with each other, we know we'll have to deal with those questions and that’s a personal choice.", 'but i can’t see myself meeting someone who has killed my father, who has cut him with a machete, and who says to me, “i cut him with the machete, forgive me.” so, i think ... i think, i won’t be going any further with ... the gacaca.', 'i see it as a policy that perhaps restricts choice.', 'if i had to give a choice—the people who held the machetes and who were killing for months and months should be relocated and live elsewhere.', 'i think that the survivors of the rwandan genocide are a sacrificed generation, they see their killers day after day.', "i think it's like ... inadvertently, this might be reinforcing their denial.", 'with a crime like that ... justice has to be done first and i don’t know if the gacaca bring real justice.', "and for this too it's not really up to me to say, because i don’t have direct experience with it."], ["it's really a question of solidarity and ... understanding each person’s experience in its singularity, even though it’s an experience shared by many survivors ... i think."], ['i really don’t know if i would live there.', 'until i graduate, i’ll continue going back and forth.', "i don’t know where i'm going to settle in this world [laughs]."], ['... i think i need to continue with the process of recovery.', 'i love rwanda very much and i go back as much as i can, but for the moment i’m not planning to live there.', 'that doesn’t mean that i won’t think about it tomorrow or in the future.'], ['not really.', "but, it's a question of how i feel when i'm there and how i feel when i'm here—that's what will determine my return there or my choice of another place to live."], ['i didn’t think about the questions [laughs].', 'i didn’t really think about the questions.'], ['yes, i know, but, no, i think ... i think we covered everything ... thank you.', '[laughs]'], ["... at times, it feels good to talk about it, but the moments right before, the preparatory stage, if you will, when we know we're going into the conversation, is not always easy, especially when we talk about it often and when we become ... overly solicited [for interviews].", "it's like ... it's as if we [inaudible] since we continuously go back to it, even though there are variations in the story, we often recount the same things.", "at the same time it's up to me to decide not to talk about it.", 'but sometimes, because we’ve become used to it, we can’t bring ourselves to say no.', 'but ... at a certain point, it’s mentally exhausting.', '[end of the session]'], ['(y): okay.', "my name is  ishimwe, i was born in kigali in a family of four children and i'm the eldest, i have two younger brothers and one younger sister and… that's it."], ['yes, that’s right.'], ['i remember really well my maternal grandparents.', 'i didn’t know my paternal grandfather, because, as far as i know, he died when i was a little child, or even before i was born.', 'but i knew my maternal grandmother who also died when i was—if i remember correctly—10 years old.', 'so i remember her vaguely, yeah.'], ['whether i think about my maternal grandparents or my paternal grandmother, i remember the summer vacations, because we went there every summer.', 'i remember how impatiently they waited for us and i remember especially the traditional dishes that they would make for us, even if we didn’t like eating that food very much.', "but they would prepare them with so much love… that's the memory i have engraved in my mind—how they wanted to make us happy, even though i— [laughs] i didn’t like the food very much, but it’s something that i remember, yeah."], ["i know that mom was her dad's favourite.", '[laughs] other than that—well, it was a pretty normal relationship.'], ['as i mentioned, we visited them every year and every time someone i knew was going to their house, i always took the time to write each one of them a letter, telling them how i was doing, how my life was going, asking them for news about themselves.', 'i know i took the time to do this at least three times a year.'], ['my father was a doctor and my mother always worked for an ngo, a non-governmental organization that helped orphaned children or children who came from families—who couldn’t take care of themselves.', 'she was project coordinator there.'], ['yes.'], ['my mother worked during the day, while we were at school, so when we came home after school she was always there.', 'sometimes, my dad worked at night because he was a doctor, but that wasn’t his regular schedule, it didn’t happen often… so almost every night we were at home with our parents.'], ['it depends on the activity.', 'i remember that my mother taught me how to do “girls’ stuff”—how to sew, how to make the bed, how to fold clothes, how to sweep, how to clean, how to… these are things that my father never taught me, it was my mom who showed me that.', 'while my dad—he was telling me about history, he was showing me some technical stuff—that was more his domain.', 'in terms of school work, my father was helping me with math and related subjects and my mother, with language and other related subjects.', 'so i knew that for math homework, i could ask my father for help and when it was something in french or kinyarwanda, i could ask my mother.'], ['as parents?'], ['i would say that as parents they were very strict.', '[laughs] i don’t know how you say that in french—the kind of parents who would look at you from the corner of their eye and, without them having to say anything, you would know what mistake you’ve made and you would correct the mistake at the same time.', 'it was—in any case, i think—not that i regret it, but i think we had a very strict upbringing.'], ['yes, we were spoiled as children, but that didn’t mean that we weren’t punished when we made mistakes.', 'but when we weren’t causing any trouble, we were pampered.', 'we ate out at the restaurant at least once a week, on saturday nights.', 'we rented movies—not during the week because those were school nights.', 'and we went to the park on saturday evenings.', 'we went on airplane tours.', 'we were more than spoiled, i would say.'], ['i started kindergarten when i was three years old and when i was five, i think, i started elementary school.', 'i finished it at 12. i started high school at the age of 13. so, the first four years of school was the common core education, without specialization.', 'later, when i started fifth grade—since i was in a high school in benin—we had to choose a specialization in fifth, sixth and seventh grade, and i continued in accounting.'], ['when the time came to make a choice, i didn’t know what [laughs] to choose, but i was—because i was quite good at math, i went into accounting.', 'at first, i wanted to study medicine, but i didn’t like—for that i had to take physics and i hated physics.', 'so i decided to study something that was easy for me and i didn’t have to deal with physics, so i went into accounting, and that’s how it happened.', '[laughs]'], ['in the neighbourhood, i would say that what “well-to-do” meant was relative, but we were—some families were richer, others poorer, so i would say ours was maybe in the middle.', 'at school we were—i went to school with the children of ministers and so on, so i would say that my classmates’ families were much better-off than us.'], ['no.', '[laughs] no.', 'so the way we were taught was that our parents would come to pick us up after school and we went straight home.', 'we weren’t allowed to go anywhere unless our parents came with us or unless we were going to visit cousins or family friends with our parents.', 'the only friends we had in the neighbourhood were our neighbours with whom we could talk over the fence.'], ['i’ll express my personal opinion about this.', 'i’d say that it wasn’t only a question of education in the sense that parents didn’t want their children to socialize with—i’d say, with just anyone… i think they were also concerned about the children’s safety.', 'these are the two reasons that come to mind.'], ['actually, after the genocide, the ngo where my mother worked opened a branch in benin.', 'i don’t know how it happened and my mom had to—in any case, she was transferred from rwanda to benin and we had to leave with her.'], ['the moment that always comes to my mind is when my mother told us we were leaving.', 'she told us—because we were going to school at the time, i think it had been a few weeks since the beginning of the school year—she told us, “you’ll have to study french harder, because in your new school you’ll be studying in french only.” the idea of leaving was exciting to me, but the fact that i was going to be in a school, in an environment where we only spoke french and that maybe i wouldn’t be able to express myself correctly was scaring me.', 'apart from that, the other [?]', 'was leaving the people i’ve grown up with, my cousins, and knowing that i wouldn’t be able to see them anytime i wanted, that was also bothering me a little.', 'but i was delighted by the idea of \u200b\u200bchange and exploring something new.'], ['for sure, the beginning was difficult for me and my brothers and sister precisely on account of the language because we were in a place where we didn’t speak the language, so it wasn’t easy.', 'we arrived in january, so right in the middle of the school year, and we had to adapt, stay on track, learn french, adapt and at the same time not fall behind, which was much more difficult.', 'it was also a cultural shock, because these are two completely different cultures, and settling down was difficult too.', 'but i don’t know if it was because we were young or because i myself was young, but i’d say that it didn’t take too long before i felt almost at home there.'], ['yes, i remember that when i arrived in the school i was introduced to the students in my class, and everyone was trying hard to help me, to explain things, to give me their notes to copy, and this felt good because when you go someplace new, you feel quite—you think to yourself, “will everything be okay?” and all that.', 'but when people would come up to me at recess and say, “don’t worry, you can copy my notes”—the teachers would also come up to me and say, “come to see me if you need explanations”—that was reassuring.', 'but the fact that we were far from our family, far away, and that we didn’t know almost anyone there, that part wasn’t easy.'], ['thirteen years old.', 'i celebrated my 13th birthday—it is in january—so i celebrated my 13th birthday there.'], ['yes, i started high school there.'], ['[laughs] i still wonder to this day.', 'because not only she was working and taking care of the children, i think it also fell on her to do the greatest share of the work of finding out how things functioned in the new place.', 'to this day, i don’t understand how she did it, with four children.', 'if i were in her place, i don’t think i would have succeeded.', 'i’m still asking myself this.', 'i think she is the right person to ask this question to.'], ['yes, many.'], ['not just from my mom [?].', 'but from her i think i have that go-getter attitude, i must have inherited that from her.', 'the resourcefulness, always wanting to go forward despite the obstacles on the way—this comes, i’m sure, from my mother.', 'another trait is the love for rwandan culture—that, i think, one hundred percent is from her, one hundred percent.', 'that little bit of calm disposition that i have, i think it’s from my mom too… [laughs] reading—i don’t remember if my dad liked to read much, but i know my mom loves it, so i have that from her as well.'], ['from dad?', 'first, him being a doctor, he was more interested in mathematics, chemistry, things like that and so i must have gotten that from my dad.', 'apparently, my dad also had a strong character, so that too must come from him… what else?', 'the side of me that wants success not only for myself but for the people around me as well, the side that likes to help others, the generous side—that, i think, i also carry from my dad.'], ['that’s correct.'], ['i’d say that when experiencing such atrocities i don’t think you make the effort to think about what will happen tomorrow, you live day-by-day.', 'actually, not even day-by-day, you live in the present moment, really, and you wait to see what happens next.'], ['i think that even me, even though i was young, i saw some signs, but i could have never, ever imagined that such a thing would happen.'], ['for two or three years before 1994 the national security in the country wasn’t good, there were always demonstrations.', 'the fact that at school the hutus were asked to stand up and people from the other ethnicity were also asked to stand up and both were counted—that was one of the signs.', 'and the fact that we had already been attacked once at home before 1994 also had to be a sign that something was wrong, yeah.'], ['yes, they would ask the students from one ethnic group to get up, they would count them and tell them to sit back down and then ask the students from the other ethnic group to get up, and so on.', 'and there was a form we had to fill in and we had to indicate our ethnicity, our names, our parents’ names, where we were born and so on and the question about ethnicity was also on there…'], ['i don’t know how i came to know it, but i did [laughs], i knew it.', "it's a form that we’ve had to fill in since we were young, when we were in kindergarten, i think, we were given that form to fill in at home and bring back.", 'and when we were in elementary school we would fill it in ourselves, and if we didn’t know how to fill in certain parts, we had to ask our parents and the next day we would bring the missing information to our teacher.'], ['personally, i was too young, i didn’t notice that.', 'it’s possible that was the case, but personally i didn’t realize it.'], ['so, how it happened… i remember, it started in the evening, we were having dinner, it was around 8 o’clock in the evening, my paternal uncle was there with us, on his vacation.', 'we were at the table, having dinner, and then we heard the sound of the plane.', 'i didn’t realize it was the sound of the plane because we were always hearing that kind of noise, grenades, here and there, and i thought that it was just the usual.', 'then my father asked us to go to bed—actually, not to bed but to go under the beds, because every time this happened, we were told to hide under the bed until my parents would come and tell us to… and then, a few minutes later, we were told: “come out from under the bed and go to sleep.” then they woke us up only the next day—because i was getting ready—i remember, i was preparing to write the final exam at the end of high school [translator’s note: or elementary school?]', 'and i was taking classes—it was easter vacation, but i was taking extra classes to prepare for my exam.', 'so when they came to wake us up, i got up quickly to go take a shower, because i thought i was late for school.', "and my dad followed me and said “no, you’re not going to school today.” then i saw that they were laying down mattresses in the corridor, and my father told us, “sit down, we'll explain what's going on.” so they laid the mattresses down, they arranged all that and said, “in fact, the president's plane has been shot down and things are not going well at all for the moment, we're going to stay here until the situation improves.” i was so young that i didn’t understand how great the danger was.", "i said, “so i'm not going to school?” my father said, “no, you’re not going to school until things get better.” i said, “okay, no problem.” and so we stayed in the house.", 'the plane was shot down on the 6th at night and we stayed there until the 11th.', 'it was on the 11th that we were attacked in the house.', 'i remember that when it happened my father went out, we stayed behind—no—yes, we stayed behind—my mom and my brothers, my sister and uncle.', 'just before he went out, my father said, “let’s sit together and pray.” we took a few minutes to pray and then my father went out of the house.', "my mother told me to get my brothers and sister and go hide under my parents’ bed, then she closed the door and said, “i'll come to get you soon.” we stayed there, but i could hear noises, there were noises."], ['they had tried to cut off his arm and he was bleeding; we tried to stop the blood with a cloth.', 'we stayed in the house, we waited for the neighbours to come back, we just stayed there, and then, much later, they returned.', 'so the man—in the house where we were—he said, “you can’t stay here because they know you’re here, and if they know it, we’re next.” and then even i said, “where do you think i can go with the kids?” he begged us to leave and then he said, “okay, no problem, you can sleep here tonight, but tomorrow morning you’ll have to start making arrangements and see where you can go.” so, we went to bed and i think that during the night, my mother and my uncle went to the neighbours’ to see if there was a place for us to stay, but—and i found that out the next day—that plan didn’t work.'], ['and i know that in the house where we were, the whole family was about to leave and go to butare.', 'so mom asked them, “can you take us with you?” but the man said, “listen, we can’t,” and so on.', 'my mother then asked if they could at least drop us off at the house of my mother’s older brother who lived in kiyovu.', 'then suddenly it started to rain.', 'it was raining heavily.', 'now that it was raining, the man said, it was certain that the guards at the roadblocks would leave their posts and so we could go, but he would only drive us to my uncle’s.', 'so we got to my uncle’s house, we knocked on the door, we knocked, we knocked and there was no one… fortunately, there was a small opening at the bottom of the fence.', 'my mom said to me: “go and see if there’s anyone in the house.” i passed through and i went upstairs.', 'my uncle and his family weren’t there, only the guard whom i knew and who said: “listen, unfortunately, you can’t stay here because there is someone watching the neighbourhood and he knows that there are two of us here in the house, if they find you here, they’ll kill us all.” so i went back and told that to my mom and the others.', 'in the meantime, my mother had an idea—my uncle’s family had a friend who worked at the belgian red cross, which was not too far away.', 'so mom said, “you’re going to drop us off there and the red cross probably—they will shelter us.” so, they dropped us off there and they left.', "once we arrived, we asked for the friend of my uncle’s family, he came out, saw us—he knew us—and he said: “unfortunately, they don’t shelter refugees here.” he said, “there’s another branch of the belgian red cross further down, we will send you there.” my mom said: “with all the roadblocks, we won’t be able to pass through.” and he said, “okay.” he took one of the cars of the red cross, we got in, and he said, “listen, we're going to pass through as if we”—we were hidden at the back of the car—“we're going to pretend that it’s a red cross vehicle passing… if they stop us, unfortunately there is nothing we can do, but if they don’t stop us, great.” but it was still raining so every time we came up to a roadblock, it was empty, so we passed through and they left us there."], ['they dropped us off at the red cross and there were other refugees living there, about sixty or so people, families, and so on.', 'so they left us there and that’s where we stayed.', 'in the beginning, everything went well, we had food to eat, but sometimes there wasn’t any food left and they told us that we would divide the work by family.', 'each family—they said that we were going to have a schedule of people who would do the cleaning, people who would do the dishes, people who would cook—who would go get food, people who would go get water.', "because i was the eldest in the family, i was told, no problem, i can help with the cooking, but for those going out—my mom couldn’t go out, and so they said, “look, you're the eldest, you have no choice.” i said, “okay.” anyway, i was young—getting water?", 'no problem!', 'but we had to get water for bathing, for washing the dishes, for cooking.', "so i said, “listen, i don’t mind going, but how will i manage?” they said, “no problem, we’ll think of something.” then they said, “we'll do two rounds because not everyone can go out at the same time.” and since they designated only one family member per activity, i said: “i’ll go out in the first round and will come back in the second, it’s not a problem.” someone was surveilling the neighbourhood.", "apparently, they were going to kill us at the very end, after they had finished killing everyone else, we were to be the dessert, that's why—because no one was protecting us at the red cross, and there was this man who told us that they were going to kill us in the end."], ['there was a roadblock right at the entrance of the red cross; so every time we went out to get water, this man accompanied us, we would get water and come back.', 'so i would leave with the first group, give the water to my mom and come back with the second group, and so on.', 'one day we went out and they started shooting at us.', 'when shots are fired, everyone starts running here and there.', 'i remember i ran, i went into a house, i didn’t know where i was, i climbed a tree and sat there and waited.', 'they were shooting, shooting, shooting... at one point, i don’t know what happened, but the shooting stopped.', 'quietly, i climbed back down from where i was, in the tree, and i was looking for the man because i knew that he was going to protect me—and i had been tracking his movements with my eyes.', 'so when everything calmed down, i climbed down and followed the man.', 'everybody came out of their hiding places except one guy who was shot and we took him back to the red cross.', 'after that, we just continued on our way to get water and then we returned to the house.', 'i think my mother was scared and she said: “i don’t think you’ll be going alone next time.” so from then on, my brother went with me.', 'and we continued going as usual.', 'one morning, apparently someone from the red cross had fired bullets at the roadblock at the entrance.', 'so the interahamwe came and said: “we won’t wait until the end, after all.” we were all taken out of the house, we were taken outside and they said: “say your prayers now, we will kill you.” we were all sitting in a row, i remember i was with my mom and i don’t know where my sister and my brothers were, but in any event, we were the last ones in the row.', 'i don’t remember where my brothers were.', 'we started praying, and then they killed the first person in the line, i think it was a mother with her baby, then the second, and then the man who was surveilling the neighbourhood showed up.', 'he said, “but why are you killing these people?” and they said, “wasn’t it them who shot at us?” the man said, “no, no, no, i know this house, i’ve searched it, there are no guns here,” and stuff like that.', 'anyway, he said, “i told you, we’ll kill these people towards the end.” and so they let us go, we went back into the house.', 'this happened, i think, twice.', 'the first time, they killed people, but the second time one person was wounded, but no one died.', 'so we stayed there and i continued going out as if that was my normal.', "i said to mom, “don’t worry, i'm not going to die.” i don’t know why, but i was sure i wasn’t going to die."], ['all three [translator’s note: or five?]', 'of us stayed there.', 'i went out every day to fetch water, to get some food with the others.', 'even when my brothers didn’t come with me, i would still go, because i was sure that nothing was going to happen to me.', 'i went out every day.', 'i saw the killings in some places, but i had gotten to a point where it didn’t shock me anymore, they could shoot someone and i could pass nearby without a problem.', 'as long as they didn’t touch me—that’s what was scaring me.', 'my mother would ask me, “have you seen the dead people?” and i would say, “they are everywhere.” it has become just a normal thing for me that one person would kill another; i saw a woman being raped and it seemed normal to me.', 'all i was doing was to go out, get water and bring it back, because i didn’t want my family to starve.', 'then finally, one morning we woke up and the roadblock was gone.', 'someone looked out the door and the people who were guarding the roadblock were no longer there.', 'then i think someone turned on the radio—the radio wasn’t working.', 'we wondered what was going on, and, i think as a reflex from the war, we said, “let’s get ready, something is happening, we should leave.” we got dressed, we started packing a few small things, because we thought we were going to change locations, and while we were getting ready a soldier arrived.', 'i remember my mother cried out, “it’s the inkotanyi.” [laughs] and my mom said, “don’t take too many things with you because they [the soldiers] have arrived.” i remember i was packing and i was asking, “mom, should we take this?” she was saying, “no, no, no, don’t worry, we’re safe.” [laughs and tears of joy] and then she said to me and my brothers, “don’t worry, everything is fine.” i remember we were the second ones to go out, there was one guy in front of us and then me and my mom, and she said, “go where this gentleman tells you to go, because it’s fine, we’re safe.” [laughs] then they took us to a place called [inaudible].', 'everyone from kiyovu was there, well, from the neighbourhood anyways.', 'they took us there, they gathered us to tell us that the war was over and that we were going to stay there until they found a safe place for us to go because there were grenades everywhere and it wasn’t safe.', 'i remember that they brought us food, they gave us to eat, and told us that the next morning we were going to kicukiro.', 'we were to walk from kiyovu to kicukiro.', 'i said, “mom, how are we going to walk?” [laughs] mom said, “now that we’ve gotten this far, we’ll do whatever it takes, if we have to walk, you’ll walk.” mom said, “in any case, don’t take too many things with you because we’re safe and these people will feed us at the place where we’re going.” the next day they told us, “it’s time to go.” we gathered our things, yes, and we set off for kicukiro.'], ['and everyone—each family was assigned to a house.', 'we were told, “this is a safe place, you’ll stay here a few days until we secure the rest of the city.” i think we stayed there for at least—i think a week, the equivalent of a week.', 'after that they said, “you can now go back to your homes.” when we returned to the house—i wasn’t—my mother was accompanied by someone, i don’t know who—apparently, we couldn’t live in our house because it wasn’t a pretty sight—in any case, i didn’t know what it looked like.', 'we were told to go sleep at my aunt’s house.', 'when they had brought us all together in that place, we reunited with one of my aunts.', 'she had lost her children.', 'she was my godmother.', 'she [yvette’s mother] told us, “you, your sister and your brothers are going to stay with your godmother and i will stay with your other uncle” with whom we had also reunited.', '“we’ll clean the house and once it’s ready you’ll come too.” i think i stayed with my aunt for two days, the time it took to put the house in order a little bit, and then we returned home.'], ['speaking for myself, i think that it was like the two sides of a mirror: i was a spoiled child who had suddenly become an adult.', 'i realized that life would never be the same.', 'i don’t know how i understood that, but it was like a mirror, yes.', 'we stayed there, we tried to clean the house—the places where there was blood, all the books that were on the floor because they had ransacked everything.', 'we cleaned the house and, fortunately, not long after, the ngo where my mom worked restarted—they opened for business again and mom had to go back to work.', 'since i was the eldest in the family, i had to take over the responsibilities of the house, i had to make sure that we had something to eat, i cooked, i cleaned the house… i had to get my brothers organized—i knew that this brother had to get up and clean this room or that brother had to get up and do something else.', 'i knew when we had to go get water and i learned to clean and cook by myself.', 'i was trying to remember what our servants did when i was little and when i was watching them work.', 'i was improvising.', 'in any case, i know that when mom would come home from work, everything was in order.', 'how did i do it?', 'i don’t know.', 'but i knew that it had to be done, because mom was working, she couldn’t be in two places at the same time, so i learned all of this by myself.', 'of course, when mom was home, i would sit next to her, ask her questions: “how do i do that?', 'how is this done?', 'what do you think i should do tomorrow?” she would tell me, but i knew that i had to take on responsibility for everything to be in order when she came home from work.'], ['i think we went back to school—if i remember correctly, it was two months later, i think in october?', 'yes, i think it was october when we went back to school.', 'two months later.'], ['yes, i went back to the same school.', 'of course, i didn’t know 90% of the faces there.', 'i remember there was a girl i went to elementary school with.', 'when i saw her after the war—she was the first one i saw—i think we hugged and we stayed in each other’s arms for a long time because she was the only one i knew from school from before.', 'and to see her alive, it felt good.', "and i remember there was a teacher who came up to me and said, “no, no, no, you can’t be alive.” i said, “yes, i'm here.” there was also a boy who was back—i remember we were in fifth grade together before the war.", 'i didn’t recognize him because they had cut him all over his body.', 'i didn’t recognize him, and the girl asked me, “did you recognize ange?” i said, “that’s ange?', 'no, i didn’t recognize him.” i felt so bad for not having recognized him, someone who had been part of my life from before, who felt much closer to me after the war than the others, especially those returning from outside of the country—he was like [part of?]', 'my family.', 'not recognizing him really affected me deeply.', 'but, for sure, when school started, those of us who were in rwanda after the war, we stuck together.', 'the people who came back from burundi, they kept their distance.', 'and so did the people who came back from uganda.', 'that’s how it always was.', 'and i remember our conversations: “how did you survive?', 'what happened?', 'how are you doing?', 'are your parents still alive?', 'are your sisters still alive?” that was it, at least for the first two weeks we only talked about that.', 'about nothing else.'], ['sixth.'], ['yes.'], ['i think, from the teachers who had taught me before, there was only one at the school after the war.', 'only one was there.', 'there were five sixth grade classes and we had about five teachers, and i knew only one of them from before.'], ['i imagine—i think that it wasn’t easy not only for the students but also for the teachers, because not only—i’m looking for the word in french—guhahamuka—'], ['the trauma, that’s it.', 'it was a recurrent thing.', 'you would be studying and all of a sudden you would hear someone scream—things like that.', 'it affected us, but i think it affected the teachers more.', 'they not only had to ensure the well-being of the students, to teach them in class, the class material and all that, the knowledge they had to transmit, but also take care of these children who were not doing very well.', 'so it was a combination of these three things, and i imagine they themselves were not feeling—anyway, i think that, psychologically, they weren’t able to combine these three roles.', 'i guess they had to do it, they had no choice.'], ['trust—my thinking was, “i have to go to school” and “life goes on, you have no choice.” whether you trusted others or not, you had to fight, you had to go forward.', 'whether you liked it or not, you thought to yourself, “yes, what happened, happened.” now i understand: whether we like it or not, life continues, we had to fight to survive and continue to live.'], ['in elementary school, this teacher—the fact that after the war, he came up to me and said, “you are alive?', "i'm glad you're here.” that touched me.", 'it felt like there was at school, besides my friends, an adult that i knew or at least someone i could talk to, someone who knew me from before.', 'and what he said was reassuring: “if there is anything i can do, i’m here for you, i can help you.” it affected me, but it didn’t—it touched me, but not so deeply.', 'for me, at the time, that wasn’t essential.', 'i told myself that i had to live and the rest was not that important.'], ['i could see how my mom was fighting and i was thinking, “if she’s fighting like that, me, as the eldest in the family, i have—,” i thought i had the duty not only to continue going to school in order to succeed, but also in order to help my mom.', 'i was thinking that she couldn’t continue going it alone.', 'that’s what motivated me.', 'and i was also thinking, “i am lucky to still have my mom, there are people who have lost their mothers.” so if i still have her and she is taking care of us, i thought “i have to do the impossible to move forward, to succeed and to lighten her load.” that’s what motivated me, because i remember that before the war, i went to school for the sake of going to school.', 'i remember that if i placed 19th or lower in my class, that was a perfect score for me.', 'as long as i wasn’t 20th, because i thought that being in 20th place was low, but up to 19 was perfect.', 'i had never wanted to be first, second, all that.', 'i wasn’t concerned with this because what mattered for me at school was to see my friends.', 'and since i knew i was going to be scolded at the end of the semester [if i had bad grades], i would do everything i could so as not to be 20th in my class.', 'but after the war, right after the war, i was first in my class because i understood better than before the use of school.', "that's why i said that before and after was like the two sides of a mirror."], ['no.', 'no, not yet.', '[laughs] i hope to, someday, but no, i haven’t reached it yet.'], ['no, we never found his body, we were not able to bury him.', '[people say?]', 'that he’s in either nyanza or kicukiro[?', '], but we really don’t have any confirmation about that.', 'we tried—we know that the person who killed my father is still in kigali.', 'apparently, during the gacaca—i wasn’t there—he was presumed innocent.', 'the trial is still going on, it’s not over yet.', 'but to date, unless he accepts his wrongdoing and tells us what happened—where he [the body of yvette’s father] is now, so far we don’t know.'], ['speaking for myself?', 'i don’t think i would have the strength to pursue this, i don’t think i have the energy.', 'i appreciate the people in kigali who follow up on this, it’s good if they do it, but personally, i don’t think i have that energy.', 'the only motivation that would make me do it is to find out where the body of my father is, but the rest—attending the gacaca and all that—it’s good, but i don’t think i have enough energy to do that.', 'even if i were there, in rwanda, i don’t think i would have the energy to do it.'], ['ideally, the killer would admit it and he would tell us exactly what happened after we left the house, so that we know.', 'that would be the only way.', 'or, if all of a sudden people who were there at the time and who know what happened—if they tell us… that would be the only way.', 'but if the only way, really, is to find out through that person, i doubt that i would be able to do it.', 'not me.'], ['i did, i went back in 1997.'], ['no.'], ["it's a good question.", 'i don’t know.', 'we think we’d meet them, but meeting someone who says “no, i didn’t do that,” yet we know they have, it’s also—in any case, i am reluctant.', 'i am not so sure.', 'but sometimes i think to myself: “i would like to meet this person and look at him.” to be face-to-face with him, to at least ask him a few questions.', 'but i think it would anger me more, it would make me even more—if he says, “no, i didn’t do that,” i don’t know if i could handle it.', 'knowing that he’s done it and that he says no, that’s what i doubt very much.', "i find it’s too—i think it's the highest level of viciousness.", 'we’re not asking for much, we want to know at least where the body is so we can bury it.', "but to meet with someone who might know where the body is and would not say, i'm not sure that i could handle it."], ['i lived there for a few months before going to benin, but i think i was more—i didn’t really think about what had happened.', 'i was trying hard to look on the bright side, to see my family that i hadn’t seen, my uncles, my aunts, my cousins… i don’t think that—i was trying not to mention it.', 'and to this day, i’m still not sure if i would go back, i don’t know what it would be like.'], ['yes.', 'but when, i don’t know.'], ['whether i like it or not, rwanda will always remain my country.', "that's one thing.", 'two, i still have relatives there.', 'but the other thing that i really want to do one day—and to have the courage to do—is to go, but not only to be there, but to be there in april, that would be ideal.', "i would like to relive that month as i lived through it in 1994. maybe it would be a kind of therapy for me, i don’t know, but that's what i'm going to do, try to live through—to remember what i was doing every day—or at least live for a few days there during that month."], ['visit the places, go to the memorials, return to the neighbourhood, yes.', 'all of this is what i would like to do.', 'when?', 'i don’t know.'], ['i came to montreal in 2001. it might sound crazy, but i have always loved canada.', 'why?', 'i don’t know.', "i was hearing montreal french being spoken, and i had a cousin who lived here and he told me, “listen, it's nice here, it's good, you'll enjoy it, come.” when i arrived, i came for—i said, “i'll try it for maybe a year.", "if i like it i'll stay, if i don’t like it, too bad.” and now, it’s been almost nine years.", '[laughs]'], ['yes, i would say yes, i loved it.'], ["it's like i say, it was the first time that—pardon me—that i was leaving my family, and i thought it was a crazy thing to do.", 'i wasn’t even eighteen years old yet, it was an adventure.', 'it was absolutely an adventure.', 'i didn’t know what i was getting myself into, i didn’t have any expectations.', "i thought, “i’ll go and then we’ll see.” i had no expectations, i didn’t know what i was getting into and i thought, “i'll go, if it works—great, if it doesn’t work—hey, we'll try something else.”"], ['i think it was hard for her to let me go because it was the first time she was separating from one of her children.', 'i don’t think she realized it until the last day or maybe she was hoping i would come back.', 'but i was thinking, “not being next to her doesn’t mean that i’m not helping her.” and i was also thinking: “i want to go there, i want to leave for—.” as the saying goes, you have to take a step backwards in order to make a better jump forward.', 'in my mind, that’s why i did it.', 'it was crazy, because i was young and i didn’t have a precise plan and all that, but it was really with the intention to move forward and become someone, to be able to reach my goals, that was the reason why.', "it was really a crazy thing to do, leave your parents while you are still under 18, go to canada, a country, i don’t know how many thousands—from your parents, it's crazy.", 'i don’t think that my children would do something like that.', '[laughs]'], ['because i was with my cousin, i had someone who was helping me do this and that.', 'that was helpful.', 'also i was still a minor and i went back to high school.', 'that was easy.', "that's what made my integration easier because when you start school, you make contacts.", 'i started school, i was at the end of high school, so i finished high school.', "and i remember, i started in grade 10 and when the school year finished everyone was looking for work and so i said “let's go!", 'let’s find a job.” i remember i didn’t know where to start.', 'i remember i was buying something at zellers and there was a line-up, a long line.', "and i thought, “these people are probably looking for cashiers.” [laughs] i went home, printed my cv, went back to the store, gave it to them and said, “i think you're looking to hire people.” i told this to the person there because there was a long queue.", 'and it was true.', 'i handed in my cv, say today, and two days later they called me, they interviewed me and i started work on monday.', 'so i worked there during the summer and in september i went back to school.', 'they kept me working part-time because i was going to school.', 'i went to school in the mornings and i was working a few evenings and on the weekend.', 'there was a project with the bank of montreal [bmo] through my school, the bank wanted to integrate young students into the financial sector.', 'so they would go into montreal schools and select a maximum of three students per school, and they chose me.', 'after they selected me, i did the internship once a week.', 'i worked at the bank as an intern and i learned in this way what the job of the people at the bank entailed.', 'and when i graduated from high school, they called me to ask if i was looking for a job.', 'i said yes.', "the woman who was coordinating the internship told me, “i’ll keep your cv, if you’re ever looking for a job.” and when they had a job opening she said to me: “listen, there are jobs that have become available, i'm going to send your cv, so expect us to call you if you are selected.” finally, they called me and i got the job at bmo, and i worked there.", 'so that was it, my integration.'], ["i'm not complaining.", 'it wasn’t that difficult.'], ['i was at école secondaire la voie and it wasn’t so difficult because i understood the language, people were speaking and i could understand very well what they were saying.', 'it wasn’t that difficult.', 'except for my shyness, which always gets in the way... but it was—i was doing so well that in the second year i was president of the student council, which probably means that if they had chosen me, i must have done pretty well, i’d think.', 'no, it wasn’t that hard.', 'i would say it was easier than the first time, but i’d also say that the language facilitated things, because i understood the language, so i went [inaudible].'], ['yes, precisely.'], ['it was easier, yes.', "it was really multicultural, it's true.", 'but i remember that the assistant director there also helped me a lot, because she would often stop by to ask me if everything was going okay: “are you having any problems?” i think she came to see me every day, and i found that also reassuring… when there’s someone you can turn to if something goes wrong, i think that also reassured me, yes.'], ['i don’t think that she did the same thing for everyone, but in any case not a day went by without me seeing her—i don’t remember.', 'of course there were days when she wasn’t there, when we had activities, but especially in the beginning, she would see me every day.', 'she would stand in front of her office, you know, and when the students came out of class and when i passed by i would say hello, she would call me and ask, “are you okay, are there any problems?”, things like that… she was my resource person, if i had any questions.'], ['yes, to the assistant director—i didn’t go into too many details, but i told her that i was rwandan.', 'she asked me if i had lived through the genocide, i said yes, that was all.', 'but i remember one time, it was during history class i think, we were talking about wars and all that, and they asked me to tell them what had happened in rwanda.', 'and again, i didn’t go into details, but i told them what had happened, and i said a few words about my own experience, so yeah, i’ve told the story at least once.'], ['it certainly changes something when you know what someone has lived through, especially if it has been something difficult.', 'as long as we don’t gloss over it and we don’t feel sorry the person… but for sure, it changes something.'], ['i did, i participated in one organization.', 'i participated once.', 'if i remember correctly, it was during the first summer, i was already working, and there was a youth centre across the street from where i worked, and i said to myself, “i’ll go and see what they’re doing there,” and i went in.', 'they said that, yes, they do activities to help young people integrate and out of curiosity i said, “i’ll join in.” i was—we were a group of young people, yes, we did some activities, we did small jobs.', 'i didn’t stay long, i think i stayed only for the two months in the summer.', 'i was just curious to see what they were doing.'], ['no, not at all.', 'not really.'], ["yes, i'm part of insangano."], ["it's a dance group where we do traditional rwandan dances and other activities.", 'we have a community newspaper, we organize christmas parties for the children, and so on.', 'and i’ve been part of insangano, i think, since 2003—no, 2004—i’ve been part of insangano.', 'i’m also part of page-rwanda, since 2005–2006, so those are the two community groups to which—i was also a member of the founding committee of the rwandan cultural centre, and i think that’s all.'], ['i know there’s another dance group, ihozo.', 'but i don’t know much more about them.', 'but i know they exist.'], ['absolutely.', 'yes.'], ['does one have to be rwandan to notice it?', 'because i am rwandan, i don’t know if people who see—but as a rwandan, yes, i am aware of it.', 'does one have to absolutely be [rwandan] to understand it?', 'i don’t know.', 'maybe yes, i don’t know.'], ['we did this as part of insangano, we had what we call tuganire, we sat down and we talked about what had happened.', 'and not only about those events—we could address all kinds of issues.', 'but we often talked about what had happened in rwanda, about education today compared to the education of our parents or our grandparents, yes, we often referred to that.'], ['yes, there is the urumuri mothers association, i imagine that they also do this kind of gatherings, yes.', 'but that’s all i know.'], ['i left bmo not long after 2003, and since then i have been with bell, since 2003, and i continued my studies in accounting, management and accounting.', 'i think i will be working more in the financial sector.'], ["i don’t think it’s been a failure, but i think—i hope—that i'm on the right track.", 'there is still a long journey ahead of me.'], ['what bothers me and affects me the most is the negationism, the fact that despite everything that has happened, that has been recorded on camera, the people who have lived through it, the victims who continue to testify, there are still people who dare say that there was no genocide—i don’t support that.'], ['when i watch movies about genocides—of course, a film can’t tell the full story of what happened during those three months and all over the country—does it represent it really or are we trying to represent reality… i think more work remains to be done on this.'], ['to be an actress?'], ['my god, no!'], ['[laughs] no, the cameras—i don’t think—i don’t like cameras and they must not like me either.'], ['personally, no.', 'i can give ideas on a project that someone else would like to carry out, but i don’t think i have the talent for that.', 'one never knows, but for the moment, i don’t think so.'], ['about me?'], ['the future is based on the past.', 'this means that, whether i like it or not, what happened is going to influence what i am doing now and what i will be doing in the future.', 'this means that what i have seen and what i have lived through will be reflected in my every decision or every action.', 'and i think to myself that, yes, what i’ve lived through was an atrocity, but at the same time it awoke a part of me that keeps telling me that life did not stop, life goes on and that we must not just survive, we must live.', 'whatever the obstacles you encounter in life, whoever you meet on your journey, don’t hesitate to move forward.', 'and i think—as they say in english, last but not least—i think that if i am who i am, if i am where i am, it is all because of god.', "and i say to myself, “everything i've lived through, everything i do, everything i know—if i didn’t have god, if i didn’t have prayer, i wouldn’t be here.” there’s no doubt about that."], ['my pleasure.'], ["thank you, that's very kind.", '[smiles] (end of interview session 1 of 1)']]

In [62]:
notension_is_hedge = 0
notension_is_nohedge = 0
notension_is_booster = 0
notension_is_nobooster = 0
notension_is_Neg_Emotion = 0
notension_is_Pos_Emotion = 0
notension_is_Question = 0
notension_is_noQuestion = 0
notension_cuePresent = 0
notension_nocuePresent = 0

In [63]:
flag_hedge = 0
flag_boost = 0
flag_emotion = 0
flag_cue = 0
flag_ques = 0
for p in no_tension_paragraph:
    for s in p:
        if IsHedgedSentence(s):
            flag_hedge = 1
            break
    if flag_hedge == 1:
        notension_is_hedge += 1
    else:
        notension_is_nohedge +=1
    for s in p:
        if IsBoosting(s):
            flag_boost = 1
            break
    if flag_boost == 1:
        notension_is_booster += 1
    else:
        notension_is_nobooster += 1
    for s in p:
        if get_emotion(s) == "negative":
            flag_emotion = 1
            break
    if flag_emotion == 1:
        notension_is_Neg_Emotion += 1
    else:
        notension_is_Pos_Emotion += 1
    #cueNum = 0
    for s in p:
        for cue in cues:
            if cue in s: 
                flag_cue = 1
                break; 
        if flag_cue == 1:
            break
    if flag_cue == 1:
        notension_cuePresent += 1
    else:
        notension_nocuePresent += 1
 
    for s in p:
        for qt in ["what", "when", "where", "who", "why", "how"]:
            if len(s) > 0 and qt in s or "?" in s:
                flag_ques = 1
                break
    if flag_ques == 1:
        notension_is_Question += 1
    else:
        notension_is_noQuestion += 1
    flag_hedge = 0
    flag_boost = 0
    flag_emotion = 0
    flag_cue = 0 
    flag_ques = 0

In [64]:
notension_is_hedge + notension_is_nohedge

225

In [65]:
prob_hedge_notension = notension_is_hedge/(notension_is_hedge + notension_is_nohedge) #P(Hedge|Not_tension)
prob_nohedge_notension = notension_is_nohedge/(notension_is_hedge + notension_is_nohedge)
prob_boost_notension = notension_is_booster/(notension_is_booster + notension_is_nobooster)
prob_noboost_notension = notension_is_nobooster/(notension_is_booster + notension_is_nobooster)
prob_negEmo_notension = notension_is_Neg_Emotion/(notension_is_Neg_Emotion + notension_is_Pos_Emotion)
prob_notneg_notension = notension_is_Pos_Emotion/(notension_is_Neg_Emotion + notension_is_Pos_Emotion)
prob_cue_notension = notension_cuePresent/(notension_cuePresent + notension_nocuePresent)
prob_no_cue_notension = notension_nocuePresent/(notension_cuePresent + notension_nocuePresent)
prob_question_notension = notension_is_Question/(notension_is_Question + notension_is_noQuestion)
prob_no_ques_notension = notension_is_noQuestion/(notension_is_Question + notension_is_noQuestion)

In [66]:
print(prob_hedge_notension)
print(prob_boost_notension)
print(prob_negEmo_notension)
print(prob_cue_notension)
print(prob_question_notension)

0.6177777777777778
0.48444444444444446
0.38666666666666666
0.28444444444444444
0.6311111111111111


In [81]:
#transition probablity
p_nottension_to_tension = 8/234
p_tension_to_nottension = 8/234
p_nottension_to_nottension = 216/234
p_tension_to_tension = 2/234

In [95]:
#hmm model

def hmm_tension(paragraph_corpus):
    Tension_prob_history = []
    Not_Tension_prob_history = []
    Result = {'Tension' : [], 'No Tension': []}
    flag_hedge = 0
    flag_boost = 0
    flag_emotion = 0
    flag_cue = 0 
    flag_ques = 0
    for i in range(len(paragraph_corpus)):
        isNegativeEmotion = False
        isHedging = False
        isQuestion = False
        isOutlier = False
        isBoosting = False
        cuePresent = False
        prob_Zt_Xt_Tension = 1
        prob_Zt_Xt_Not = 1
        for s in paragraph_corpus[i]:
            if IsHedgedSentence(s):
                flag_hedge = 1
                break
        if flag_hedge == 1:
            prob_Zt_Xt_Tension *= prob_hedge_tension #P(Xt = Tension | Zt) = P(hedge | Xt = tension)
            prob_Zt_Xt_Not *= prob_hedge_notension # P(Xt = Not Tension|Zt) = P(hedge|Xt = Not tension)
        else: # If this is not a hedge sentence
            prob_Zt_Xt_Tension *= prob_nohedge_tension #P(Xt = Tension |Zt) = P(no hedge | Xt = tension)
            prob_Zt_Xt_Not *= prob_nohedge_notension #P(Xt = Not Tension | Zt) = P(no hedge | Xt = not tension)
        for s in paragraph_corpus[i]:
            if IsBoosting(s):
                flag_boost = 1
                break
        if flag_boost == 1:
            prob_Zt_Xt_Tension *= prob_boost_tension
            prob_Zt_Xt_Not *= prob_boost_notension
        else:
            prob_Zt_Xt_Tension *= prob_noboost_tension
            prob_Zt_Xt_Not *= prob_noboost_notension
        for s in paragraph_corpus[i]:
            if get_emotion(s) == "negative":
                flag_emotion = 1
                break
        if flag_emotion == 1:
            prob_Zt_Xt_Tension *= prob_negEmo_tension
            prob_Zt_Xt_Not *= prob_negEmo_notension
        else:
            prob_Zt_Xt_Tension *= prob_notneg_tension
            prob_Zt_Xt_Not *= prob_notneg_notension
        for s in paragraph_corpus[i]:
            for cue in cues:
                if cue in s:
                    flag_cue = 1
                    break
            if flag_cue == 1:
                break
        if flag_cue == 1: 
            prob_Zt_Xt_Tension *= prob_cue_tension
            prob_Zt_Xt_Not *= prob_cue_notension
        else:
            prob_Zt_Xt_Tension *= prob_no_cue_tension
            prob_Zt_Xt_Not *= prob_no_cue_notension
        for s in paragraph_corpus[i]:
            for qt in ["what", "when", "where", "who", "why", "how"]:
                if len(s) > 0 and qt in s or "?" in s:
                    flag_ques = 1
                    break
        if flag_ques == 1:
            prob_Zt_Xt_Tension *= prob_question_tension 
            prob_Zt_Xt_Not *= prob_question_notension
        else:
            prob_Zt_Xt_Tension *= prob_no_ques_tension #P(Xt = Tension |Zt) = P(no hedge | Xt = tension)
            prob_Zt_Xt_Not *= prob_no_ques_notension
        if i == 0:
            prob_Zt_Xt_Tension = 0.5
            prob_Zt_Xt_Not = 0.5
        else:
            prob_Zt_Xt_Tension = prob_Zt_Xt_Tension *(p_tension_to_tension * Tension_prob_history[i-1]
                                                     + p_nottension_to_tension * Not_Tension_prob_history[i-1])
            prob_Zt_Xt_Not = prob_Zt_Xt_Not *(p_tension_to_nottension * Tension_prob_history[i-1]
                                             + p_nottension_to_nottension * Not_Tension_prob_history[i-1]) 
        if prob_Zt_Xt_Tension < 1:
            prob_Zt_Xt_Tension *= 100
            prob_Zt_Xt_Not *= 100
        elif prob_Zt_Xt_Tension > 10000000000:
            prob_Zt_Xt_Tension /= 100
            prob_Zt_Xt_Not /= 100
        
        if Decimal(prob_Zt_Xt_Tension) >= Decimal(prob_Zt_Xt_Not):
            Result['Tension'].append(paragraph_corpus[i])
        else:
            Result['No Tension'].append(paragraph_corpus[i])
        Tension_prob_history.append(prob_Zt_Xt_Tension)
        Not_Tension_prob_history.append(prob_Zt_Xt_Not)
        flag_hedge = 0
        flag_boost = 0
        flag_emotion = 0
        flag_cue = 0 
        flag_ques = 0
    print( "Tension_prob_history",  Tension_prob_history)
    print(" Not_Tension_prob_history",  Not_Tension_prob_history)
    return Result

In [96]:
hmm_tension(BertheKayitesi_ans)

Tension_prob_history [50.0, 3.8461538461538463, 3.988788531332966, 17.532570569263086, 72.56429281833721, 1.4151324159488143, 1.2115589176434443, 9.078775880868122, 31.691056970315874, 93.41720520894464, 4.702173153662422, 55.2943443624107, 6.762176317837373, 9.96314618488633, 75.00121208347046, 3.107983591399903, 1.1545143395550725, 11.661159478752941, 35.78077650433037, 1.177341661075419, 0.6688053788385616, 1.9461064921129665, 20.068178909809106, 38.0914259089981, 1.8594427318669153, 3.4714738852546922, 15.261239161412618, 74.40774314306431, 41.52778685096407, 1.7716207963702222, 12.822742041203359, 14.971625235161099, 18.92509472759898, 15.265962115099875, 1.1184986165958113, 0.9664035313096395, 9.894702814362043, 14.571441543490666, 42.80573879486292, 11.31171925557008, 4.240495682326743, 42.84042851096204, 3.1135668966667365, 6.481503374166701, 8.335975440066532, 26.370144462670947, 86.18918779653553, 2.540206886736641, 7.484433876120137, 22.052023562583653, 97.46062628322406, 1.

{'Tension': [['... my name is berthe kayitesi ... i am from gisenyi in rwanda.',
   'i was born in gisenyi, but i spent part of my childhood with my maternal grandmother in the former kibuye prefecture ... the former commune of [inaudible].',
   "... my father's name was [inaudible], my mother's name was mukabilima spéciose, my grandmother's name was [inaudible] valérie.",
   "i think that’s [inaudible], i think that's all."]],
 'No Tension': [['...i was born on november 11, 1978 ...'],
  ['in gisenyi.'],
  ['[laughs] [break in the recording]'],
  ['... of my mother, i remember her courage in the face of life’s difficulties, she was a fighter.',
   'she made sure that everything was going well at home, but she was also very strict, that’s the side of her i didn’t like.',
   'but i remember her strong character, her courage, her fighting spirit.',
   'that’s my lasting memory of her.',
   "for example, i remember that every day she would get up at almost five o'clock, five-thirty, and w